In [1]:
#RUN IN SAOD or SAGEO (sageo has a problem with pandas min function)
from __future__ import annotations
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import geopandas as gpd
import scipy.io
from functools import reduce
from pyproj import Proj, transform #can run this fram sageo
import rasterio
import rasterstats
from netCDF4 import Dataset
import shapely
import rioxarray
import xarray
from functools import reduce
import glob
import re
import math
import folium
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.ticker as mticker
from matplotlib.transforms import Affine2D
from matplotlib.lines import Line2D
# from geocube.api.core import make_geocube
from shapely.wkt import loads, dumps
from shapely.ops import polygonize, split
from fiona.crs import from_epsg

from typing import Callable
from scipy.spatial.distance import pdist
import skgstat
import xdem

import skgstat as skg
# import geostatspy.GSLIB as GSLIB                       # GSLIB utilies, visualization and wrapper
# import geostatspy.geostats as geostats                 # GSLIB methods convert to Python
# import os                                               # to set current working directory 
# import sys                                              # supress output to screen for interactive variogram modeling
# import io
# import numpy                            # DataFrames
from matplotlib.pyplot import cm                        # color maps
from ipywidgets import interactive                      # widgets and interactivity
from ipywidgets import widgets                            
from ipywidgets import Layout
from ipywidgets import Label
from ipywidgets import VBox, HBox

# from osgeo import ogr
from shapely.geometry import MultiLineString, Point, LineString, Polygon
from shapely import wkt
import sys

pd.set_option('display.max_columns', None)
shapely.speedups.enable()

# from jupyterthemes import jtplot
# jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False)

%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# date_string='24July'
date_string='Summer2022_v13'

In [2]:
#read in fluxgate lintes
flux_lines = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/vector_files/Arctic_marine_terminating_fluxgates.gpkg')  

#Make points every distance along your fluxgates or lines
distance=25 #change this distance based on how frequently you want points
export_flux_gate_points = pd.DataFrame()
inProj = Proj(init='epsg:3995')
outProj = Proj(init='epsg:4326')
counter=0
for i in range(0,len(flux_lines)): #iterate through all your fluxgate lines and create points every distance along them
    x1,y1 = flux_lines.loc[i].geometry.centroid.x,flux_lines.loc[i].geometry.centroid.y
    x2,y2 = transform(inProj,outProj,x1,y1)
    projection = ccrs.Orthographic(central_longitude=x2, central_latitude=y2)
    flux_lines_proj = flux_lines.to_crs(projection)
    easting = []
    northing = []
    creation = []
    region = []
    for x in range(0,int(flux_lines_proj.loc[i].geometry.length),distance):
        x,y = flux_lines_proj.loc[i].geometry.interpolate(x).xy   
        easting.append(x[0])
        northing.append(y[0])
        creation.append(flux_lines_proj.loc[i].creation) #save other attributes
        region.append(flux_lines_proj.loc[i].Region) #save other attribute        
    #read the data to a pandas dataframe
    lists = {'region':region,'creation':creation,'easting':easting,'northing':northing}
    df = pd.DataFrame(lists)
    #read the data into a GeoDataFrame
    flux_gate_points = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.easting, df.northing))
    flux_gate_points.crs = projection#{'init' :'epsg:3995'} #define projection
    flux_gate_points = flux_gate_points.to_crs('epsg:3995')
    if counter == 0:
        export_flux_gate_points=flux_gate_points
        counter=counter+1
    else:
        export_flux_gate_points=export_flux_gate_points.append(flux_gate_points)
        
export_flux_gate_points.loc[:,'easting']=export_flux_gate_points.geometry.x
export_flux_gate_points.loc[:,'northing']=export_flux_gate_points.geometry.y
export_flux_gate_points.to_file('flux_gate_points_25m_'+date_string)#change file name based on distance

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. See: http

In [3]:
#Read in RGI and fluxgate points (on and off glacier) data
flux_gate_points = gpd.read_file('/Users/willkochtitzky/bin/Jupyter_notebooks/Arctic/Arctic_discharge/flux_gate_points_25m_'+date_string+'/flux_gate_points_25m_'+date_string+'.shp')  
RGI = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/RGI_Arctic_marine_terminating_glacier_outlines.gpkg')   
RGI['geometry'] = RGI.geometry.buffer(0)
RGI['Name']=RGI['Name2']
#Get only the points on the glaciers (RGI outlines)
fluxgate_points_on_glaciers=gpd.sjoin(flux_gate_points,RGI,how='inner', op='intersects')
fluxgate_points_on_glaciers.to_file('fluxgate_points_on_glaciers_25m_'+date_string) #save to file so you can import later

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  


In [4]:
#import the files you just wrote above or import them if you are rerunning the script
fluxgate_points_on_glaciers = gpd.read_file('/Users/willkochtitzky/bin/Jupyter_notebooks/Arctic/Arctic_discharge/fluxgate_points_on_glaciers_25m_'+date_string+'/fluxgate_points_on_glaciers_25m_'+date_string+'.shp')

In [5]:
#select by region
CAA_flux_gates = fluxgate_points_on_glaciers[(fluxgate_points_on_glaciers['region']=='QEI')|(fluxgate_points_on_glaciers['region']=='BBI')]
Greenland_flux_gates = fluxgate_points_on_glaciers[fluxgate_points_on_glaciers['region']=='Greenland']
Alaska_flux_gates = fluxgate_points_on_glaciers[fluxgate_points_on_glaciers['region']=='Alaska']
Eastern_Arctic_flux_gates = fluxgate_points_on_glaciers[(fluxgate_points_on_glaciers['region']=='Svalbard')|(fluxgate_points_on_glaciers['region']=='Russia')|(fluxgate_points_on_glaciers['region']=='Jan Mayen')]
Iceland_flux_gates = fluxgate_points_on_glaciers[(fluxgate_points_on_glaciers['region']=='Iceland')|(fluxgate_points_on_glaciers['region']=='Jan Mayen')]
# fluxgate_points_on_glaciers.region.unique()

In [6]:
# #loop through all the ITS_LIVE datasets and export tif files
# datasets = sorted(glob.glob('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/ITS_LIVE/*.nc'))

# for i in range(0,len(datasets)):
#     xds = xarray.open_dataset(datasets[i])
#     xds['v'].rio.to_raster(str(datasets[i]+'_v.tif'))
#     xds['vx'].rio.to_raster(str(datasets[i]+'_vx.tif'))
#     xds['vy'].rio.to_raster(str(datasets[i]+'_vy.tif'))
#     xds['vx_err'].rio.to_raster(str(datasets[i]+'_vx_err.tif'))
#     xds['vy_err'].rio.to_raster(str(datasets[i]+'_vy_err.tif'))
    
# #Next you need to run the following commands in terminal to get them in the right projection
# # for file in *.tif; do gdal_edit.py -a_srs EPSG:3413 $file; done
# # for file in *.tif; do gdalwarp -t_srs EPSG:3995 $file polar_$file; done

In [6]:
#now point query all the tiff data by region - Canada
caa_datasets = sorted(glob.glob('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/ITS_LIVE/polar_CAN*.tif'))
for i in range(0,len(caa_datasets)):
    year = re.search('240_(.*).nc',caa_datasets[i]).group(1)
    data_name= re.search('.nc_(.*).tif',caa_datasets[i]).group(1)
    str(year+'_'+data_name)
    CAA_flux_gates[str(year+'_'+data_name)]=rasterstats.point_query(CAA_flux_gates.geometry,caa_datasets[i],interpolate='nearest')
    
#now point query all the tiff data for each year by region - Canada
caa_datasets = sorted(glob.glob('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/ITS_LIVE/annual_velocity_maps/CAN*_polar.tif'))
for i in range(0,len(caa_datasets)):
    data_name= re.search('240_(.*)_polar.tif',caa_datasets[i]).group(1)
    CAA_flux_gates[str(data_name)]=rasterstats.point_query(CAA_flux_gates.geometry,caa_datasets[i],interpolate='nearest')

#Pull in special velocity data for northern Ellesemere
CAA_flux_gates['Northern_velocity_Ell']=rasterstats.point_query(CAA_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/northern_velocities/Ellesmere_northern_velocity_2018to2020_composite.tif',interpolate='nearest')   
CAA_flux_gates['Northern_velocity_Baf']=rasterstats.point_query(CAA_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/northern_velocities/Southern_baffin_velocity_2019to2020_polar.tif',interpolate='nearest')

CAA_flux_gates['Northern_velocity_Ell_x']=rasterstats.point_query(CAA_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/northern_velocities/Ellesmere_northern_velocity_2018to2020_composite_x.tif',interpolate='nearest')
CAA_flux_gates['Northern_velocity_Baf_x']=rasterstats.point_query(CAA_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/northern_velocities/Southern_baffin_velocity_2019to2020_polar_x.tif',interpolate='nearest')
CAA_flux_gates['Northern_velocity_Ell_y']=rasterstats.point_query(CAA_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/northern_velocities/Ellesmere_northern_velocity_2018to2020_composite_y.tif',interpolate='nearest')
CAA_flux_gates['Northern_velocity_Baf_y']=rasterstats.point_query(CAA_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/northern_velocities/Southern_baffin_velocity_2019to2020_polar_y.tif',interpolate='nearest')


CAA_flux_gates.to_file('CAA_flux_gates_'+date_string+'2020.gpkg', driver="GPKG")#change file name based on distance

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/rasterstats/io.py:302: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/rasterstats/io.py:302: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: Se

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: Se

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: Se

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: Se

...

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceW

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceW

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceW

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceW

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceW

In [7]:
#now point query all the tiff data by region - Greenland
gre_datasets = sorted(glob.glob('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/ITS_LIVE/polar_GRE*.tif'))
for i in range(0,len(gre_datasets)):
    year = re.search('240_(.*).nc',gre_datasets[i]).group(1)
    data_name= re.search('.nc_(.*).tif',gre_datasets[i]).group(1)
    str(year+'_'+data_name)
    Greenland_flux_gates[str(year+'_'+data_name)]=rasterstats.point_query(Greenland_flux_gates.geometry,gre_datasets[i],interpolate='nearest')
    
#now point query all the tiff data for each year by region - Greenland
gre_datasets = sorted(glob.glob('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/ITS_LIVE/annual_velocity_maps/GRE*_polar.tif'))
for i in range(0,len(gre_datasets)):
    data_name= re.search('240_(.*)_polar.tif',gre_datasets[i]).group(1)
    Greenland_flux_gates[str(data_name)]=rasterstats.point_query(Greenland_flux_gates.geometry,gre_datasets[i],interpolate='nearest')    

#Pull in measures data
gre_measures_datasets = sorted(glob.glob('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Greenland_measures/*polar.tif'))
for i in range(0,len(gre_measures_datasets)):
    data_name= re.search('00_(.*)_v02.1_polar.tif',gre_measures_datasets[i]).group(1)
    data_name = str('GM_'+data_name)
    Greenland_flux_gates[str(data_name)]=rasterstats.point_query(Greenland_flux_gates.geometry,gre_measures_datasets[i],interpolate='nearest')    

    
#Pull in special velocity data for northern Greenland
Greenland_flux_gates['Northern_velocity']=rasterstats.point_query(Greenland_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/northern_velocities/Greenland_W_northern_velocity_2016to2020_composite.tif',interpolate='nearest')
Greenland_flux_gates['Northern_velocity_x']=rasterstats.point_query(Greenland_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/northern_velocities/Greenland_W_northern_velocity_2016to2020_composite_x.tif',interpolate='nearest')
Greenland_flux_gates['Northern_velocity_y']=rasterstats.point_query(Greenland_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/northern_velocities/Greenland_W_northern_velocity_2016to2020_composite_y.tif',interpolate='nearest')
# Greenland_flux_gates['Northern_velocity2']=rasterstats.point_query(Greenland_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/northern_velocities/Greenland_E_northern_velocity_2016to2020_composite.tif',interpolate='nearest')
Greenland_flux_gates['Northern_velocity_x2']=rasterstats.point_query(Greenland_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/northern_velocities/Greenland_E_northern_velocity_2016to2020_composite_x.tif',interpolate='nearest')
Greenland_flux_gates['Northern_velocity_y2']=rasterstats.point_query(Greenland_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/northern_velocities/Greenland_E_northern_velocity_2016to2020_composite_y.tif',interpolate='nearest')

Greenland_flux_gates['Northern_velocity'].fillna(np.sqrt(Greenland_flux_gates['Northern_velocity_x2']**2+Greenland_flux_gates['Northern_velocity_y2']**2),inplace=True)
Greenland_flux_gates['Northern_velocity_x'].fillna(Greenland_flux_gates['Northern_velocity_x2'],inplace=True)
Greenland_flux_gates['Northern_velocity_y'].fillna(Greenland_flux_gates['Northern_velocity_y2'],inplace=True)
Greenland_flux_gates.drop(columns=['Northern_velocity_x2','Northern_velocity_y2'],inplace=True)

Greenland_flux_gates.to_file('GRE_flux_gates_'+date_string+'2020.gpkg', driver="GPKG")#change file name based on distance


/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/rasterstats/io.py:302: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/rasterstats/io.py:302: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: Se

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: Se

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: Se

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: Se

...

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceW

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceW

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceW

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: Se

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/pandas/core/generic.py:6392: SettingWithCopy

In [8]:
#now point query all the tiff data by region - Eastern Arctic
sra_datasets = sorted(glob.glob('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/ITS_LIVE/polar_SRA*.tif'))
for i in range(0,len(sra_datasets)):
    year = re.search('240_(.*).nc',sra_datasets[i]).group(1)
    data_name= re.search('.nc_(.*).tif',sra_datasets[i]).group(1)
    str(year+'_'+data_name)
    Eastern_Arctic_flux_gates[str(year+'_'+data_name)]=rasterstats.point_query(Eastern_Arctic_flux_gates.geometry,sra_datasets[i],interpolate='nearest')
    
#now point query all the tiff data for each year by region - Eastern Arctic
sra_datasets = sorted(glob.glob('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/ITS_LIVE/annual_velocity_maps/SRA*_polar.tif'))
for i in range(0,len(sra_datasets)):
    data_name= re.search('240_(.*)_polar.tif',sra_datasets[i]).group(1)
    Eastern_Arctic_flux_gates[str(data_name)]=rasterstats.point_query(Eastern_Arctic_flux_gates.geometry,sra_datasets[i],interpolate='nearest')    

#Read in sentiel-1 velocity data
sra_datasets = sorted(glob.glob('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/velocity_mapping/retreat/*polar.tif'))
for i in range(0,len(sra_datasets)):
    print(sra_datasets[i])
    year = re.search('stack_(.*)_polar.tif',sra_datasets[i]).group(1)
    data_name= re.search('08_(.*)_stack',sra_datasets[i]).group(1)
    Eastern_Arctic_flux_gates[str('S1_'+year+'_'+data_name)]=rasterstats.point_query(Eastern_Arctic_flux_gates.geometry,sra_datasets[i],interpolate='nearest')
    
#Read in data from Tazio
Eastern_Arctic_flux_gates['Austfonna_ERS_1995']=rasterstats.point_query(Eastern_Arctic_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/velocity_mapping/Tazio/ERS_1995_dv_norm_m_per_day.tif_polar.tif',interpolate='nearest')
Eastern_Arctic_flux_gates['Svalbard_ALOS1_2007_2011']=rasterstats.point_query(Eastern_Arctic_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/velocity_mapping/Tazio/Glaciers_CCI_IV_RGI07_Svalbard_ALOS1_20070904_20110301.tif_polar.tif',interpolate='nearest')
Eastern_Arctic_flux_gates['Svalbard_ERS1_1992']=rasterstats.point_query(Eastern_Arctic_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/velocity_mapping/Tazio/Glaciers_CCI_IV_RGI07_Svalbard_ERS1_19920103_19920115.tif_polar.tif',interpolate='nearest')
Eastern_Arctic_flux_gates['Svalbard_JERS1_1994_1998']=rasterstats.point_query(Eastern_Arctic_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/velocity_mapping/Tazio/Glaciers_CCI_IV_RGI07_Svalbard_JERS1_19940205_19980326.tif_polar.tif',interpolate='nearest')
Eastern_Arctic_flux_gates['Svalbard_TSX_2008_2012']=rasterstats.point_query(Eastern_Arctic_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/velocity_mapping/Tazio/Glaciers_CCI_IV_RGI07_Svalbard_TSX_20080427_20121226.tif_polar.tif',interpolate='nearest')
Eastern_Arctic_flux_gates['FranzJosefLand_ALOS1_2010_2011']=rasterstats.point_query(Eastern_Arctic_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/velocity_mapping/Tazio/Glaciers_CCI_IV_RGI09_FranzJosefLand_ALOS1_20101222_20110417.tif_polar.tif',interpolate='nearest')
Eastern_Arctic_flux_gates['FranzJosefLand_ALOS2_2016']=rasterstats.point_query(Eastern_Arctic_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/velocity_mapping/Tazio/Glaciers_CCI_IV_RGI09_FranzJosefLand_ALOS2_20160208_20160307.tif_polar.tif',interpolate='nearest')
Eastern_Arctic_flux_gates['FranzJosefLand_JERS1_1996_1998']=rasterstats.point_query(Eastern_Arctic_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/velocity_mapping/Tazio/Glaciers_CCI_IV_RGI09_FranzJosefLand_JERS1_19960404_19980529.tif_polar.tif',interpolate='nearest')
Eastern_Arctic_flux_gates['FranzJosefLand_JERS1_1996_1997']=rasterstats.point_query(Eastern_Arctic_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/velocity_mapping/Tazio/Glaciers_CCI_IV_RGI09_FranzJosefLand_JERS1_19960808_19971011.tif_polar.tif',interpolate='nearest')
Eastern_Arctic_flux_gates['NovayaZemlya_ALOS1_2008_2010']=rasterstats.point_query(Eastern_Arctic_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/velocity_mapping/Tazio/Glaciers_CCI_IV_RGI09_NovayaZemlya_ALOS1_20081211_20100429.tif_polar.tif',interpolate='nearest')
Eastern_Arctic_flux_gates['NovayaZemlya_JERS1_1998']=rasterstats.point_query(Eastern_Arctic_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/velocity_mapping/Tazio/Glaciers_CCI_IV_RGI09_NovayaZemlya_JERS1_19980121_19980325.tif_polar.tif',interpolate='nearest')
Eastern_Arctic_flux_gates['SevernayaZemlya_ALOS1_2010']=rasterstats.point_query(Eastern_Arctic_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/velocity_mapping/Tazio/Glaciers_CCI_IV_RGI09_SevernayaZemlya_ALOS1_20100704_20101217.tif_polar.tif',interpolate='nearest')
Eastern_Arctic_flux_gates['SevernayaZemlya_ERS1_1991_1992']=rasterstats.point_query(Eastern_Arctic_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/velocity_mapping/Tazio/Glaciers_CCI_IV_RGI09_SevernayaZemlya_ERS1_19911018_19920302.tif_polar.tif',interpolate='nearest')

Eastern_Arctic_flux_gates['Northern_velocity']=rasterstats.point_query(Eastern_Arctic_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/northern_velocities/Kara_sea_velocity_2018_polar.tif',interpolate='nearest')
Eastern_Arctic_flux_gates['Northern_velocity_x']=rasterstats.point_query(Eastern_Arctic_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/northern_velocities/Kara_sea_velocity_2018_polar_x.tif',interpolate='nearest')
Eastern_Arctic_flux_gates['Northern_velocity_y']=rasterstats.point_query(Eastern_Arctic_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/northern_velocities/Kara_sea_velocity_2018_polar_y.tif',interpolate='nearest')

Eastern_Arctic_flux_gates.to_file('SRA_flux_gates_'+date_string+'2020.gpkg', driver="GPKG")#change file name based on distance


/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/rasterstats/io.py:302: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/rasterstats/io.py:302: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: Se

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: Se

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: Se

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: Se

...

/Users/willkochtitzky/Projects/Arctic_frontal_ablation/velocity_mapping/retreat/08_y_stack_2018_polar.tif


/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


/Users/willkochtitzky/Projects/Arctic_frontal_ablation/velocity_mapping/retreat/08_y_stack_2019_polar.tif


/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceW

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceW

In [9]:
#now point query all the tiff data by region - Alaska
ala_datasets = sorted(glob.glob('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/ITS_LIVE/polar_ALA*.tif'))
for i in range(0,len(ala_datasets)):
    year = re.search('240_(.*).nc',ala_datasets[i]).group(1)
    data_name= re.search('.nc_(.*).tif',ala_datasets[i]).group(1)
    str(year+'_'+data_name)
    Alaska_flux_gates[str(year+'_'+data_name)]=rasterstats.point_query(Alaska_flux_gates.geometry,ala_datasets[i],interpolate='nearest')
    
#now point query all the tiff data for each year by region - Alaska
ala_datasets = sorted(glob.glob('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/ITS_LIVE/annual_velocity_maps/ALA*_polar.tif'))
for i in range(0,len(ala_datasets)):
    data_name= re.search('240_(.*)_polar.tif',ala_datasets[i]).group(1)
    Alaska_flux_gates[str(data_name)]=rasterstats.point_query(Alaska_flux_gates.geometry,ala_datasets[i],interpolate='nearest')    
    
Alaska_flux_gates.to_file('ALA_flux_gates_'+date_string+'2020.gpkg', driver="GPKG")#change file name based on distance

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/rasterstats/io.py:302: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/rasterstats/io.py:302: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: Se

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: Se

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: Se

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: Se

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: Se

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceW

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceW

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceW

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceW

In [10]:
#now point query all the tiff data by region - Iceland
ice_datasets = sorted(glob.glob('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/ITS_LIVE/polar_ICE*.tif'))
for i in range(0,len(ice_datasets)):
    year = re.search('240_(.*).nc',ice_datasets[i]).group(1)
    data_name= re.search('.nc_(.*).tif',ice_datasets[i]).group(1)
    str(year+'_'+data_name)
    Iceland_flux_gates[str(year+'_'+data_name)]=rasterstats.point_query(Iceland_flux_gates.geometry,ice_datasets[i],interpolate='nearest')
    
#now point query all the tiff data for each year by region - Iceland
ice_datasets = sorted(glob.glob('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/ITS_LIVE/annual_velocity_maps/ICE*_polar.tif'))
for i in range(0,len(ice_datasets)):
    data_name= re.search('240_(.*)_polar.tif',ice_datasets[i]).group(1)
    Iceland_flux_gates[str(data_name)]=rasterstats.point_query(Iceland_flux_gates.geometry,ice_datasets[i],interpolate='nearest')    
    
Iceland_flux_gates.to_file('ICE_flux_gates_'+date_string+'2020.gpkg', driver="GPKG")#change file name based on distance

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/rasterstats/io.py:302: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/rasterstats/io.py:302: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: Se

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: Se

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: Se

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: Se

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: Se

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceW

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceW

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceW

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: PerformanceW

In [11]:
#Read in GlaThiDa
GlaThiDa= gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Arctic_GlaThiDa_EPSG3995.gpkg')
#Add GlaThiDa data to the dataset
GlaThiDa['geometry']=GlaThiDa.buffer(100)

In [12]:
#Extract thickness data for each region RGI 3 (BBI) and 4 (QEI)
CAA_flux_gates = gpd.read_file('/Users/willkochtitzky/bin/Jupyter_notebooks/Arctic/Arctic_discharge/CAA_flux_gates_'+date_string+'2020.gpkg')
CAA_BBI_flux_gates = CAA_flux_gates[CAA_flux_gates['region']=='BBI']
CAA_QEI_flux_gates = CAA_flux_gates[CAA_flux_gates['region']=='QEI']
CAA_BBI_flux_gates['Farinotti_thickness']=rasterstats.point_query(CAA_BBI_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RGI60-04/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')
CAA_QEI_flux_gates['Farinotti_thickness']=rasterstats.point_query(CAA_QEI_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RGI60-03/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')
CAA_QEI_flux_gates['tomo_thickness']=rasterstats.point_query(CAA_QEI_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/tomo/tomo_glacier_thickness.tif',interpolate='nearest')
CAA_QEI_flux_gates.loc[CAA_QEI_flux_gates['tomo_thickness']>0,'thickness_obs_date']=2014

CAA_BBI_flux_gates['Milan_thickness']=rasterstats.point_query(CAA_BBI_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/GLIDATASET_BAFFIN_1_thickness_polar.tif',interpolate='nearest')
CAA_BBI_flux_gates['Milan_thickness2']=rasterstats.point_query(CAA_BBI_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/GLIDATASET_BAFFIN_2_thickness_polar.tif',interpolate='nearest')
CAA_QEI_flux_gates['Milan_thickness']=rasterstats.point_query(CAA_QEI_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/GLIDATASET_CANADIAN_ARCTIC_thickness_final.tif',interpolate='nearest')
CAA_BBI_flux_gates['Milan_thickness_err']=rasterstats.point_query(CAA_BBI_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/GLIDATASET_BAFFIN_1_errthickness_polar.tif',interpolate='nearest')
CAA_BBI_flux_gates['Milan_thickness2_err']=rasterstats.point_query(CAA_BBI_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/GLIDATASET_BAFFIN_2_errthickness_polar.tif',interpolate='nearest')
CAA_QEI_flux_gates['Milan_thickness_err']=rasterstats.point_query(CAA_QEI_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/GLIDATASET_CANADIAN_ARCTIC_errthickness_final.tif',interpolate='nearest')

CAA_BBI_flux_gates['Milan_thickness'].fillna(CAA_BBI_flux_gates['Milan_thickness2'], inplace=True)
CAA_BBI_flux_gates['Milan_thickness_err'].fillna(CAA_BBI_flux_gates['Milan_thickness2_err'], inplace=True)
CAA_BBI_flux_gates.drop(columns=['Milan_thickness2', 'Milan_thickness2_err'], inplace=True)

CAA_BBI_flux_gates['model1_thickness']=rasterstats.point_query(CAA_BBI_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/other_models/results_model_1/RGI60-04/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')
CAA_QEI_flux_gates['model1_thickness']=rasterstats.point_query(CAA_QEI_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/other_models/results_model_1/RGI60-03/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')
CAA_BBI_flux_gates['model2_thickness']=rasterstats.point_query(CAA_BBI_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/other_models/results_model_2/RGI60-04/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')
CAA_QEI_flux_gates['model2_thickness']=rasterstats.point_query(CAA_QEI_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/other_models/results_model_2/RGI60-03/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')
CAA_BBI_flux_gates['model3_thickness']=rasterstats.point_query(CAA_BBI_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/other_models/results_model_3/RGI60-04/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')
CAA_QEI_flux_gates['model3_thickness']=rasterstats.point_query(CAA_QEI_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/other_models/results_model_3/RGI60-03/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')

#Pull in GlaThiDa for QEI
CAA_QEI_flux_gates_with_GlaThiDa = gpd.sjoin(CAA_QEI_flux_gates,GlaThiDa,how='left', op='intersects')
CAA_QEI_flux_gates_with_GlaThiDa=CAA_QEI_flux_gates_with_GlaThiDa[['easting','northing','THICKNESS','THICKNESS_UNCERTAINTY','SURVEY_DATE']]#loose all the attributes we don't care about
CAA_QEI_flux_gates_with_GlaThiDa_grouped = CAA_QEI_flux_gates_with_GlaThiDa.groupby(['easting','northing']).mean().reset_index() #take the mean of all the GlaThiDa data within the buffer distance
CAA_QEI_flux_gates=CAA_QEI_flux_gates.merge(CAA_QEI_flux_gates_with_GlaThiDa_grouped,left_on=['easting','northing'],right_on=['easting','northing'])#join the data that have the same easting/northing

#Pull in GlaThiDa for BBI
CAA_BBI_flux_gates_with_GlaThiDa = gpd.sjoin(CAA_BBI_flux_gates,GlaThiDa,how='left', op='intersects')
CAA_BBI_flux_gates_with_GlaThiDa=CAA_BBI_flux_gates_with_GlaThiDa[['easting','northing','THICKNESS','THICKNESS_UNCERTAINTY','SURVEY_DATE']]#loose all the attributes we don't care about
CAA_BBI_flux_gates_with_GlaThiDa_grouped = CAA_BBI_flux_gates_with_GlaThiDa.groupby(['easting','northing']).mean().reset_index() #take the mean of all the GlaThiDa data within the buffer distance
CAA_BBI_flux_gates=CAA_BBI_flux_gates.merge(CAA_BBI_flux_gates_with_GlaThiDa_grouped,left_on=['easting','northing'],right_on=['easting','northing'])#join the data that have the same easting/northing

CAA_BBI_flux_gates.to_file('CAN_BBI_flux_gates_thick_'+date_string+'2020.gpkg', driver="GPKG")#change file name based on distance
CAA_QEI_flux_gates.to_file('CAN_QEI_flux_gates_thick_'+date_string+'2020.gpkg', driver="GPKG")#change file name based on distance


/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: Se

In [13]:
#RGI 5
Greenland_flux_gates = gpd.read_file('/Users/willkochtitzky/bin/Jupyter_notebooks/Arctic/Arctic_discharge/GRE_flux_gates_'+date_string+'2020.gpkg')
Greenland_flux_gates['Farinotti_thickness']=rasterstats.point_query(Greenland_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RGI60-05/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')
Greenland_flux_gates['model1_thickness']=rasterstats.point_query(Greenland_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/other_models/results_model_1/RGI60-05/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')
Greenland_flux_gates['model2_thickness']=rasterstats.point_query(Greenland_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/other_models/results_model_2/RGI60-05/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')

Greenland_flux_gates['Milan_thickness']=rasterstats.point_query(Greenland_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/Milan_thickness_mosaic.vrt',interpolate='nearest')
Greenland_flux_gates['Milan_thickness_err']=rasterstats.point_query(Greenland_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/Milan_thickness_err_mosaic.vrt',interpolate='nearest')

#Pull in GlaThiDa for Greenland
Greenland_flux_gates_with_GlaThiDa = gpd.sjoin(Greenland_flux_gates,GlaThiDa,how='left', op='intersects')
Greenland_flux_gates_with_GlaThiDa=Greenland_flux_gates_with_GlaThiDa[['easting','northing','THICKNESS','THICKNESS_UNCERTAINTY','SURVEY_DATE']]#loose all the attributes we don't care about
Greenland_flux_gates_with_GlaThiDa_grouped = Greenland_flux_gates_with_GlaThiDa.groupby(['easting','northing']).mean().reset_index() #take the mean of all the GlaThiDa data within the buffer distance
Greenland_flux_gates=Greenland_flux_gates.merge(Greenland_flux_gates_with_GlaThiDa_grouped,left_on=['easting','northing'],right_on=['easting','northing'])#join the data that have the same easting/northing

Greenland_flux_gates.to_file('GRE_flux_gates_thick_'+date_string+'2020.gpkg', driver="GPKG")#change file name based on distance


In [14]:
#RGI 7 (Svalbard) and 9 (Russia)
Eastern_Arctic_flux_gates = gpd.read_file('/Users/willkochtitzky/bin/Jupyter_notebooks/Arctic/Arctic_discharge/SRA_flux_gates_'+date_string+'2020.gpkg')
Svalbard_flux_gates = Eastern_Arctic_flux_gates[Eastern_Arctic_flux_gates['region']=='Svalbard']
Russia_flux_gates = Eastern_Arctic_flux_gates[Eastern_Arctic_flux_gates['region']=='Russia']

Svalbard_flux_gates['Farinotti_thickness']=rasterstats.point_query(Svalbard_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RGI60-07/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')
Russia_flux_gates['Farinotti_thickness']=rasterstats.point_query(Russia_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RGI60-09/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')

Svalbard_flux_gates['Milan_thickness']=rasterstats.point_query(Svalbard_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/GLIDATASET_SPITZBERG_thickness_polar.tif',interpolate='nearest')
Svalbard_flux_gates['Milan_thickness_err']=rasterstats.point_query(Svalbard_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/GLIDATASET_SPITZBERG_errthickness_polar.tif',interpolate='nearest')

Russia_flux_gates['Milan_thickness']=rasterstats.point_query(Russia_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/Milan_thickness_mosaic.vrt',interpolate='nearest')
Russia_flux_gates['Milan_thickness_err']=rasterstats.point_query(Russia_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/Milan_thickness_err_mosaic.vrt',interpolate='nearest')
# Russia_flux_gates['Milan_thickness']=rasterstats.point_query(Russia_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/GLIDATASET_RUSSIAN_ARCTIC_FRANZ_JOSEPH_thickness_polar.tif',interpolate='nearest')
# Russia_flux_gates['Milan_thickness_err']=rasterstats.point_query(Russia_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/GLIDATASET_RUSSIAN_ARCTIC_FRANZ_JOSEPH_errthickness_polar.tif',interpolate='nearest')
# Russia_flux_gates['Milan_thickness2']=rasterstats.point_query(Russia_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/GLIDATASET_RUSSIAN_ARCTIC_NOVAYA_ZEMLYA_thickness_polar.tif',interpolate='nearest')
# Russia_flux_gates['Milan_thickness2_err']=rasterstats.point_query(Russia_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/GLIDATASET_RUSSIAN_ARCTIC_NOVAYA_ZEMLYA_errthickness_polar.tif',interpolate='nearest')
# Russia_flux_gates['Milan_thickness3']=rasterstats.point_query(Russia_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/GLIDATASET_RUSSIAN_ARCTIC_SEVERNAYA_ZEMLYA_thickness_polar.tif',interpolate='nearest')
# Russia_flux_gates['Milan_thickness3_err']=rasterstats.point_query(Russia_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/GLIDATASET_RUSSIAN_ARCTIC_SEVERNAYA_ZEMLYA_errthickness_polar.tif',interpolate='nearest')

# Russia_flux_gates['Milan_thickness'].fillna(Russia_flux_gates['Milan_thickness2'], inplace=True)
# Russia_flux_gates['Milan_thickness'].fillna(Russia_flux_gates['Milan_thickness3'], inplace=True)
# Russia_flux_gates['Milan_thickness_err'].fillna(Russia_flux_gates['Milan_thickness2_err'], inplace=True)
# Russia_flux_gates['Milan_thickness_err'].fillna(Russia_flux_gates['Milan_thickness3_err'], inplace=True)
# Russia_flux_gates.drop(columns=['Milan_thickness2', 'Milan_thickness2_err','Milan_thickness3', 'Milan_thickness3_err'], inplace=True)

Svalbard_flux_gates['model1_thickness']=rasterstats.point_query(Svalbard_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/other_models/results_model_1/RGI60-07/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')
Russia_flux_gates['model1_thickness']=rasterstats.point_query(Russia_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/other_models/results_model_1/RGI60-09/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')
Svalbard_flux_gates['model2_thickness']=rasterstats.point_query(Svalbard_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/other_models/results_model_2/RGI60-07/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')
Russia_flux_gates['model2_thickness']=rasterstats.point_query(Russia_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/other_models/results_model_2/RGI60-09/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')
Svalbard_flux_gates['model3_thickness']=rasterstats.point_query(Svalbard_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/other_models/results_model_3/RGI60-07/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')
Russia_flux_gates['model3_thickness']=rasterstats.point_query(Russia_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/other_models/results_model_3/RGI60-09/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')

#Pull in GlaThiDa for Svalbard
Svalbard_flux_gates_with_GlaThiDa = gpd.sjoin(Svalbard_flux_gates,GlaThiDa,how='left', op='intersects')
Svalbard_flux_gates_with_GlaThiDa=Svalbard_flux_gates_with_GlaThiDa[['easting','northing','THICKNESS','THICKNESS_UNCERTAINTY','SURVEY_DATE']]#loose all the attributes we don't care about
Svalbard_flux_gates_with_GlaThiDa_grouped = Svalbard_flux_gates_with_GlaThiDa.groupby(['easting','northing']).mean().reset_index() #take the mean of all the GlaThiDa data within the buffer distance
Svalbard_flux_gates=Svalbard_flux_gates.merge(Svalbard_flux_gates_with_GlaThiDa_grouped,left_on=['easting','northing'],right_on=['easting','northing'])#join the data that have the same easting/northing
#Pull in GlaThiDa for Russia
Russia_flux_gates_with_GlaThiDa = gpd.sjoin(Russia_flux_gates,GlaThiDa,how='left', op='intersects')
Russia_flux_gates_with_GlaThiDa=Russia_flux_gates_with_GlaThiDa[['easting','northing','THICKNESS','THICKNESS_UNCERTAINTY','SURVEY_DATE']]#loose all the attributes we don't care about
Russia_flux_gates_with_GlaThiDa_grouped = Russia_flux_gates_with_GlaThiDa.groupby(['easting','northing']).mean().reset_index() #take the mean of all the GlaThiDa data within the buffer distance
Russia_flux_gates=Russia_flux_gates.merge(Russia_flux_gates_with_GlaThiDa_grouped,left_on=['easting','northing'],right_on=['easting','northing'])#join the data that have the same easting/northing

#Pull in Andrey's thickness obs for Russia
Russia_thickness = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Russia/Russia_thickness_merged_EPSG3995.geojson')
Russia_thickness['geometry']=Russia_thickness.buffer(100)
Russia_flux_gates_with_Russia_thickness = gpd.sjoin(Russia_flux_gates,Russia_thickness,how='left', op='intersects')
Russia_flux_gates_with_Russia_thickness=Russia_flux_gates_with_Russia_thickness[['easting','northing','Russian_thickness']]#loose all the attributes we don't care about
Russia_flux_gates_with_Russia_thickness_grouped = Russia_flux_gates_with_Russia_thickness.groupby(['easting','northing']).mean().reset_index() #take the mean of all the GlaThiDa data within the buffer distance
Russia_flux_gates=Russia_flux_gates.merge(Russia_flux_gates_with_Russia_thickness_grouped,left_on=['easting','northing'],right_on=['easting','northing'])#join the data that have the same easting/northing
Russia_flux_gates.loc[Russia_flux_gates['Russian_thickness']>0,'thickness_obs_date']=2015

print('nans before RES data',Russia_flux_gates['Russian_thickness'].isna().sum())

#Read in RES data from Toby and Julian
Russia_thickness = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RES_data/RES_thickness_V_sep2020.gpkg')
Russia_thickness['geometry']=Russia_thickness.buffer(100)
Russia_flux_gates_with_Russia_thickness = gpd.sjoin(Russia_flux_gates,Russia_thickness,how='left', op='intersects')
Russia_flux_gates_with_Russia_thickness=Russia_flux_gates_with_Russia_thickness[['easting','northing','thickness(m)']]#loose all the attributes we don't care about
Russia_flux_gates_with_Russia_thickness_grouped = Russia_flux_gates_with_Russia_thickness.groupby(['easting','northing']).mean().reset_index() #take the mean of all the GlaThiDa data within the buffer distance
Russia_flux_gates=Russia_flux_gates.merge(Russia_flux_gates_with_Russia_thickness_grouped,left_on=['easting','northing'],right_on=['easting','northing'])#join the data that have the same easting/northing

Russia_flux_gates['Russian_thickness'].fillna(Russia_flux_gates['thickness(m)'],inplace=True) #fill in values from Julian and Toby
Russia_flux_gates.loc[Russia_flux_gates['thickness(m)']>0,'thickness_obs_date'].fillna(2000)
Russia_flux_gates = Russia_flux_gates.drop(columns='thickness(m)')  #thickness(m) column because we dont need it now
print('nans after RES data',Russia_flux_gates['Russian_thickness'].isna().sum())



Svalbard_flux_gates.to_file('Svalbard_flux_gates_thick_'+date_string+'2020.gpkg', driver="GPKG")#change file name based on distance
Russia_flux_gates.to_file('Russia_flux_gates_thick_'+date_string+'2020.gpkg', driver="GPKG")#change file name based on distance

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: Se

nans before RES data 97600
nans after RES data 71388


In [15]:
#RGI 1
Alaska_flux_gates = gpd.read_file('/Users/willkochtitzky/bin/Jupyter_notebooks/Arctic/Arctic_discharge/ALA_flux_gates_'+date_string+'2020.gpkg')
Alaska_flux_gates['Farinotti_thickness']=rasterstats.point_query(Alaska_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RGI60-01/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')

Alaska_flux_gates['model1_thickness']=rasterstats.point_query(Alaska_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/other_models/results_model_1/RGI60-01/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')
Alaska_flux_gates['model2_thickness']=rasterstats.point_query(Alaska_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/other_models/results_model_2/RGI60-01/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')
Alaska_flux_gates['model3_thickness']=rasterstats.point_query(Alaska_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/other_models/results_model_3/RGI60-01/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')

Alaska_flux_gates['Milan_thickness']=rasterstats.point_query(Alaska_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/GLIDATASET_ALASKA_thickness_polar.tif',interpolate='nearest')
Alaska_flux_gates['Milan_thickness_err']=rasterstats.point_query(Alaska_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/GLIDATASET_ALASKA_errthickness_polar.tif',interpolate='nearest')
Alaska_flux_gates['Milan_thickness2']=rasterstats.point_query(Alaska_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/GLIDATASET_WEST_CANADA_1B_thickness_polar.tif',interpolate='nearest')
Alaska_flux_gates['Milan_thickness2_err']=rasterstats.point_query(Alaska_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/GLIDATASET_WEST_CANADA_1B_errthickness_polar.tif',interpolate='nearest')
Alaska_flux_gates['Milan_thickness'].fillna(Alaska_flux_gates['Milan_thickness2'], inplace=True)
Alaska_flux_gates['Milan_thickness_err'].fillna(Alaska_flux_gates['Milan_thickness2_err'], inplace=True)
Alaska_flux_gates.drop(columns=['Milan_thickness2', 'Milan_thickness2_err'], inplace=True)

#Pull in GlaThiDa for Alaska
Alaska_flux_gates_with_GlaThiDa = gpd.sjoin(Alaska_flux_gates,GlaThiDa,how='left', op='intersects')
Alaska_flux_gates_with_GlaThiDa=Alaska_flux_gates_with_GlaThiDa[['easting','northing','THICKNESS','THICKNESS_UNCERTAINTY','SURVEY_DATE']]#loose all the attributes we don't care about
Alaska_flux_gates_with_GlaThiDa_grouped = Alaska_flux_gates_with_GlaThiDa.groupby(['easting','northing']).mean().reset_index() #take the mean of all the GlaThiDa data within the buffer distance
Alaska_flux_gates=Alaska_flux_gates.merge(Alaska_flux_gates_with_GlaThiDa_grouped,left_on=['easting','northing'],right_on=['easting','northing'])#join the data that have the same easting/northing

Alaska_flux_gates.to_file('ALA_flux_gates_thick_'+date_string+'2020.gpkg', driver="GPKG")#change file name based on distance

In [16]:
#RGI 6
Iceland_flux_gates = gpd.read_file('/Users/willkochtitzky/bin/Jupyter_notebooks/Arctic/Arctic_discharge/ICE_flux_gates_'+date_string+'2020.gpkg')
Iceland_flux_gates['Farinotti_thickness']=rasterstats.point_query(Iceland_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RGI60-06/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')

Iceland_flux_gates['model1_thickness']=rasterstats.point_query(Iceland_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/other_models/results_model_1/RGI60-06/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')
Iceland_flux_gates['model2_thickness']=rasterstats.point_query(Iceland_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/other_models/results_model_2/RGI60-06/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')
Iceland_flux_gates['model3_thickness']=rasterstats.point_query(Iceland_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/other_models/results_model_3/RGI60-06/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')

Iceland_flux_gates['Milan_thickness']=rasterstats.point_query(Iceland_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/GLIDATASET_ICELAND_thickness_polar.tif',interpolate='nearest')
Iceland_flux_gates['Milan_thickness_err']=rasterstats.point_query(Iceland_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/GLIDATASET_ICELAND_errthickness_polar.tif',interpolate='nearest')

#Pull in GlaThiDa for Iceland
Iceland_flux_gates_with_GlaThiDa = gpd.sjoin(Iceland_flux_gates,GlaThiDa,how='left', op='intersects')
Iceland_flux_gates_with_GlaThiDa=Iceland_flux_gates_with_GlaThiDa[['easting','northing','THICKNESS','THICKNESS_UNCERTAINTY','SURVEY_DATE']]#loose all the attributes we don't care about
Iceland_flux_gates_with_GlaThiDa_grouped = Iceland_flux_gates_with_GlaThiDa.groupby(['easting','northing']).mean().reset_index() #take the mean of all the GlaThiDa data within the buffer distance
Iceland_flux_gates=Iceland_flux_gates.merge(Iceland_flux_gates_with_GlaThiDa_grouped,left_on=['easting','northing'],right_on=['easting','northing'])#join the data that have the same easting/northing

Iceland_flux_gates.to_file('ICE_flux_gates_thick_'+date_string+'2020.gpkg', driver="GPKG")#change file name based on distance

In [17]:
#Read in all the files with thickness data
CAA_BBI_flux_gates = gpd.read_file('/Users/willkochtitzky/bin/Jupyter_notebooks/Arctic/Arctic_discharge/CAN_BBI_flux_gates_thick_'+date_string+'2020.gpkg')
CAA_QEI_flux_gates = gpd.read_file('/Users/willkochtitzky/bin/Jupyter_notebooks/Arctic/Arctic_discharge/CAN_QEI_flux_gates_thick_'+date_string+'2020.gpkg')
Greenland_flux_gates = gpd.read_file('/Users/willkochtitzky/bin/Jupyter_notebooks/Arctic/Arctic_discharge/GRE_flux_gates_thick_'+date_string+'2020.gpkg')
Svalbard_flux_gates = gpd.read_file('/Users/willkochtitzky/bin/Jupyter_notebooks/Arctic/Arctic_discharge/Svalbard_flux_gates_thick_'+date_string+'2020.gpkg')
Russia_flux_gates = gpd.read_file('/Users/willkochtitzky/bin/Jupyter_notebooks/Arctic/Arctic_discharge/Russia_flux_gates_thick_'+date_string+'2020.gpkg')
Alaska_flux_gates = gpd.read_file('/Users/willkochtitzky/bin/Jupyter_notebooks/Arctic/Arctic_discharge/ALA_flux_gates_thick_'+date_string+'2020.gpkg')
Iceland_flux_gates = gpd.read_file('/Users/willkochtitzky/bin/Jupyter_notebooks/Arctic/Arctic_discharge/ICE_flux_gates_thick_'+date_string+'2020.gpkg')

CAA_BBI_flux_gates['region']='Canada'
CAA_QEI_flux_gates['region']='Canada'
Greenland_flux_gates['region']='Greenland'
Svalbard_flux_gates['region']='Svalbard'
Russia_flux_gates['region']='Russia'
Alaska_flux_gates['region']='Alaska'
Iceland_flux_gates['region']='Iceland'

Arctic_discharge = pd.concat([CAA_BBI_flux_gates,CAA_QEI_flux_gates,Greenland_flux_gates,Svalbard_flux_gates,Russia_flux_gates,Alaska_flux_gates,Iceland_flux_gates],sort=False)
Arctic_discharge['length']=25 #length of point spacing from fluxlines

#read in thickness change for each point
Arctic_discharge['dhdt_early']=rasterstats.point_query(Arctic_discharge.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Hugonnet_thickness_adjustment/dhdt/aa_Hugonnet_thickness_early_mosaic.vrt',interpolate='nearest')
Arctic_discharge['dhdt_err_early']=rasterstats.point_query(Arctic_discharge.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Hugonnet_thickness_adjustment/dhdt_err/aa_Hugonnet_thickness_err_early_mosaic.vrt',interpolate='nearest')
Arctic_discharge['dhdt_late']=rasterstats.point_query(Arctic_discharge.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Hugonnet_thickness_adjustment/dhdt/aa_Hugonnet_thickness_late_mosaic.vrt',interpolate='nearest')
Arctic_discharge['dhdt_err_late']=rasterstats.point_query(Arctic_discharge.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Hugonnet_thickness_adjustment/dhdt_err/aa_Hugonnet_thickness_err_late_mosaic.vrt',interpolate='nearest')

Arctic_discharge.to_file('Arctic_discharge_points_pre_processing_epsg3995_'+date_string+'.gpkg', driver="GPKG")#export

In [18]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [19]:
pwd

'/Users/willkochtitzky/bin/Jupyter_notebooks/Arctic/Arctic_discharge'

In [20]:
Arctic_discharge

,region,creation,easting,northing,index_righ,Id,Area,RGIId,GLIMSId,BgnDate,EndDate,CenLon,CenLat,O1Region,O2Region,Zmin,Zmax,Zmed,Slope,Aspect,Lmax,Status,Connect,Form,TermType,Surging,Linkages,Name,layer,path,RGIid_2,Name2,Area_calcu,0000_v,0000_vx,0000_vx_err,0000_vy,0000_vy_err,1999_v_err,1999_v,1999_vx_err,1999_vx,1999_vy_err,1999_vy,2000_count,2000_v_err,2000_v,2000_vx_err,2000_vx,2000_vy_err,2000_vy,2001_count,2001_v_err,2001_v,2001_vx_err,2001_vx,2001_vy_err,2001_vy,2002_count,2002_v_err,2002_v,2002_vx_err,2002_vx,2002_vy_err,2002_vy,2003_count,2003_v_err,2003_v,2003_vx_err,2003_vx,2003_vy_err,2003_vy,2004_count,2004_v_err,2004_v,2004_vx_err,2004_vx,2004_vy_err,2004_vy,2005_count,2005_v_err,2005_v,2005_vx_err,2005_vx,2005_vy_err,2005_vy,2006_count,2006_v_err,2006_v,2006_vx_err,2006_vx,2006_vy_err,2006_vy,2007_count,2007_v_err,2007_v,2007_vx_err,2007_vx,2007_vy_err,2007_vy,2008_count,2008_v_err,2008_v,2008_vx_err,2008_vx,2008_vy_err,2008_vy,2009_count,2009_v_err,2009_v,2009_vx_err,2009_vx,2009_vy_err,2009_vy,2010_count,2010_v_err,2010_v,2010_vx_err,2010_vx,2010_vy_err,2010_vy,2011_count,2011_v_err,2011_v,2011_vx_err,2011_vx,2011_vy_err,2011_vy,2012_count,2012_v_err,2012_v,2012_vx_err,2012_vx,2012_vy_err,2012_vy,2013_count,2013_v_err,2013_v,2013_vx_err,2013_vx,2013_vy_err,2013_vy,2014_count,2014_v_err,2014_v,2014_vx_err,2014_vx,2014_vy_err,2014_vy,2015_count,2015_v_err,2015_v,2015_vx_err,2015_vx,2015_vy_err,2015_vy,2016_count,2016_v_err,2016_v,2016_vx_err,2016_vx,2016_vy_err,2016_vy,2017_count,2017_v_err,2017_v,2017_vx_err,2017_vx,2017_vy_err,2017_vy,2018_count,2018_v_err,2018_v,2018_vx_err,2018_vx,2018_vy_err,2018_vy,Northern_velocity_Ell,Northern_velocity_Baf,Northern_velocity_Ell_x,Northern_velocity_Baf_x,Northern_velocity_Ell_y,Northern_velocity_Baf_y,Farinotti_thickness,Milan_thickness,Milan_thickness_err,model1_thickness,model2_thickness,model3_thickness,THICKNESS,THICKNESS_UNCERTAINTY,SURVEY_DATE,geometry,tomo_thickness,thickness_obs_date,GM_2014_2015_ex,GM_2014_2015_ey,GM_2014_2015_vv,GM_2014_2015_vx,GM_2014_2015_vy,GM_2015_2016_ex,GM_2015_2016_ey,GM_2015_2016_vv,GM_2015_2016_vx,GM_2015_2016_vy,GM_2016_2017_ex,GM_2016_2017_ey,GM_2016_2017_vv,GM_2016_2017_vx,GM_2016_2017_vy,GM_2017_2018_ex,GM_2017_2018_ey,GM_2017_2018_vv,GM_2017_2018_vx,GM_2017_2018_vy,GM_2000_2001_ex,GM_2000_2001_ey,GM_2000_2001_vv,GM_2000_2001_vx,GM_2000_2001_vy,GM_2005_2006_ex,GM_2005_2006_ey,GM_2005_2006_vv,GM_2005_2006_vx,GM_2005_2006_vy,GM_2006_2007_ex,GM_2006_2007_ey,GM_2006_2007_vv,GM_2006_2007_vx,GM_2006_2007_vy,GM_2007_2008_ex,GM_2007_2008_ey,GM_2007_2008_vv,GM_2007_2008_vx,GM_2007_2008_vy,GM_2008_2009_ex,GM_2008_2009_ey,GM_2008_2009_vv,GM_2008_2009_vx,GM_2008_2009_vy,GM_2009_2010_ex,GM_2009_2010_ey,GM_2009_2010_vv,GM_2009_2010_vx,GM_2009_2010_vy,GM_2012_2013_ex,GM_2012_2013_ey,GM_2012_2013_vv,GM_2012_2013_vx,GM_2012_2013_vy,Northern_velocity,Northern_velocity_x,Northern_velocity_y,S1_2015_mag_se,S1_2016_mag_se,S1_2017_mag_se,S1_2018_mag_se,S1_2019_mag_se,S1_2015_mag,S1_2016_mag,S1_2017_mag,S1_2018_mag,S1_2019_mag,S1_2015_x_se,S1_2016_x_se,S1_2017_x_se,S1_2018_x_se,S1_2019_x_se,S1_2015_x,S1_2016_x,S1_2017_x,S1_2018_x,S1_2019_x,S1_2015_y_se,S1_2016_y_se,S1_2017_y_se,S1_2018_y_se,S1_2019_y_se,S1_2015_y,S1_2016_y,S1_2017_y,S1_2018_y,S1_2019_y,Austfonna_ERS_1995,Svalbard_ALOS1_2007_2011,Svalbard_ERS1_1992,Svalbard_JERS1_1994_1998,Svalbard_TSX_2008_2012,FranzJosefLand_ALOS1_2010_2011,FranzJosefLand_ALOS2_2016,FranzJosefLand_JERS1_1996_1998,FranzJosefLand_JERS1_1996_1997,NovayaZemlya_ALOS1_2008_2010,NovayaZemlya_JERS1_1998,SevernayaZemlya_ALOS1_2010,SevernayaZemlya_ERS1_1991_1992,Russian_thickness,length,dhdt_early,dhdt_err_early,dhdt_late,dhdt_err_late
0,Canada,new fluxgate,-2.784497e+06,-1.190481e+06,265,None,9.987,RGI60-04.00172,G293152E62614N,19990914,-9999999,-66.84760,62.6143,4,8,42,775,605,11.7,18,6905,0,0,1,1,9,9,None,rgi60_merge_orginal,/Users/willkochtitzky/data/repository/rgi60/re...,RGI60-04.00172,None,10.605257,NaN,NaN,-32767.0,NaN

In [2]:
Arctic_discharge = gpd.read_file('Arctic_discharge_points_pre_processing_epsg3995_'+date_string+'.gpkg', driver="GPKG")#change file name based on distance

#convert some data types
Arctic_discharge[['2003_count', '2003_v_err', '2003_v', '2003_vx', '2003_vy',
    '2004_count', '2004_v_err', '2004_v', '2004_vx', '2004_vy',
    '2005_count', '2005_v_err', '2005_v', '2005_vx', '2005_vy',
    '2006_count', '2006_v_err', '2006_v', '2006_vx', '2006_vy',
    '2007_count', '2007_v_err', '2007_v', '2007_vx', '2007_vy',
    '2008_count', '2008_v_err', '2008_v', '2008_vx', '2008_vy',
    '2011_count', '2011_v_err', '2011_v', '2011_vx', '2011_vy',
    '2012_count', '2012_v_err', '2012_v', '2012_vx', '2012_vy',
    'Northern_velocity_Ell', 'Northern_velocity_Baf',
    'Northern_velocity_Ell_x', 'Northern_velocity_Baf_x',
    'Northern_velocity_Ell_y', 'Northern_velocity_Baf_y',
    'THICKNESS','THICKNESS_UNCERTAINTY', 'SURVEY_DATE',
    'Northern_velocity', 'Northern_velocity_x',
    'Northern_velocity_y', 'S1_2015_mag_se', 'S1_2016_mag_se',
    'S1_2017_mag_se', 'S1_2018_mag_se', 'S1_2019_mag_se', 'S1_2015_mag',
    'S1_2016_mag', 'S1_2017_mag', 'S1_2018_mag', 'S1_2019_mag',
    'S1_2015_x_se', 'S1_2016_x_se', 'S1_2017_x_se', 'S1_2018_x_se',
    'S1_2019_x_se', 'S1_2015_x', 'S1_2016_x', 'S1_2017_x', 'S1_2018_x',
    'S1_2019_x', 'S1_2015_y_se', 'S1_2016_y_se', 'S1_2017_y_se',
    'S1_2018_y_se', 'S1_2019_y_se', 'S1_2015_y', 'S1_2016_y', 'S1_2017_y',
    'S1_2018_y', 'S1_2019_y', 'Austfonna_ERS_1995',
    'Svalbard_ALOS1_2007_2011', 'Svalbard_ERS1_1992',
    'Svalbard_JERS1_1994_1998', 'Svalbard_TSX_2008_2012',
    'FranzJosefLand_ALOS1_2010_2011', 'FranzJosefLand_ALOS2_2016',
    'FranzJosefLand_JERS1_1996_1998', 'FranzJosefLand_JERS1_1996_1997',
    'NovayaZemlya_ALOS1_2008_2010', 'NovayaZemlya_JERS1_1998',
    'SevernayaZemlya_ALOS1_2010', 'SevernayaZemlya_ERS1_1991_1992']]=Arctic_discharge[['2003_count', '2003_v_err', '2003_v', '2003_vx', '2003_vy',
    '2004_count', '2004_v_err', '2004_v', '2004_vx', '2004_vy',
    '2005_count', '2005_v_err', '2005_v', '2005_vx', '2005_vy',
    '2006_count', '2006_v_err', '2006_v', '2006_vx', '2006_vy',
    '2007_count', '2007_v_err', '2007_v', '2007_vx', '2007_vy',
    '2008_count', '2008_v_err', '2008_v', '2008_vx', '2008_vy',
    '2011_count', '2011_v_err', '2011_v', '2011_vx', '2011_vy',
    '2012_count', '2012_v_err', '2012_v', '2012_vx', '2012_vy',
    'Northern_velocity_Ell', 'Northern_velocity_Baf',
    'Northern_velocity_Ell_x', 'Northern_velocity_Baf_x',
    'Northern_velocity_Ell_y', 'Northern_velocity_Baf_y',
    'THICKNESS','THICKNESS_UNCERTAINTY', 'SURVEY_DATE',
    'Northern_velocity', 'Northern_velocity_x',
    'Northern_velocity_y', 'S1_2015_mag_se', 'S1_2016_mag_se',
    'S1_2017_mag_se', 'S1_2018_mag_se', 'S1_2019_mag_se', 'S1_2015_mag',
    'S1_2016_mag', 'S1_2017_mag', 'S1_2018_mag', 'S1_2019_mag',
    'S1_2015_x_se', 'S1_2016_x_se', 'S1_2017_x_se', 'S1_2018_x_se',
    'S1_2019_x_se', 'S1_2015_x', 'S1_2016_x', 'S1_2017_x', 'S1_2018_x',
    'S1_2019_x', 'S1_2015_y_se', 'S1_2016_y_se', 'S1_2017_y_se',
    'S1_2018_y_se', 'S1_2019_y_se', 'S1_2015_y', 'S1_2016_y', 'S1_2017_y',
    'S1_2018_y', 'S1_2019_y', 'Austfonna_ERS_1995',
    'Svalbard_ALOS1_2007_2011', 'Svalbard_ERS1_1992',
    'Svalbard_JERS1_1994_1998', 'Svalbard_TSX_2008_2012',
    'FranzJosefLand_ALOS1_2010_2011', 'FranzJosefLand_ALOS2_2016',
    'FranzJosefLand_JERS1_1996_1998', 'FranzJosefLand_JERS1_1996_1997',
    'NovayaZemlya_ALOS1_2008_2010', 'NovayaZemlya_JERS1_1998',
    'SevernayaZemlya_ALOS1_2010', 'SevernayaZemlya_ERS1_1991_1992']].astype(np.float)

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


In [3]:
Arctic_discharge.loc[Arctic_discharge['Milan_thickness']<0,'Milan_thickness']=Arctic_discharge['Farinotti_thickness']
Arctic_discharge.loc[:,'Milan_thickness'] = Arctic_discharge['Milan_thickness'].fillna(Arctic_discharge['Farinotti_thickness'])
Arctic_discharge.loc[:,'Milan_thickness_err'] = Arctic_discharge['Milan_thickness_err'].fillna(Arctic_discharge['Farinotti_thickness']*.36)

Arctic_discharge.loc[:,'Milan_thickness'] = Arctic_discharge['Milan_thickness'].fillna(0)

#Fix RGI numbers
Arctic_discharge['RGIId_original']=Arctic_discharge['RGIId']
Arctic_discharge.loc[:,'RGIId']=Arctic_discharge['RGIid_2']
Arctic_discharge=Arctic_discharge.drop('RGIid_2',axis=1)

# #fill glaciers that have no thickness data due to mistakes in Farinotti and RGI
# mask = Arctic_discharge['RGIId']=='RGI60-05.07646'
# Arctic_discharge.loc[mask, 'Farinotti_thickness']=Arctic_discharge.loc[mask, 'Farinotti_thickness'].replace(0,70)
mask = Arctic_discharge['RGIId']=='RGI60-05.10938'
Arctic_discharge.loc[mask, 'Milan_thickness']=Arctic_discharge.loc[mask, 'Milan_thickness'].replace(0,170)
Arctic_discharge.loc[mask, 'Milan_thickness_err']=Arctic_discharge.loc[mask, 'Milan_thickness_err'].replace(0,50)
mask = Arctic_discharge['RGIId']=='RGI60-09.01069'
Arctic_discharge.loc[mask, 'Milan_thickness']=Arctic_discharge.loc[mask, 'Milan_thickness'].replace(0,120)
Arctic_discharge.loc[mask, 'Milan_thickness_err']=Arctic_discharge.loc[mask, 'Milan_thickness_err'].replace(0,40)
mask = Arctic_discharge['RGIId']=='RGI60-05.10945'
Arctic_discharge.loc[mask, 'Milan_thickness']=Arctic_discharge.loc[mask, 'Milan_thickness'].replace(0,170)
Arctic_discharge.loc[mask, 'Milan_thickness_err']=Arctic_discharge.loc[mask, 'Milan_thickness_err'].replace(0,50)
mask = Arctic_discharge['RGIId']=='RGI60-10.00004'
Arctic_discharge.loc[mask, 'Milan_thickness']=Arctic_discharge.loc[mask, 'Milan_thickness'].replace(0,30)
Arctic_discharge.loc[mask, 'Milan_thickness_err']=Arctic_discharge.loc[mask, 'Milan_thickness_err'].replace(0,20)

#Jan Mayen ice thickness
mask = Arctic_discharge['RGIId']=='RGI60-07.01575' # Weyprechtbreen
Arctic_discharge.loc[mask, 'Milan_thickness']=Arctic_discharge.loc[mask, 'Milan_thickness'].replace(0,55)
Arctic_discharge.loc[mask, 'Milan_thickness_err']=Arctic_discharge.loc[mask, 'Milan_thickness_err'].replace(0,15)
mask = Arctic_discharge['RGIId']=='RGI60-07.01577'#Kjerulfbreen
Arctic_discharge.loc[mask, 'Milan_thickness']=Arctic_discharge.loc[mask, 'Milan_thickness'].replace(0,55)
Arctic_discharge.loc[mask, 'Milan_thickness_err']=Arctic_discharge.loc[mask, 'Milan_thickness_err'].replace(0,15)
mask = Arctic_discharge['RGIId']=='RGI60-07.01576' #Gjuvbreen
Arctic_discharge.loc[mask, 'Milan_thickness']=Arctic_discharge.loc[mask, 'Milan_thickness'].replace(0,50)
Arctic_discharge.loc[mask, 'Milan_thickness_err']=Arctic_discharge.loc[mask, 'Milan_thickness_err'].replace(0,10)
mask = Arctic_discharge['RGIId']=='RGI60-07.01581'#Frielebreen
Arctic_discharge.loc[mask, 'Milan_thickness']=Arctic_discharge.loc[mask, 'Milan_thickness'].replace(0,45)
Arctic_discharge.loc[mask, 'Milan_thickness_err']=Arctic_discharge.loc[mask, 'Milan_thickness_err'].replace(0,15)
mask = Arctic_discharge['RGIId']=='RGI60-07.01582'#Pr Haralds Breen
Arctic_discharge.loc[mask, 'Milan_thickness']=Arctic_discharge.loc[mask, 'Milan_thickness'].replace(0,55)
Arctic_discharge.loc[mask, 'Milan_thickness_err']=Arctic_discharge.loc[mask, 'Milan_thickness_err'].replace(0,15)
mask = Arctic_discharge['RGIId']=='RGI60-07.01578'#Svend Foynbreen
Arctic_discharge.loc[mask, 'Milan_thickness']=Arctic_discharge.loc[mask, 'Milan_thickness'].replace(0,47)
Arctic_discharge.loc[mask, 'Milan_thickness_err']=Arctic_discharge.loc[mask, 'Milan_thickness_err'].replace(0,10)

Arctic_discharge.loc[Arctic_discharge['Milan_thickness']>2000,'Milan_thickness']=0

#convert the dataframe to all numbers (some columns are objects not float 64)
for i in range(0, 85):
    Arctic_discharge.iloc[:,i] = pd.to_numeric(Arctic_discharge.iloc[:,i], errors='ignore')
    
Arctic_discharge['Northern_velocity'].fillna(Arctic_discharge['Northern_velocity_Baf'], inplace=True)
Arctic_discharge['Northern_velocity'].fillna(Arctic_discharge['Northern_velocity_Ell'], inplace=True)

Arctic_discharge['Northern_velocity_x'].fillna(Arctic_discharge['Northern_velocity_Baf_x'], inplace=True)
Arctic_discharge['Northern_velocity_x'].fillna(Arctic_discharge['Northern_velocity_Ell_x'], inplace=True)

Arctic_discharge['Northern_velocity_y'].fillna(Arctic_discharge['Northern_velocity_Baf_y'], inplace=True)
Arctic_discharge['Northern_velocity_y'].fillna(Arctic_discharge['Northern_velocity_Ell_y'], inplace=True)

In [4]:
#clean up mistakes in the ITS_LIVE error record (where there is an error value but no observation, this messes up the northern extras)
no_vel_obs_idx = Arctic_discharge['0000_v'].isna()
Arctic_discharge.loc[no_vel_obs_idx,'0000_vx_err']=np.nan
Arctic_discharge.loc[no_vel_obs_idx,'0000_vy_err']=np.nan

#calculate ITS_LIVE error
Arctic_discharge.loc[:,'0000_v_err']=np.sqrt(Arctic_discharge['0000_vy_err']**2+Arctic_discharge['0000_vx_err']**2)
Arctic_discharge = Arctic_discharge.reset_index()

In [5]:
#Pull in data from Tazio
#Fill errs with nan
Arctic_discharge['Austfonna_ERS_1995_err'] = np.nan
Arctic_discharge['Svalbard_ALOS1_2007_2011_err'] = np.nan
Arctic_discharge['Svalbard_ERS1_1992_err'] = np.nan
Arctic_discharge['Svalbard_JERS1_1994_1998_err'] = np.nan
Arctic_discharge['Svalbard_TSX_2008_2012_err'] = np.nan
Arctic_discharge['FranzJosefLand_ALOS1_2010_2011_err'] = np.nan
Arctic_discharge['FranzJosefLand_ALOS2_2016_err'] = np.nan
Arctic_discharge['FranzJosefLand_JERS1_1996_1998_err'] = np.nan
Arctic_discharge['FranzJosefLand_JERS1_1996_1997_err'] = np.nan
Arctic_discharge['NovayaZemlya_ALOS1_2008_2010_err'] = np.nan
Arctic_discharge['NovayaZemlya_JERS1_1998_err'] = np.nan
Arctic_discharge['SevernayaZemlya_ALOS1_2010_err'] = np.nan
Arctic_discharge['SevernayaZemlya_ERS1_1991_1992_err'] = np.nan
#convert to float so that the below code works
Arctic_discharge.loc[:,'Austfonna_ERS_1995'] = Arctic_discharge['Austfonna_ERS_1995'].astype(float)
Arctic_discharge.loc[:,'Svalbard_ALOS1_2007_2011'] = Arctic_discharge['Svalbard_ALOS1_2007_2011'].astype(float)
Arctic_discharge.loc[:,'Svalbard_ERS1_1992'] = Arctic_discharge['Svalbard_ERS1_1992'].astype(float)
Arctic_discharge.loc[:,'Svalbard_JERS1_1994_1998'] = Arctic_discharge['Svalbard_JERS1_1994_1998'].astype(float)
Arctic_discharge.loc[:,'Svalbard_TSX_2008_2012'] = Arctic_discharge['Svalbard_TSX_2008_2012'].astype(float)
Arctic_discharge.loc[:,'FranzJosefLand_ALOS1_2010_2011'] = Arctic_discharge['FranzJosefLand_ALOS1_2010_2011'].astype(float)
Arctic_discharge.loc[:,'FranzJosefLand_ALOS2_2016'] = Arctic_discharge['FranzJosefLand_ALOS2_2016'].astype(float)
Arctic_discharge.loc[:,'FranzJosefLand_JERS1_1996_1998'] = Arctic_discharge['FranzJosefLand_JERS1_1996_1998'].astype(float)
Arctic_discharge.loc[:,'FranzJosefLand_JERS1_1996_1997'] = Arctic_discharge['FranzJosefLand_JERS1_1996_1997'].astype(float)
Arctic_discharge.loc[:,'NovayaZemlya_ALOS1_2008_2010'] = Arctic_discharge['NovayaZemlya_ALOS1_2008_2010'].astype(float)
Arctic_discharge.loc[:,'NovayaZemlya_JERS1_1998'] = Arctic_discharge['NovayaZemlya_JERS1_1998'].astype(float)
Arctic_discharge.loc[:,'SevernayaZemlya_ALOS1_2010'] = Arctic_discharge['SevernayaZemlya_ALOS1_2010'].astype(float)
Arctic_discharge.loc[:,'SevernayaZemlya_ERS1_1991_1992'] = Arctic_discharge['SevernayaZemlya_ERS1_1991_1992'].astype(float)

#incorporate data from Tazio   
i=Arctic_discharge['Austfonna_ERS_1995']>0
Arctic_discharge.loc[i,'Austfonna_ERS_1995_err'] = 10
i=Arctic_discharge['Svalbard_ALOS1_2007_2011']>0
Arctic_discharge.loc[i,'Svalbard_ALOS1_2007_2011_err'] = 10
i=Arctic_discharge['Svalbard_ERS1_1992']>0
Arctic_discharge.loc[i,'Svalbard_ERS1_1992_err'] = 40
i=Arctic_discharge['Svalbard_JERS1_1994_1998']>0
Arctic_discharge.loc[i,'Svalbard_JERS1_1994_1998_err'] = 20
i=Arctic_discharge['Svalbard_TSX_2008_2012']>0
Arctic_discharge.loc[i,'Svalbard_TSX_2008_2012_err'] = 10
i=Arctic_discharge['FranzJosefLand_ALOS1_2010_2011']>0
Arctic_discharge.loc[i,'FranzJosefLand_ALOS1_2010_2011_err'] = 10
i=Arctic_discharge['FranzJosefLand_ALOS2_2016']>0
Arctic_discharge.loc[i,'FranzJosefLand_ALOS2_2016_err'] = 10
i=Arctic_discharge['FranzJosefLand_JERS1_1996_1998']>0
Arctic_discharge.loc[i,'FranzJosefLand_JERS1_1996_1998_err'] = 20
i=Arctic_discharge['FranzJosefLand_JERS1_1996_1997']>0
Arctic_discharge.loc[i,'FranzJosefLand_JERS1_1996_1997_err'] = 20
i=Arctic_discharge['NovayaZemlya_ALOS1_2008_2010']>0
Arctic_discharge.loc[i,'NovayaZemlya_ALOS1_2008_2010_err'] = 10
i=Arctic_discharge['NovayaZemlya_JERS1_1998']>0
Arctic_discharge.loc[i,'NovayaZemlya_JERS1_1998_err'] = 20
i=Arctic_discharge['SevernayaZemlya_ALOS1_2010']>0
Arctic_discharge.loc[i,'SevernayaZemlya_ALOS1_2010_err'] = 10
i=Arctic_discharge['SevernayaZemlya_ERS1_1991_1992']>0
Arctic_discharge.loc[i,'SevernayaZemlya_ERS1_1991_1992_err'] = 40

#For sentinel-1 velocities we need to convert them to m/yr not m/d
for column_name in ['S1_2015_mag_se', 'S1_2016_mag_se','S1_2017_mag_se','S1_2018_mag_se','S1_2019_mag_se','S1_2015_mag','S1_2016_mag','S1_2017_mag','S1_2018_mag','S1_2019_mag',
                   'S1_2015_x', 'S1_2016_x','S1_2017_x','S1_2018_x','S1_2019_x','S1_2015_y', 'S1_2016_y','S1_2017_y','S1_2018_y','S1_2019_y']:
    Arctic_discharge.loc[:,column_name]=Arctic_discharge[column_name]*365

In [6]:
#Replaec for MEaSURES data in Greenland
GM_errx = ['GM_2000_2001_ex', 'GM_2005_2006_ex', 'GM_2006_2007_ex', 'GM_2007_2008_ex', 'GM_2008_2009_ex', 'GM_2009_2010_ex', 'GM_2012_2013_ex', 'GM_2014_2015_ex', 'GM_2015_2016_ex', 'GM_2016_2017_ex', 'GM_2017_2018_ex']
GM_erry = ['GM_2000_2001_ey', 'GM_2005_2006_ey', 'GM_2006_2007_ey', 'GM_2007_2008_ey', 'GM_2008_2009_ey', 'GM_2009_2010_ey', 'GM_2012_2013_ey', 'GM_2014_2015_ey', 'GM_2015_2016_ey', 'GM_2016_2017_ey', 'GM_2017_2018_ey']
GM_v = ['GM_2000_2001_vv', 'GM_2005_2006_vv', 'GM_2006_2007_vv', 'GM_2007_2008_vv', 'GM_2008_2009_vv', 'GM_2009_2010_vv', 'GM_2012_2013_vv', 'GM_2014_2015_vv', 'GM_2015_2016_vv', 'GM_2016_2017_vv', 'GM_2017_2018_vv']
GM_vx = ['GM_2000_2001_vx', 'GM_2005_2006_vx', 'GM_2006_2007_vx', 'GM_2007_2008_vx', 'GM_2008_2009_vx', 'GM_2009_2010_vx', 'GM_2012_2013_vx', 'GM_2014_2015_vx', 'GM_2015_2016_vx', 'GM_2016_2017_vx', 'GM_2017_2018_vx']
GM_vy = ['GM_2000_2001_vy', 'GM_2005_2006_vy', 'GM_2006_2007_vy', 'GM_2007_2008_vy', 'GM_2008_2009_vy', 'GM_2009_2010_vy', 'GM_2012_2013_vy', 'GM_2014_2015_vy', 'GM_2015_2016_vy', 'GM_2016_2017_vy', 'GM_2017_2018_vy']

IL_v_err = ['2001_v_err', '2006_v_err', '2007_v_err', '2008_v_err', '2009_v_err', '2010_v_err', '2013_v_err', '2015_v_err', '2016_v_err', '2017_v_err', '2018_v_err']
IL_v = ['2001_v', '2006_v', '2007_v', '2008_v', '2009_v', '2010_v', '2013_v', '2015_v', '2016_v', '2017_v', '2018_v']
IL_vx = ['2001_vx', '2006_vx', '2007_vx', '2008_vx', '2009_vx', '2010_vx', '2013_vx', '2015_vx', '2016_vx', '2017_vx', '2018_vx']
IL_vy = ['2001_vy', '2006_vy', '2007_vy', '2008_vy', '2009_vy', '2010_vy', '2013_vy', '2015_vy', '2016_vy', '2017_vy', '2018_vy']

# %matplotlib notebook
for n in range(0,len(GM_errx)):
    GM_verr_field = 'GM_' + str(IL_v[n]) + '_err'
    GM_err_ratio_field = 'GM_' + str(IL_v[n]) + '_err_ratio'
    Arctic_discharge[GM_verr_field] = np.sqrt(Arctic_discharge[GM_errx[n]]**2+Arctic_discharge[GM_erry[n]]**2)
    Arctic_discharge[GM_err_ratio_field] = Arctic_discharge[GM_verr_field]/Arctic_discharge[GM_v[n]]
    
    for i in range(0,len(Arctic_discharge)):
        if (Arctic_discharge[GM_err_ratio_field][i]<(Arctic_discharge[IL_v_err[n]][i]/Arctic_discharge[IL_v[n]][i])):
            Arctic_discharge.loc[i,IL_v_err[n]] = Arctic_discharge[GM_verr_field][i]
            Arctic_discharge.loc[i,IL_v[n]] = Arctic_discharge[GM_v[n]][i]
            Arctic_discharge.loc[i,IL_vx[n]] = Arctic_discharge[GM_vx[n]][i]
            Arctic_discharge.loc[i,IL_vy[n]] = Arctic_discharge[GM_vy[n]][i]

In [7]:
#Replace for Sentinel-1 data
S1_v_err = ['S1_2015_mag_se', 'S1_2016_mag_se','S1_2017_mag_se','S1_2018_mag_se']#,'S1_2019_mag_se']
S1_v = ['S1_2015_mag','S1_2016_mag','S1_2017_mag','S1_2018_mag']#,'S1_2019_mag']
S1_vx = ['S1_2015_x', 'S1_2016_x','S1_2017_x','S1_2018_x']#,'S1_2019_x']
S1_vy = ['S1_2015_y', 'S1_2016_y','S1_2017_y','S1_2018_y']#,'S1_2019_y']

IL_v_err = ['2015_v_err', '2016_v_err', '2017_v_err', '2018_v_err']#, '2019_v_err']
IL_v = ['2015_v', '2016_v', '2017_v', '2018_v']#, '2019_v']
IL_vx = ['2015_vx', '2016_vx', '2017_vx', '2018_vx']#, '2019_vx']
IL_vy = ['2015_vy', '2016_vy', '2017_vy', '2018_vy']#, '2019_vy']

# %matplotlib notebook
for n in range(0,len(S1_v_err)):
    for i in range(0,len(Arctic_discharge)):
        if (((Arctic_discharge[S1_v_err[n]][i]/Arctic_discharge[S1_v[n]][i])<(Arctic_discharge[IL_v_err[n]][i]/Arctic_discharge[IL_v[n]][i])))|np.isnan(Arctic_discharge[IL_v[n]][i]):
            Arctic_discharge.loc[i,IL_v_err[n]] = Arctic_discharge[S1_v_err[n]][i]
            Arctic_discharge.loc[i,IL_v[n]] = Arctic_discharge[S1_v[n]][i]
            Arctic_discharge.loc[i,IL_vx[n]] = Arctic_discharge[S1_vx[n]][i]
            Arctic_discharge.loc[i,IL_vy[n]] = Arctic_discharge[S1_vy[n]][i]

v_vals = ['1999_v','2000_v','2001_v','2002_v','2003_v','2004_v','2005_v','2006_v','2007_v','2008_v','2009_v','2010_v','2011_v','2012_v','2013_v','2014_v','2015_v','2016_v','2017_v','2018_v']
err_vals = ['1999_v_err','2000_v_err','2001_v_err','2002_v_err','2003_v_err','2004_v_err','2005_v_err','2006_v_err','2007_v_err','2008_v_err','2009_v_err','2010_v_err','2011_v_err','2012_v_err','2013_v_err','2014_v_err','2015_v_err','2016_v_err','2017_v_err','2018_v_err']

for i in range(0,len(v_vals)):
    ratio_vals=str(err_vals[i])+'_ratio'
    Arctic_discharge.loc[Arctic_discharge[v_vals[i]]==0,v_vals[i]]=np.nan#replace values of 0 velocity with nan to avoid problems with dividing by 0
    Arctic_discharge[ratio_vals]=Arctic_discharge[err_vals[i]]/Arctic_discharge[v_vals[i]]
    
Arctic_discharge['2000_2010_min_vel_ratio'] = Arctic_discharge[['2000_v_err_ratio', '2001_v_err_ratio', '2002_v_err_ratio', '2003_v_err_ratio', '2004_v_err_ratio', '2005_v_err_ratio', '2006_v_err_ratio', '2007_v_err_ratio', '2008_v_err_ratio', '2009_v_err_ratio']].min(axis=1,skipna=True)
Arctic_discharge['2010_2018_min_vel_ratio'] = Arctic_discharge[['2010_v_err_ratio', '2011_v_err_ratio', '2012_v_err_ratio', '2013_v_err_ratio', '2014_v_err_ratio', '2015_v_err_ratio', '2016_v_err_ratio', '2017_v_err_ratio', '2018_v_err_ratio']].min(axis=1,skipna=True)

In [8]:
#Figure out how many good obs we have, so we can throw out bad obs
Arctic_discharge['num_bad_obs_snr_too_high_early'] = 0
Arctic_discharge['num_good_obs_early'] = 0
Arctic_discharge['num_nan_obs_early'] = 0

for i in range(0,len(Arctic_discharge)):
    for err_ratio_field in ['2000_v_err_ratio', '2001_v_err_ratio', '2002_v_err_ratio', '2003_v_err_ratio', '2004_v_err_ratio', '2005_v_err_ratio', '2006_v_err_ratio', '2007_v_err_ratio', '2008_v_err_ratio', '2009_v_err_ratio']:
        if math.isnan(Arctic_discharge[err_ratio_field][i]):
            Arctic_discharge.loc[i,'num_nan_obs_early'] = Arctic_discharge['num_nan_obs_early'][i]+1
        elif Arctic_discharge[err_ratio_field][i] >= 1:
            Arctic_discharge.loc[i,'num_bad_obs_snr_too_high_early'] = Arctic_discharge['num_bad_obs_snr_too_high_early'][i]+1
        elif Arctic_discharge[err_ratio_field][i] < 1:
            Arctic_discharge.loc[i,'num_good_obs_early'] = Arctic_discharge['num_good_obs_early'][i]+1
            
#put a nan in places where the errors are high and we have good data elsewhere.
for i in range(0,len(Arctic_discharge)):
    if Arctic_discharge['num_good_obs_early'][i]>=1:
        for err_ratio_field in ['2000_v_err_ratio', '2001_v_err_ratio', '2002_v_err_ratio', '2003_v_err_ratio', '2004_v_err_ratio', '2005_v_err_ratio', '2006_v_err_ratio', '2007_v_err_ratio', '2008_v_err_ratio', '2009_v_err_ratio']:
            if Arctic_discharge[err_ratio_field][i]>=1:
                Arctic_discharge.loc[i,err_ratio_field[0:6]]=np.nan #get the velocity field and set to nan
                Arctic_discharge.loc[i,err_ratio_field[0:6]]=np.nan # get the error field and set to nan
                Arctic_discharge.loc[i,err_ratio_field[0:6]+'y']=np.nan # get the x field and set to nan
                Arctic_discharge.loc[i,err_ratio_field[0:6]+'y']=np.nan # get the y field and set to nan
                
Arctic_discharge['num_bad_obs_snr_too_high_late'] = 0
Arctic_discharge['num_good_obs_late'] = 0
Arctic_discharge['num_nan_obs_late'] = 0

for i in range(0,len(Arctic_discharge)):
    for err_ratio_field in ['2000_v_err_ratio', '2001_v_err_ratio', '2002_v_err_ratio', '2003_v_err_ratio', '2004_v_err_ratio', '2005_v_err_ratio', '2006_v_err_ratio', '2007_v_err_ratio', '2008_v_err_ratio', '2009_v_err_ratio']:
        if math.isnan(Arctic_discharge[err_ratio_field][i]):
            Arctic_discharge.loc[i,'num_nan_obs_late'] = Arctic_discharge['num_nan_obs_late'][i]+1
        elif Arctic_discharge[err_ratio_field][i] >= 1:
            Arctic_discharge.loc[i,'num_bad_obs_snr_too_high_late'] = Arctic_discharge['num_bad_obs_snr_too_high_late'][i]+1
        elif Arctic_discharge[err_ratio_field][i] < 1:
            Arctic_discharge.loc[i,'num_good_obs_late'] = Arctic_discharge['num_good_obs_late'][i]+1
            
#put a nan in places where the errors are high and we have good data elsewhere.
for i in range(0,len(Arctic_discharge)):
    if Arctic_discharge['num_good_obs_late'][i]>=1:
        for err_ratio_field in ['2010_v_err_ratio', '2011_v_err_ratio', '2012_v_err_ratio', '2013_v_err_ratio', '2014_v_err_ratio', '2015_v_err_ratio', '2016_v_err_ratio', '2017_v_err_ratio', '2018_v_err_ratio']:
            if Arctic_discharge[err_ratio_field][i]>=1:
                Arctic_discharge.loc[i,err_ratio_field[0:6]]=np.nan #get the velocity field and set to nan
                Arctic_discharge.loc[i,err_ratio_field[0:6]]=np.nan # get the error field and set to nan
                Arctic_discharge.loc[i,err_ratio_field[0:6]+'y']=np.nan # get the x field and set to nan
                Arctic_discharge.loc[i,err_ratio_field[0:6]+'y']=np.nan # get the y field and set to nan

In [9]:
#Create velocity products
#Fill the uncertainty from custom velocity maps with 0.4 m/yr (bedrock movement in these maps)
northern_vel_idx = Arctic_discharge['0000_v'].isna()&Arctic_discharge['Northern_velocity'].notna()
Arctic_discharge['Northern_vel_err']=np.nan
Arctic_discharge.loc[northern_vel_idx&Arctic_discharge['Northern_velocity']>=30,'Northern_vel_err']=30
Arctic_discharge.loc[northern_vel_idx&Arctic_discharge['Northern_velocity']<30,'Northern_vel_err']=Arctic_discharge['Northern_velocity']

#insert northern_velocities where there are holes in ITS_LIVE
Arctic_discharge['0000_v'].fillna(Arctic_discharge['Northern_velocity'], inplace=True)
#calculate errors
Arctic_discharge['0000_v_err'].fillna(Arctic_discharge['Northern_vel_err'], inplace=True)

#figure out the direction of flow (use mean)
Arctic_discharge.loc[:,'2000_2010_mean_x'] = Arctic_discharge[['2000_vx','2001_vx','2002_vx','2003_vx','2004_vx','2005_vx','2006_vx','2007_vx','2008_vx','2009_vx']].mean(axis=1,skipna=True)
Arctic_discharge.loc[:,'2000_2010_mean_y'] = Arctic_discharge[['2000_vy','2001_vy','2002_vy','2003_vy','2004_vy','2005_vy','2006_vy','2007_vy','2008_vy','2009_vy']].mean(axis=1,skipna=True)
Arctic_discharge.loc[:,'2010_2018_mean_x'] = Arctic_discharge[['2010_vx','2011_vx','2012_vx','2013_vx','2014_vx','2015_vx','2016_vx','2017_vx','2018_vx']].mean(axis=1,skipna=True)
Arctic_discharge.loc[:,'2010_2018_mean_y'] = Arctic_discharge[['2010_vy','2011_vy','2012_vy','2013_vy','2014_vy','2015_vy','2016_vy','2017_vy','2018_vy']].mean(axis=1,skipna=True)
Arctic_discharge.loc[:,'2000_2010_mean_count'] = Arctic_discharge[['2000_count','2001_count','2002_count','2003_count','2004_count','2005_count','2006_count','2007_count','2008_count','2009_count']].mean(axis=1,skipna=True)
Arctic_discharge.loc[:,'2010_2018_mean_count'] = Arctic_discharge[['2010_count','2011_count','2012_count','2013_count','2014_count','2015_count','2016_count','2017_count','2018_count']].mean(axis=1,skipna=True)
#add in x and y component errors
Arctic_discharge.loc[:,'2000_2010_mean_x_err'] = Arctic_discharge[['2000_vx_err','2001_vx_err','2002_vx_err','2003_vx_err','2004_vx_err','2005_vx_err','2006_vx_err','2007_vx_err','2008_vx_err','2009_vx_err']].mean(axis=1,skipna=True)
Arctic_discharge.loc[:,'2000_2010_mean_y_err'] = Arctic_discharge[['2000_vy_err','2001_vy_err','2002_vy_err','2003_vy_err','2004_vy_err','2005_vy_err','2006_vy_err','2007_vy_err','2008_vy_err','2009_vy_err']].mean(axis=1,skipna=True)
Arctic_discharge.loc[:,'2010_2018_mean_x_err'] = Arctic_discharge[['2010_vx_err','2011_vx_err','2012_vx_err','2013_vx_err','2014_vx_err','2015_vx_err','2016_vx_err','2017_vx_err','2018_vx_err']].mean(axis=1,skipna=True)
Arctic_discharge.loc[:,'2010_2018_mean_y_err'] = Arctic_discharge[['2010_vy_err','2011_vy_err','2012_vy_err','2013_vy_err','2014_vy_err','2015_vy_err','2016_vy_err','2017_vy_err','2018_vy_err']].mean(axis=1,skipna=True)

#Take a mean including other years
Arctic_discharge.loc[:,'2000_2010_mean'] = Arctic_discharge[['2000_v','2001_v','2002_v','2003_v','2004_v','2005_v','2006_v','2007_v','2008_v','2009_v',\
    'Austfonna_ERS_1995','FranzJosefLand_JERS1_1996_1998', 'FranzJosefLand_JERS1_1996_1997',\
    'NovayaZemlya_ALOS1_2008_2010', 'NovayaZemlya_JERS1_1998','SevernayaZemlya_ALOS1_2010',\
    'SevernayaZemlya_ERS1_1991_1992','Svalbard_ALOS1_2007_2011', 'Svalbard_ERS1_1992','Svalbard_JERS1_1994_1998', 'Svalbard_TSX_2008_2012',\
     ]].mean(axis=1,skipna=True)
Arctic_discharge['2000_2010_mean'].fillna(Arctic_discharge['Northern_velocity'], inplace=True)
#fill velocity data for Barents Sea glacier that has no velocity observations
mask = (Arctic_discharge['RGIId']=='RGI60-09.01069')|(Arctic_discharge['RGIId']=='RGI60-10.00004')
Arctic_discharge.loc[mask, '2000_2010_mean']=Arctic_discharge.loc[mask, '2000_2010_mean'].fillna(5)#fill Barents sea and de long glaciers with 5
Arctic_discharge.loc[:,'2000_2010_mean_err'] = Arctic_discharge[['2000_v_err','2001_v_err','2002_v_err','2003_v_err','2004_v_err','2005_v_err','2006_v_err','2007_v_err','2008_v_err','2009_v_err',\
    'Austfonna_ERS_1995_err','FranzJosefLand_JERS1_1996_1998_err', 'FranzJosefLand_JERS1_1996_1997_err',\
    'NovayaZemlya_ALOS1_2008_2010_err', 'NovayaZemlya_JERS1_1998_err','SevernayaZemlya_ALOS1_2010_err',\
    'SevernayaZemlya_ERS1_1991_1992_err','Svalbard_ALOS1_2007_2011_err', 'Svalbard_ERS1_1992_err','Svalbard_JERS1_1994_1998_err', 'Svalbard_TSX_2008_2012_err',\
    ]].mean(axis=1,skipna=True)
Arctic_discharge['2000_2010_mean_err'].fillna(Arctic_discharge['Northern_vel_err'], inplace=True)
Arctic_discharge.loc[mask, '2000_2010_mean_err']=Arctic_discharge.loc[mask, '2000_2010_mean_err'].fillna(5) #fill error with 5 for Barents sea and de long glaciers

Arctic_discharge['2010_2018_mean'] = Arctic_discharge[['2010_v','2011_v','2012_v','2013_v','2014_v','2015_v','2016_v','2017_v','2018_v',\
    'FranzJosefLand_ALOS1_2010_2011','FranzJosefLand_ALOS2_2016']].mean(axis=1,skipna=True)
Arctic_discharge['2010_2018_mean'].fillna(Arctic_discharge['Northern_velocity'], inplace=True)
Arctic_discharge.loc[mask, '2010_2018_mean']=Arctic_discharge.loc[mask, '2010_2018_mean'].fillna(5) #fill Barents sea and de long glaciers with 5
Arctic_discharge.loc[:,'2010_2018_mean_err'] = Arctic_discharge[['2010_v_err','2011_v_err','2012_v_err','2013_v_err','2014_v_err','2015_v_err','2016_v_err','2017_v_err','2018_v_err',\
    'FranzJosefLand_ALOS1_2010_2011_err','FranzJosefLand_ALOS2_2016_err']].mean(axis=1,skipna=True)
Arctic_discharge['2010_2018_mean_err'].fillna(Arctic_discharge['Northern_vel_err'], inplace=True)
Arctic_discharge.loc[mask, '2010_2018_mean_err']=Arctic_discharge.loc[mask, '2010_2018_mean_err'].fillna(5) #fill error with 5 for Barents sea and de long glaciers

#count velocity obs for each point so that we can use the number of observations in our error analysis
Arctic_discharge['number_velocity_obs_early']=Arctic_discharge[['2000_v','2001_v','2002_v','2003_v','2004_v','2005_v','2006_v','2007_v','2008_v','2009_v',\
    'Austfonna_ERS_1995','FranzJosefLand_JERS1_1996_1998', 'FranzJosefLand_JERS1_1996_1997',\
    'NovayaZemlya_ALOS1_2008_2010', 'NovayaZemlya_JERS1_1998','SevernayaZemlya_ALOS1_2010',\
    'SevernayaZemlya_ERS1_1991_1992','Svalbard_ALOS1_2007_2011', 'Svalbard_ERS1_1992','Svalbard_JERS1_1994_1998', 'Svalbard_TSX_2008_2012']].count(axis=1)

Arctic_discharge['number_velocity_obs_late']=Arctic_discharge[['2000_v_err','2001_v_err','2002_v_err','2003_v_err','2004_v_err','2005_v_err','2006_v_err','2007_v_err','2008_v_err','2009_v_err',\
    'Austfonna_ERS_1995_err','FranzJosefLand_JERS1_1996_1998_err', 'FranzJosefLand_JERS1_1996_1997_err',\
    'NovayaZemlya_ALOS1_2008_2010_err', 'NovayaZemlya_JERS1_1998_err','SevernayaZemlya_ALOS1_2010_err',\
    'SevernayaZemlya_ERS1_1991_1992_err','Svalbard_ALOS1_2007_2011_err', 'Svalbard_ERS1_1992_err','Svalbard_JERS1_1994_1998_err', 'Svalbard_TSX_2008_2012_err']].count(axis=1)

#if there are more than 10, replace with 10
Arctic_discharge.loc[Arctic_discharge['number_velocity_obs_early']>10,'number_velocity_obs_early']=10
Arctic_discharge.loc[Arctic_discharge['number_velocity_obs_late']>10,'number_velocity_obs_late']=10

#divide error by the sqrt of observations
Arctic_discharge.loc[:,'2000_2010_mean_err']=Arctic_discharge['2000_2010_mean_err']/np.sqrt(Arctic_discharge['number_velocity_obs_early'])
Arctic_discharge.loc[:,'2010_2018_mean_err']=Arctic_discharge['2010_2018_mean_err']/np.sqrt(Arctic_discharge['number_velocity_obs_late'])


# fill gaps in the velocity record
# fill 2000-2010 holes with newer data
Arctic_discharge.loc[:,'2010_2018_mean'] = Arctic_discharge['2010_2018_mean'].interpolate(method='nearest',limit=20) #fill no data gaps with nearby values, limit to prevent data filling in Jan Mayen where there is lots of nans
Arctic_discharge.loc[:,'2010_2018_mean_err'] = Arctic_discharge['2010_2018_mean_err'].interpolate(method='nearest',limit=20) #fill no data gaps with nearby values, limit to prevent data filling in Jan Mayen where there is lots of nans

print('number of nans in 2000-2010 velocity layer that are filled with 2010-2020',Arctic_discharge['2000_2010_mean'].isna().sum(),'out of ', len(Arctic_discharge))
print(Arctic_discharge['2000_2010_mean'].isnull().groupby(Arctic_discharge['region']).sum())

Arctic_discharge['2000_2010_mean'].fillna(Arctic_discharge['2010_2018_mean'], inplace=True)
Arctic_discharge['2000_2010_mean_err'].fillna(Arctic_discharge['2000_2010_mean_err'], inplace=True)

print(np.mean(Arctic_discharge['2000_2010_mean_err']))
#if the error is bigger than the velocity, replace with the velocity value

i = Arctic_discharge['2000_2010_mean_err']>Arctic_discharge['2000_2010_mean']
Arctic_discharge.loc[i,'2000_2010_mean_err']=Arctic_discharge['2000_2010_mean']
i = Arctic_discharge['2010_2018_mean_err']>Arctic_discharge['2010_2018_mean']
Arctic_discharge.loc[i,'2010_2018_mean_err']=Arctic_discharge['2010_2018_mean']
print(np.mean(Arctic_discharge['2000_2010_mean_err']))

Arctic_discharge['2000_2010_mean'].fillna(value = 0, inplace = True)
Arctic_discharge['2000_2010_mean_err'].fillna(value = 0, inplace = True)
Arctic_discharge['2010_2018_mean'].fillna(value = 0, inplace = True)
Arctic_discharge['2010_2018_mean_err'].fillna(value = 0, inplace = True)

#Jan Mayen ice velocity
mask = Arctic_discharge['RGIId']=='RGI60-07.01575' # Weyprechtbreen
Arctic_discharge.loc[mask, '2000_2010_mean']=Arctic_discharge.loc[mask, '2000_2010_mean'].replace(0,30)
Arctic_discharge.loc[mask, '2000_2010_mean_err']=Arctic_discharge.loc[mask, '2000_2010_mean_err'].replace(0,20)
Arctic_discharge.loc[mask, '2010_2018_mean']=Arctic_discharge.loc[mask, '2010_2018_mean'].replace(0,30)
Arctic_discharge.loc[mask, '2010_2018_mean_err']=Arctic_discharge.loc[mask, '2010_2018_mean_err'].replace(0,20)
mask = Arctic_discharge['RGIId']=='RGI60-07.01577'#Kjerulfbreen
Arctic_discharge.loc[mask, '2000_2010_mean']=Arctic_discharge.loc[mask, '2000_2010_mean'].replace(0,25)
Arctic_discharge.loc[mask, '2000_2010_mean_err']=Arctic_discharge.loc[mask, '2000_2010_mean_err'].replace(0,15)
Arctic_discharge.loc[mask, '2010_2018_mean']=Arctic_discharge.loc[mask, '2010_2018_mean'].replace(0,25)
Arctic_discharge.loc[mask, '2010_2018_mean_err']=Arctic_discharge.loc[mask, '2010_2018_mean_err'].replace(0,15)
mask = Arctic_discharge['RGIId']=='RGI60-07.01576' #Gjuvbreen
Arctic_discharge.loc[mask, '2000_2010_mean']=Arctic_discharge.loc[mask, '2000_2010_mean'].replace(0,12)
Arctic_discharge.loc[mask, '2000_2010_mean_err']=Arctic_discharge.loc[mask, '2000_2010_mean_err'].replace(0,5)
Arctic_discharge.loc[mask, '2010_2018_mean']=Arctic_discharge.loc[mask, '2010_2018_mean'].replace(0,12)
Arctic_discharge.loc[mask, '2010_2018_mean_err']=Arctic_discharge.loc[mask, '2010_2018_mean_err'].replace(0,5)
mask = Arctic_discharge['RGIId']=='RGI60-07.01581'#Frielebreen
Arctic_discharge.loc[mask, '2000_2010_mean']=Arctic_discharge.loc[mask, '2000_2010_mean'].replace(0,18)
Arctic_discharge.loc[mask, '2000_2010_mean_err']=Arctic_discharge.loc[mask, '2000_2010_mean_err'].replace(0,5)
Arctic_discharge.loc[mask, '2010_2018_mean']=Arctic_discharge.loc[mask, '2010_2018_mean'].replace(0,18)
Arctic_discharge.loc[mask, '2010_2018_mean_err']=Arctic_discharge.loc[mask, '2010_2018_mean_err'].replace(0,5)
mask = Arctic_discharge['RGIId']=='RGI60-07.01582'#Pr Haralds Breen
Arctic_discharge.loc[mask, '2000_2010_mean']=Arctic_discharge.loc[mask, '2000_2010_mean'].replace(0,20)
Arctic_discharge.loc[mask, '2000_2010_mean_err']=Arctic_discharge.loc[mask, '2000_2010_mean_err'].replace(0,10)
Arctic_discharge.loc[mask, '2010_2018_mean']=Arctic_discharge.loc[mask, '2010_2018_mean'].replace(0,20)
Arctic_discharge.loc[mask, '2010_2018_mean_err']=Arctic_discharge.loc[mask, '2010_2018_mean_err'].replace(0,10)
mask = Arctic_discharge['RGIId']=='RGI60-07.01578'#Svend Foynbreen
Arctic_discharge.loc[mask, '2000_2010_mean']=Arctic_discharge.loc[mask, '2000_2010_mean'].replace(0,12)
Arctic_discharge.loc[mask, '2000_2010_mean_err']=Arctic_discharge.loc[mask, '2000_2010_mean_err'].replace(0,5)
Arctic_discharge.loc[mask, '2010_2018_mean']=Arctic_discharge.loc[mask, '2010_2018_mean'].replace(0,12)
Arctic_discharge.loc[mask, '2010_2018_mean_err']=Arctic_discharge.loc[mask, '2010_2018_mean_err'].replace(0,5)

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:21: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:22: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.

number of nans in 2000-2010 velocity layer that are filled with 2010-2020 7584 out of  200643
region
Alaska         46
Canada         69
Greenland      40
Iceland       120
Russia       4629
Svalbard     2680
Name: 2000_2010_mean, dtype: int64
inf
40.75927212348829


In [10]:
#figure out glacier thickness
Arctic_discharge['tomo_thickness'].replace(0, np.nan, inplace=True)
Arctic_discharge.loc[Arctic_discharge['tomo_thickness']<0,'tomo_thickness']=np.nan #some large negative values that we delete here
Arctic_discharge['best_thickness']=Arctic_discharge['tomo_thickness']
print('Number of data points filled with tomo data',Arctic_discharge['best_thickness'].count())
Arctic_discharge['best_thickness'].fillna(Arctic_discharge['THICKNESS'], inplace=True) #if tomo is null, then fill with GLATHIDA
print('Number of data points filled with tomo data and GlaThiDa',Arctic_discharge['best_thickness'].count())
Arctic_discharge['best_thickness'].fillna(Arctic_discharge['Russian_thickness'], inplace=True) #if tomo+GlaThiDa are null, then fill with Russian thickness data
print('Number of data points filled with tomo data, GlaThiDa, and Russian data',Arctic_discharge['best_thickness'].count())

#insert uncertainties
# Arctic_discharge=Arctic_discharge.reset_index()
observation_idx=Arctic_discharge['THICKNESS'].notna()|Arctic_discharge['Russian_thickness'].notna()|Arctic_discharge['tomo_thickness'].notna()
Arctic_discharge['thickness_uncertainty']=Arctic_discharge['THICKNESS_UNCERTAINTY']#start by filling with all GlaThiDa data
Arctic_discharge.loc[observation_idx,'thickness_uncertainty']=0.10*Arctic_discharge[observation_idx]['THICKNESS']

#fill in GlaThiDa uncertainties where you have them
glathida_uncertain_idx=(Arctic_discharge['THICKNESS'].notna())&(Arctic_discharge['THICKNESS_UNCERTAINTY'].notna())
Arctic_discharge.loc[glathida_uncertain_idx,'thickness_uncertainty']=Arctic_discharge['THICKNESS_UNCERTAINTY'][glathida_uncertain_idx]#start by filling with all GlaThiDa data

#calculate data you need for interpolated thickness modeling
#Calculate your own U shaped valley for all glaciers and fill in holes in observations
Arctic_discharge['position']=Arctic_discharge.index #create positions
Arctic_discharge_position_counter = Arctic_discharge[['RGIId','position']].groupby(['RGIId']).min().reset_index()
Arctic_discharge_position_counter = Arctic_discharge_position_counter.rename(columns={"position": "min_position"})
Arctic_discharge=Arctic_discharge.merge(Arctic_discharge_position_counter,how='left',left_on='RGIId',right_on='RGIId')

Arctic_discharge_position_counter = Arctic_discharge[['RGIId','position']].groupby(['RGIId']).max().reset_index()
Arctic_discharge_position_counter = Arctic_discharge_position_counter.rename(columns={"position": "max_position"})
Arctic_discharge=Arctic_discharge.merge(Arctic_discharge_position_counter,how='left',left_on='RGIId',right_on='RGIId')

Arctic_discharge['distance_across']=(Arctic_discharge['position']-Arctic_discharge['min_position'])*25#25 meters for the spacing you defined above
Arctic_discharge['position']=(Arctic_discharge['position']-Arctic_discharge['min_position'])

#create a mask to find areas near the glacier margin
Arctic_discharge['velocity_glacier_edge_mask']=0

i= (Arctic_discharge['position']<20)|((Arctic_discharge['max_position']-Arctic_discharge['min_position']-Arctic_discharge['position'])<20) #if you are within 500 m (20*25 m points), then we create a mask for velocity orientation mapping later
Arctic_discharge.loc[i,'velocity_glacier_edge_mask']=1
#drop this to avoid problems later
Arctic_discharge = Arctic_discharge.drop(columns='THICKNESS_UNCERTAINTY')

Number of data points filled with tomo data 8537
Number of data points filled with tomo data and GlaThiDa 18829
Number of data points filled with tomo data, GlaThiDa, and Russian data 50648


In [11]:
#fill everything else with 1 velocity observation
Arctic_discharge['number_velocity_obs_early'].fillna(1,inplace=True)
Arctic_discharge['number_velocity_obs_late'].fillna(1,inplace=True)
Arctic_discharge.loc[Arctic_discharge['number_velocity_obs_early']==0,'number_velocity_obs_early']=1
Arctic_discharge.loc[Arctic_discharge['number_velocity_obs_late']==0,'number_velocity_obs_late']=1

#Add more dates for thickness data to some regions and clean it up
#add in regions so that we can add SZ and FJL survey dates from Julian more easily
arctic_aois = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/vector_files/Arctic_AOIs.gpkg')
Arctic_discharge = gpd.sjoin(Arctic_discharge,arctic_aois)
Arctic_discharge = Arctic_discharge.drop(columns='index_right')

Arctic_discharge.loc[(Arctic_discharge['Region_name']=='FJL')&(~Arctic_discharge['Region_name'].isna()),'thickness_obs_date']=1994
Arctic_discharge.loc[(Arctic_discharge['Region_name']=='SZ')&(~Arctic_discharge['Region_name'].isna()),'thickness_obs_date']=1997

Arctic_discharge['SURVEY_DATE'] = Arctic_discharge['SURVEY_DATE'].astype(np.float)
#Put GlaThiDa dates into year format and put them in the thickness_obs_date column

for i in range(0,len(Arctic_discharge)):
    if ~np.isnan(Arctic_discharge['SURVEY_DATE'][i]):
#         print(int(str(Arctic_discharge['SURVEY_DATE'][i])[0:4]))
        Arctic_discharge.loc[i,'thickness_obs_date']=int(str(Arctic_discharge['SURVEY_DATE'][i])[0:4])
            
#fill missing GlaThiDa dates with 2000
Arctic_discharge.loc[Arctic_discharge['THICKNESS'].astype(np.float)>0,'thickness_obs_date'].fillna(2000)

#assign thickness dates from obs and give everything else a date of 2000
Arctic_discharge['thickness_date']=Arctic_discharge['thickness_obs_date']
Arctic_discharge['thickness_date'].fillna(2017,inplace=True)

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  app.launch_new_instance()
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


In [12]:
#set up data for interpolated thickness model
glacier_best_thickness=[]
glimsid = []
centerline_distance_across = []
centerline_thickness_obs_date = []

for i in range(0,len(Arctic_discharge)):
    #          dividing by 5 gives you middle 20% of the glacier                             This subtraction is giving you the number of position away you are from the middle, so if you are in the middle 20% these calculations will happen
    if (((Arctic_discharge['max_position'][i]-Arctic_discharge['min_position'][i]))/5 >= np.abs(Arctic_discharge['position'][i]-((Arctic_discharge['max_position'][i]-Arctic_discharge['min_position'][i])/2))): #change 2.5 to include more or less points
        glimsid.append(Arctic_discharge['RGIId'][i])
        glacier_best_thickness.append(Arctic_discharge['best_thickness'][i])
        centerline_distance_across.append(Arctic_discharge['distance_across'][i])
        centerline_thickness_obs_date.append(Arctic_discharge['thickness_obs_date'][i])

centerline_thickness = pd.DataFrame(data={'RGIId':glimsid,'centerline_depth':glacier_best_thickness,'centerline_distance_across':centerline_distance_across,'centerline_thickness_obs_date':centerline_thickness_obs_date})
centerline_thickness=centerline_thickness.groupby(['RGIId']).mean().reset_index()
centerline_thickness['do_we_have_an_observation_in_middle_fifth']=np.nan
centerline_thickness.loc[centerline_thickness['centerline_depth']>0,'do_we_have_an_observation_in_middle_fifth']=1 #add a one if there is a centerline depth greater than 0 because this means this glacier has a centerline observation
Arctic_discharge=Arctic_discharge.merge(centerline_thickness,how='left',left_on='RGIId',right_on='RGIId')

Arctic_discharge['velocity_glacier_edge_mask']=0
i= (Arctic_discharge['position']<20)|(Arctic_discharge['max_position']-Arctic_discharge['min_position']-Arctic_discharge['position']<20) #if you are within 500 m (20*25 m points), then we create a mask for velocity orientation mapping later
Arctic_discharge.loc[i,'velocity_glacier_edge_mask']=1


#calculate U shaped valley from centerline thickness
centerline_depth=Arctic_discharge['centerline_depth']
distance1=Arctic_discharge['centerline_distance_across']
distance2=np.abs(Arctic_discharge['centerline_distance_across']-Arctic_discharge['distance_across'])
Arctic_discharge.loc[:,'interpolated_thickness']=((10-centerline_depth)/distance1**2)*(distance2**2)+centerline_depth

#figure out where interpolated data will go
interp_idx = Arctic_discharge['best_thickness'].isnull()&(Arctic_discharge['interpolated_thickness']>0)
#fill with interpolated data where possible
Arctic_discharge['best_thickness'].fillna(Arctic_discharge['interpolated_thickness'], inplace=True)
#figure out where Farinotti will go
Milan_idx = Arctic_discharge['best_thickness'].isnull()
#fill gaps in best_thickness with Farinotti
Arctic_discharge['best_thickness'].fillna(Arctic_discharge['Milan_thickness'], inplace=True) #if still null, fill with Farinotti
#get uncertainties for Farinotti and intepolated
Arctic_discharge.loc[interp_idx,'thickness_uncertainty']=Arctic_discharge['interpolated_thickness']*.10 #10% error on interpolated thickness
Arctic_discharge.loc[Milan_idx,'thickness_uncertainty']=Arctic_discharge['Milan_thickness_err']


Arctic_discharge.loc[Arctic_discharge['best_thickness']<Arctic_discharge['thickness_uncertainty'],'thickness_uncertainty']=Arctic_discharge['best_thickness']

#for Svalbard, just use Farinotti
# Arctic_discharge['best_thickness'][Arctic_discharge['region']=='Svalbard']=Arctic_discharge['Milan_thickness']
# Arctic_discharge['thickness_uncertainty'][Arctic_discharge['region']=='Svalbard']=Arctic_discharge['Farinotti_thickness']*.18#18% error on Svalbard thickness inline with Furst results


In [13]:
#First interpolate 2000 dhdht, then fill it with dhdt_late data
Arctic_discharge.loc[:,'dhdt_early'] = Arctic_discharge['dhdt_early'].interpolate(method='nearest',limit=20)
Arctic_discharge.loc[:,'dhdt_err_early'] = Arctic_discharge['dhdt_err_early'].interpolate(method='nearest',limit=20)
#interpolate dhdt to fill data gaps
Arctic_discharge.loc[:,'dhdt_late'] = Arctic_discharge['dhdt_late'].interpolate(method='nearest',limit=30)
Arctic_discharge.loc[:,'dhdt_err_late'] = Arctic_discharge['dhdt_err_late'].interpolate(method='nearest',limit=30)
#fill gaps with late data
Arctic_discharge.loc[:,'dhdt_early'].fillna(Arctic_discharge['dhdt_late'],inplace=True)
Arctic_discharge.loc[:,'dhdt_err_early'].fillna(Arctic_discharge['dhdt_err_late'],inplace=True)

Arctic_discharge.loc[Arctic_discharge['best_thickness']<Arctic_discharge['thickness_uncertainty'],'thickness_uncertainty']=Arctic_discharge['best_thickness']

i=Arctic_discharge['thickness_date']<2010
Arctic_discharge.loc[i,'best_thickness_early'] = ((2005-Arctic_discharge['thickness_date'])*Arctic_discharge['dhdt_early'])+Arctic_discharge['best_thickness']
Arctic_discharge.loc[i,'best_thickness_late'] = (((2010-Arctic_discharge['thickness_date'])*Arctic_discharge['dhdt_early'])+(5*Arctic_discharge['dhdt_late']))+Arctic_discharge['best_thickness']
Arctic_discharge.loc[i,'best_thickness_early_err']=np.sqrt(((2005-Arctic_discharge['thickness_date'])*Arctic_discharge['dhdt_err_early'])**2+(Arctic_discharge['thickness_uncertainty'])**2)
Arctic_discharge.loc[i,'best_thickness_late_err']=np.sqrt((((2010-Arctic_discharge['thickness_date'])*Arctic_discharge['dhdt_err_early'])**2+(5*Arctic_discharge['dhdt_err_late']))+(Arctic_discharge['thickness_uncertainty'])**2)

i=Arctic_discharge['thickness_date']>=2010
Arctic_discharge.loc[i,'best_thickness_early'] = (-5*Arctic_discharge['dhdt_early'])+(-(Arctic_discharge['thickness_date']-2010)*Arctic_discharge['dhdt_late'])+Arctic_discharge['best_thickness']
Arctic_discharge.loc[i,'best_thickness_late'] = ((2015-Arctic_discharge['thickness_date'])*Arctic_discharge['dhdt_late'])+Arctic_discharge['best_thickness']
Arctic_discharge.loc[i,'best_thickness_early_err']=np.sqrt((-5*Arctic_discharge['dhdt_err_early'])**2+(-(Arctic_discharge['thickness_date']-2010)*Arctic_discharge['dhdt_err_late'])**2+(Arctic_discharge['thickness_uncertainty'])**2)
Arctic_discharge.loc[i,'best_thickness_late_err']=np.sqrt(((2015-Arctic_discharge['thickness_date'])*Arctic_discharge['dhdt_err_late'])**2+(Arctic_discharge['thickness_uncertainty'])**2)

#add column for data sources
Arctic_discharge['thickness_data_source']=np.nan
Arctic_discharge.loc[Arctic_discharge['tomo_thickness']>0,'thickness_data_source']=1
print(Arctic_discharge['thickness_data_source'].count())
Arctic_discharge.loc[(Arctic_discharge['THICKNESS']>0)&np.isnan(Arctic_discharge['thickness_data_source']),'thickness_data_source']=2
print(Arctic_discharge['thickness_data_source'].count())
Arctic_discharge.loc[(Arctic_discharge['Russian_thickness']>0)&np.isnan(Arctic_discharge['thickness_data_source']),'thickness_data_source']=3
print(Arctic_discharge['thickness_data_source'].count())
Arctic_discharge.loc[(Arctic_discharge['interpolated_thickness']>0)&np.isnan(Arctic_discharge['thickness_data_source']),'thickness_data_source']=4
print(Arctic_discharge['thickness_data_source'].count())
Arctic_discharge['thickness_data_source'].fillna(5,inplace=True)#fill the rest with 5 for Millan
#measured ice thickness corrected for melt
Arctic_discharge['best_measured_ice_thickness_early'] = np.nan
Arctic_discharge['best_measured_ice_thickness_late'] = np.nan
Arctic_discharge.loc[Arctic_discharge['thickness_data_source']<4,'best_measured_ice_thickness_early']=Arctic_discharge['best_thickness_early']
Arctic_discharge.loc[Arctic_discharge['thickness_data_source']<4,'best_measured_ice_thickness_late']=Arctic_discharge['best_thickness_late']
#Millan ice thickness corrected for melt
Arctic_discharge.loc[:,'Milan_ice_thickness_early'] = (-5*Arctic_discharge['dhdt_early'])+(-(2017-2010)*Arctic_discharge['dhdt_late'])+Arctic_discharge['Milan_thickness']
Arctic_discharge.loc[:,'Milan_ice_thickness_late'] = ((2015-2017)*Arctic_discharge['dhdt_late'])+Arctic_discharge['Milan_thickness']
#Farinotti ice thickness corrected for melt
Arctic_discharge.loc[:,'Farinotti_ice_thickness_early'] = ((2005-2000)*Arctic_discharge['dhdt_early'])+Arctic_discharge['Farinotti_thickness']
Arctic_discharge.loc[:,'Farinotti_ice_thickness_late'] = (((2010-2000)*Arctic_discharge['dhdt_early'])+(5*Arctic_discharge['dhdt_late']))+Arctic_discharge['Farinotti_thickness']

#we dont want negative thicknesses
Arctic_discharge.loc[Arctic_discharge['best_thickness_early']<0,'best_thickness_early']=0
Arctic_discharge.loc[Arctic_discharge['best_thickness_late']<0,'best_thickness_late']=0
Arctic_discharge.loc[Arctic_discharge['Milan_ice_thickness_early']<0,'Milan_ice_thickness_early']=0
Arctic_discharge.loc[Arctic_discharge['Milan_ice_thickness_late']<0,'Milan_ice_thickness_late']=0
Arctic_discharge.loc[Arctic_discharge['Farinotti_ice_thickness_early']<0,'Farinotti_ice_thickness_early']=0
Arctic_discharge.loc[Arctic_discharge['Farinotti_ice_thickness_late']<0,'Farinotti_ice_thickness_late']=0

Arctic_discharge['Difference_between_late_and_early_thickness'] = Arctic_discharge['best_thickness_late']-Arctic_discharge['best_thickness_early']

8537
18829
50648
74681


In [14]:
Arctic_discharge = Arctic_discharge.sort_values(by=['RGIId','distance_across'])

Arctic_discharge['bearing_to_next_point'] = np.nan
Arctic_discharge['bearing_of_velocity_early'] = np.nan
Arctic_discharge['bearing_of_velocity_late'] = np.nan
Arctic_discharge['velocity_offsett_angle_early'] = np.nan
Arctic_discharge['velocity_offsett_angle_late'] = np.nan
Arctic_discharge['claculated_fluxgate_width_early_min'] = np.nan
Arctic_discharge['claculated_fluxgate_width_late_min'] = np.nan
Arctic_discharge['claculated_fluxgate_width_early_max'] = np.nan
Arctic_discharge['claculated_fluxgate_width_late_max'] = np.nan
Arctic_discharge['velocity_offset_angle_early_min'] = np.nan
Arctic_discharge['velocity_offset_angle_early_max'] = np.nan
Arctic_discharge['velocity_offset_angle_late_min'] = np.nan
Arctic_discharge['velocity_offset_angle_late_max'] = np.nan

Arctic_discharge['length_early_min'] = 25
Arctic_discharge['length_late_min'] = 25
Arctic_discharge['length_early_max'] = 25
Arctic_discharge['length_late_max'] = 25

#calculate the angle that flow is hitting the fluxgate
for i in range(0,len(Arctic_discharge)-1):
    if (Arctic_discharge['RGIId'][i]!=Arctic_discharge['RGIId'][i+1]):
        Arctic_discharge.loc[i,'bearing_to_next_point'] = Arctic_discharge['bearing_to_next_point'][i-1]
    elif Arctic_discharge['RGIId'][i]==Arctic_discharge['RGIId'][i+1]:
        hypot=np.sqrt((Arctic_discharge['easting'][i+1]-Arctic_discharge['easting'][i])**2+(Arctic_discharge['northing'][i+1]-Arctic_discharge['northing'][i])**2)
        Arctic_discharge.loc[i,'bearing_to_next_point'] = math.atan2((Arctic_discharge['easting'][i+1]-Arctic_discharge['easting'][i])/hypot,(Arctic_discharge['northing'][i+1]-Arctic_discharge['northing'][i])/hypot)*180/math.pi #% 360
        
    Arctic_discharge.loc[i,'bearing_of_velocity_early'] = math.atan2(Arctic_discharge['2000_2010_mean_x'][i]/Arctic_discharge['2000_2010_mean'][i],Arctic_discharge['2000_2010_mean_y'][i]/Arctic_discharge['2000_2010_mean'][i])*180/math.pi+45 % 360 #need +45 to convert coordiante systems
    Arctic_discharge.loc[i,'bearing_of_velocity_late'] = math.atan2(Arctic_discharge['2010_2018_mean_x'][i]/Arctic_discharge['2010_2018_mean'][i],Arctic_discharge['2010_2018_mean_y'][i]/Arctic_discharge['2010_2018_mean'][i])*180/math.pi+45 % 360 #need +45 to convert coordiante systems
    
    #IF we dont know the velocity, then assume it is 90° for the late period, for the early period, copy the late period bearing of velocity, which is probably not 90°
    if np.isnan(Arctic_discharge['bearing_of_velocity_late'][i]):
        Arctic_discharge['bearing_of_velocity_late'][i]=90
    if np.isnan(Arctic_discharge['bearing_of_velocity_early'][i]):
        Arctic_discharge['bearing_of_velocity_early'][i]=Arctic_discharge['bearing_of_velocity_late'][i]
        
    Arctic_discharge.loc[i,'velocity_offset_angle_early'] = np.abs((Arctic_discharge['bearing_to_next_point'][i] - Arctic_discharge['bearing_of_velocity_early'][i]) % 360)
    if Arctic_discharge['velocity_offset_angle_early'][i]>180:
        Arctic_discharge.loc[i,'velocity_offset_angle_early']=360-Arctic_discharge['velocity_offset_angle_early'][i]
#
    Arctic_discharge.loc[i,'velocity_offset_angle_late'] = np.abs((Arctic_discharge['bearing_to_next_point'][i] - Arctic_discharge['bearing_of_velocity_late'][i]) % 360)
    if Arctic_discharge.loc[i,'velocity_offset_angle_late']>180:
        Arctic_discharge.loc[i,'velocity_offset_angle_late']=360-Arctic_discharge['velocity_offset_angle_late'][i]

    if (~np.isnan(Arctic_discharge['2000_2010_mean_x_err'][i])&~np.isnan(Arctic_discharge['2000_2010_mean_y_err'][i])):
        #get the min and max bearing based on velocities uncertainties
        Arctic_discharge.loc[i,'bearing_of_velocity_early_min'] = math.atan2((Arctic_discharge['2000_2010_mean_x'][i]+Arctic_discharge['2000_2010_mean_x_err'][i])/Arctic_discharge['2000_2010_mean'][i],(Arctic_discharge['2000_2010_mean_y'][i]-Arctic_discharge['2000_2010_mean_y_err'][i])/Arctic_discharge['2000_2010_mean'][i])*180/math.pi+45 % 360 #need +45 to convert coordiante systems
        Arctic_discharge.loc[i,'bearing_of_velocity_early_max'] = math.atan2((Arctic_discharge['2000_2010_mean_x'][i]-Arctic_discharge['2000_2010_mean_x_err'][i])/Arctic_discharge['2000_2010_mean'][i],(Arctic_discharge['2000_2010_mean_y'][i]+Arctic_discharge['2000_2010_mean_y_err'][i])/Arctic_discharge['2000_2010_mean'][i])*180/math.pi+45 % 360 #need +45 to convert coordiante systems
        #get the offset angle from perpendicular
        Arctic_discharge.loc[i,'velocity_offset_angle_early_min'] = np.abs((Arctic_discharge['bearing_to_next_point'][i] - Arctic_discharge['bearing_of_velocity_early_min'][i]) % 360)
        Arctic_discharge.loc[i,'velocity_offset_angle_early_max'] = np.abs((Arctic_discharge['bearing_to_next_point'][i] - Arctic_discharge['bearing_of_velocity_early_max'][i]) % 360)
    else: #if we dont have the x and y err then just assume ±10
        if Arctic_discharge['velocity_offsett_angle_early'][i]>90:
            Arctic_discharge.loc[i,'velocity_offset_angle_early_min'] = np.abs(math.sin(math.radians(Arctic_discharge['velocity_offsett_angle_early'][i]+10))*25)
            Arctic_discharge.loc[i,'velocity_offset_angle_early_max'] = np.abs(math.sin(math.radians(Arctic_discharge['velocity_offsett_angle_early'][i]-10))*25)
        if Arctic_discharge['velocity_offsett_angle_early'][i]<90:
            Arctic_discharge.loc[i,'velocity_offset_angle_early_min'] = np.abs(math.sin(math.radians(Arctic_discharge['velocity_offsett_angle_early'][i]-10))*25)
            Arctic_discharge.loc[i,'velocity_offset_angle_early_max'] = np.abs(math.sin(math.radians(Arctic_discharge['velocity_offsett_angle_early'][i]+10))*25)
        
    if (~np.isnan(Arctic_discharge['2010_2018_mean_x_err'][i])&~np.isnan(Arctic_discharge['2010_2018_mean_y_err'][i])):
        #get the min and max bearing based on velocities uncertainties
        Arctic_discharge.loc[i,'bearing_of_velocity_late_min'] = math.atan2((Arctic_discharge['2010_2018_mean_x'][i]+Arctic_discharge['2010_2018_mean_x_err'][i])/Arctic_discharge['2010_2018_mean'][i],(Arctic_discharge['2010_2018_mean_y'][i]-Arctic_discharge['2010_2018_mean_y_err'][i])/Arctic_discharge['2010_2018_mean'][i])*180/math.pi+45 % 360 #need +45 to convert coordiante systems
        Arctic_discharge.loc[i,'bearing_of_velocity_late_max'] = math.atan2((Arctic_discharge['2010_2018_mean_x'][i]-Arctic_discharge['2010_2018_mean_x_err'][i])/Arctic_discharge['2010_2018_mean'][i],(Arctic_discharge['2010_2018_mean_y'][i]+Arctic_discharge['2010_2018_mean_y_err'][i])/Arctic_discharge['2010_2018_mean'][i])*180/math.pi+45 % 360 #need +45 to convert coordiante systems
        #get the offset angle from perpendicular
        Arctic_discharge.loc[i,'velocity_offset_angle_late_min'] = np.abs((Arctic_discharge['bearing_to_next_point'][i] - Arctic_discharge['bearing_of_velocity_late_min'][i]) % 360)
        Arctic_discharge.loc[i,'velocity_offset_angle_late_max'] = np.abs((Arctic_discharge['bearing_to_next_point'][i] - Arctic_discharge['bearing_of_velocity_late_max'][i]) % 360)
    else: #if we dont have the x and y err then just assume ±10
        if Arctic_discharge['velocity_offsett_angle_late'][i]>90:
            Arctic_discharge.loc[i,'velocity_offset_angle_late_min'] = np.abs(math.sin(math.radians(Arctic_discharge['velocity_offsett_angle_late'][i]+10))*25)
            Arctic_discharge.loc[i,'velocity_offset_angle_late_max'] = np.abs(math.sin(math.radians(Arctic_discharge['velocity_offsett_angle_late'][i]-10))*25)
        if Arctic_discharge['velocity_offsett_angle_late'][i]<90:
            Arctic_discharge.loc[i,'velocity_offset_angle_late_min'] = np.abs(math.sin(math.radians(Arctic_discharge['velocity_offsett_angle_late'][i]-10))*25)
            Arctic_discharge.loc[i,'velocity_offset_angle_late_max'] = np.abs(math.sin(math.radians(Arctic_discharge['velocity_offsett_angle_late'][i]+10))*25)
        
    #fix big angles
    if Arctic_discharge['velocity_offset_angle_early_min'][i]>180:
        Arctic_discharge.loc[i,'velocity_offset_angle_early_min']=360-Arctic_discharge['velocity_offset_angle_early_min'][i]
    if Arctic_discharge['velocity_offset_angle_early_max'][i]>180:
        Arctic_discharge.loc[i,'velocity_offset_angle_early_max']=360-Arctic_discharge['velocity_offset_angle_early_max'][i]
    if Arctic_discharge['velocity_offset_angle_late_min'][i]>180:
        Arctic_discharge.loc[i,'velocity_offset_angle_late_min']=360-Arctic_discharge['velocity_offset_angle_late_min'][i]
    if Arctic_discharge['velocity_offset_angle_late_max'][i]>180:
        Arctic_discharge.loc[i,'velocity_offset_angle_late_max']=360-Arctic_discharge['velocity_offset_angle_late_max'][i]

    #convert to degrees needed in next steps
    Arctic_discharge.loc[i,'offset_length_early_min'] = np.abs(math.sin(math.radians(Arctic_discharge['velocity_offset_angle_early_min'][i]))*25)
    Arctic_discharge.loc[i,'offset_length_early_max'] = np.abs(math.sin(math.radians(Arctic_discharge['velocity_offset_angle_early_max'][i]))*25)
    Arctic_discharge.loc[i,'offset_length_late_min'] = np.abs(math.sin(math.radians(Arctic_discharge['velocity_offset_angle_late_min'][i]))*25)
    Arctic_discharge.loc[i,'offset_length_late_max'] = np.abs(math.sin(math.radians(Arctic_discharge['velocity_offset_angle_late_max'][i]))*25)

    #use the angle that it should be directly from the offset angle, dont consider uncertainties
    Arctic_discharge.loc[i,'length_early_avg'] = np.abs(math.sin(math.radians(Arctic_discharge['velocity_offset_angle_early'][i]))*25)
    Arctic_discharge.loc[i,'length_late_avg'] = np.abs(math.sin(math.radians(Arctic_discharge['velocity_offset_angle_late'][i]))*25)

    #for the error, look at the min and max based on the x and y component uncertanties and work out the math from above
    Arctic_discharge.loc[i,'length_early_avg_err'] = np.abs((Arctic_discharge['offset_length_early_max'][i]-Arctic_discharge['offset_length_early_min'][i])/2)#use abs because of when velocity offset angle is >180, you get a negative number but will be the same magnitude
    Arctic_discharge.loc[i,'length_late_avg_err'] = np.abs((Arctic_discharge['offset_length_late_max'][i]-Arctic_discharge['offset_length_late_min'][i])/2)
    
    #if you have slow velocities, replace fluxgate with 25 m width
#     if (Arctic_discharge['2000_2010_mean'][i] > 25)&(45<Arctic_discharge['velocity_offset_angle_early'][i]<135):
#         Arctic_discharge.loc[i,'length_early_min'] = Arctic_discharge['claculated_fluxgate_width_early_min'][i]
#         Arctic_discharge.loc[i,'length_early_max'] = Arctic_discharge['claculated_fluxgate_width_early_max'][i]
#     if (Arctic_discharge['2000_2010_mean'][i] < 25)|(Arctic_discharge['velocity_offsett_angle_early'][i]<45)|(Arctic_discharge['velocity_offset_angle_early'][i]>135)|(Arctic_discharge['velocity_glacier_edge_mask'][i] == 1):#if the velocity is either slow, high angle, or near the edge, replace with max uncertainty
#         Arctic_discharge.loc[i,'length_early_min'] = np.abs(math.sin(math.radians(45))*25)#gate width at 45°
#         Arctic_discharge.loc[i,'length_early_max'] = 25        
        
#     if (Arctic_discharge['2010_2018_mean'][i] > 25)&(45<Arctic_discharge['velocity_offset_angle_late'][i]<135):
#         Arctic_discharge.loc[i,'length_late_min'] = Arctic_discharge['claculated_fluxgate_width_late_min'][i]
#         Arctic_discharge.loc[i,'length_late_max'] = Arctic_discharge['claculated_fluxgate_width_late_max'][i]
#     if (Arctic_discharge['2010_2018_mean'][i] < 25)|(Arctic_discharge['velocity_offset_angle_late'][i]<45)|(Arctic_discharge['velocity_offset_angle_late'][i]>135)|(Arctic_discharge['velocity_glacier_edge_mask'][i] == 1):#if the velocity is either slow, high angle, or near the edge, replace with max uncertainty
#         Arctic_discharge.loc[i,'length_late_min'] = np.abs(math.sin(math.radians(45))*25)#gate width at 45°
#         Arctic_discharge.loc[i,'length_late_max'] = 25


# Arctic_discharge_testing_export.to_file('Arctic_testing.gpkg', driver="GPKG")

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:52: RuntimeWarning: invalid value encountered in double_scalars
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:53: RuntimeWarning: invalid value encountered in double_scalars
/Users/willkochtit

In [15]:
#calculate the flux from discharge

# Arctic_discharge.loc[:,'length_early_avg'] = (Arctic_discharge['length_early_min']+Arctic_discharge['length_early_max'])/2
# Arctic_discharge.loc[:,'length_late_avg'] = (Arctic_discharge['length_late_min']+Arctic_discharge['length_late_max'])/2
# Arctic_discharge.loc[:,'length_early_avg_err'] = (Arctic_discharge['length_early_max']-Arctic_discharge['length_early_min'])/2
# Arctic_discharge.loc[:,'length_late_avg_err'] = (Arctic_discharge['length_late_max']-Arctic_discharge['length_late_min'])/2

Arctic_discharge['2000_2010_mean_flux_gt'] = np.nan
Arctic_discharge['2000_2010_mean_flux_err_gt'] = np.nan
Arctic_discharge['2010_2020_mean_flux_gt'] = np.nan
Arctic_discharge['2010_2020_mean_flux_err_gt'] = np.nan

#if thickness is less than 0, reset to 0
Arctic_discharge.loc[Arctic_discharge['best_thickness_early']<0,'best_thickness_early'] = 0
Arctic_discharge.loc[Arctic_discharge['best_thickness_late']<0,'best_thickness_late'] = 0

#If the glacier has a average thickness of less than 30 m, make the thickness 30±20 m
stored_rgi_early = []
stored_rgi_late = []
for i in range(0,len(Arctic_discharge)):
    if (Arctic_discharge['best_thickness'][Arctic_discharge['RGIId']==Arctic_discharge['RGIId'][i]].mean()<30)&(Arctic_discharge['best_thickness'][i]<30):
        Arctic_discharge.loc[i,'best_thickness'] = 30
        Arctic_discharge.loc[i,'thickness_uncertainty'] = 20
#         stored_rgi.append(Arctic_discharge['RGIId'][i])
    if (Arctic_discharge['best_thickness_early'][Arctic_discharge['RGIId']==Arctic_discharge['RGIId'][i]].mean()<30)&(Arctic_discharge['best_thickness_early'][i]<30):
        Arctic_discharge.loc[i,'best_thickness_early'] = 30
        Arctic_discharge.loc[i,'thickness_uncertainty'] = 20
        stored_rgi_early.append(Arctic_discharge['RGIId'][i])
    if (Arctic_discharge['best_thickness_late'][Arctic_discharge['RGIId']==Arctic_discharge['RGIId'][i]].mean()<30)&(Arctic_discharge['best_thickness_late'][i]<30):
        Arctic_discharge.loc[i,'best_thickness_late'] = 30
        Arctic_discharge.loc[i,'thickness_uncertainty'] = 20
        stored_rgi_late.append(Arctic_discharge['RGIId'][i])
#         print(Arctic_discharge['RGIId'][i],'----', Arctic_discharge['best_thickness'][Arctic_discharge['RGIId']==Arctic_discharge['RGIId'][i]].mean())
print('number of glaciers with thickness less than 30 m early:  ',len(set(stored_rgi_early)))
print('number of glaciers with thickness less than 30 m late:  ',len(set(stored_rgi_late)))
        #.                                                                             >depth average velocity adjustment                                                      
i=Arctic_discharge['best_thickness_early'] > 0
Arctic_discharge.loc[i,'2000_2010_mean_flux_gt'] = Arctic_discharge['2000_2010_mean']*.95*Arctic_discharge['length_early_avg']*Arctic_discharge['best_thickness_early']/1000000000*0.900 #0.9 converts to Gt
Arctic_discharge.loc[i,'2000_2010_mean_flux_err_gt'] = np.sqrt((Arctic_discharge['2000_2010_mean_err']/Arctic_discharge['2000_2010_mean'])**2
                                                  +(Arctic_discharge['length_early_avg_err']/Arctic_discharge['length_early_avg'])**2
                                                  +(Arctic_discharge['best_thickness_early_err']/Arctic_discharge['best_thickness_early'])**2)*Arctic_discharge['2000_2010_mean_flux_gt']*0.900 #0.9 converts to Gt
                                                                                                                                        #    ^multiply the percent error by total flux to get the error in km3 a-1   
i=Arctic_discharge['best_thickness_early']<=0
Arctic_discharge.loc[i,'2000_2010_mean_flux_gt'] = 0
Arctic_discharge.loc[i,'2000_2010_mean_flux_err_gt'] = 0

i=Arctic_discharge['best_thickness_late']>0
#.                                                                             >depth average velocity adjustment                                                      
Arctic_discharge.loc[i,'2010_2020_mean_flux_gt'] = Arctic_discharge['2010_2018_mean']*.95*Arctic_discharge['length_early_avg']*Arctic_discharge['best_thickness_late']/1000000000*0.900 #0.9 converts to Gt
Arctic_discharge.loc[i,'2010_2020_mean_flux_err_gt'] = np.sqrt((Arctic_discharge['2010_2018_mean_err']/Arctic_discharge['2010_2018_mean'])**2
                                                  +(Arctic_discharge['length_late_avg_err']/Arctic_discharge['length_late_avg'])**2
                                                  +(Arctic_discharge['best_thickness_late_err']/Arctic_discharge['best_thickness_late'])**2)*Arctic_discharge['2010_2020_mean_flux_gt']*0.900 #0.9 converts to Gt
                                                                                                                                    #    ^multiply the percent error by total flux to get the error in km3 a-1   
i=Arctic_discharge['best_thickness_late']<=0
Arctic_discharge.loc[i,'2010_2020_mean_flux_gt'] = 0
Arctic_discharge.loc[i,'2010_2020_mean_flux_err_gt'] = 0

Arctic_discharge.loc[:,'2000_2010_mean_flux'] = Arctic_discharge['2000_2010_mean_flux_gt'].astype(float)
Arctic_discharge.loc[:,'2000_2010_mean_flux_err'] = Arctic_discharge['2000_2010_mean_flux_err_gt'].astype(float)
Arctic_discharge.loc[:,'2010_2020_mean_flux'] = Arctic_discharge['2010_2020_mean_flux_gt'].astype(float)
Arctic_discharge.loc[:,'2010_2020_mean_flux_err'] = Arctic_discharge['2010_2020_mean_flux_err_gt'].astype(float)
# Arctic_discharge.loc[:,'flux_min'] = Arctic_discharge['flux_min'].astype(float)
# Arctic_discharge.loc[:,'flux_max'] = Arctic_discharge['flux_max'].astype(float)
Arctic_discharge.loc[:,'thickness_uncertainty'] = Arctic_discharge['thickness_uncertainty'].astype(float)

number of glaciers with thickness less than 30 m early:   144
number of glaciers with thickness less than 30 m late:   230


In [36]:
print(Arctic_discharge['2000_2010_mean_flux_gt'].sum())
print(Arctic_discharge['2000_2010_mean_flux_err_gt'].sum())

print(Arctic_discharge['2010_2020_mean_flux_gt'].sum())
print(Arctic_discharge['2010_2020_mean_flux_err_gt'].sum())

48.908630214168134
16.052210771686934
45.40416907145162
11.165679935103581


In [17]:
print(Arctic_discharge['2000_2010_mean_flux_gt'].sum())
print(Arctic_discharge['2000_2010_mean_flux_err_gt'].sum())

print(Arctic_discharge['2010_2020_mean_flux_gt'].sum())
print(Arctic_discharge['2010_2020_mean_flux_err_gt'].sum())

60.41816898369989
24.25016689805003
59.36573948799521
15.634682807112375


In [17]:
print(Arctic_discharge['2000_2010_mean_flux_gt'].sum())
print(Arctic_discharge['2000_2010_mean_flux_err_gt'].sum())

print(Arctic_discharge['2010_2020_mean_flux_gt'].sum())
print(Arctic_discharge['2010_2020_mean_flux_err_gt'].sum())

48.908630214168134
16.052210771686934
45.40416907145162
11.165679935103581


In [16]:
#values from Aug 5 2022
print(Arctic_discharge['2000_2010_mean_flux_gt'].sum())
print(Arctic_discharge['2000_2010_mean_flux_err_gt'].sum())

print(Arctic_discharge['2010_2020_mean_flux_gt'].sum())
print(Arctic_discharge['2010_2020_mean_flux_err_gt'].sum())

54.806188605734846
16.052210771686934
55.40308624610424
14.861357668085548


In [ ]:
# write out file for observations for every point
Arctic_discharge.to_file('Arctic_discharge_points_epsg3995_'+date_string+'.gpkg', driver="GPKG")#change file name based on distance

In [19]:
Arctic_discharge = gpd.read_file('Arctic_discharge_points_epsg3995_'+date_string+'.gpkg', driver="GPKG")

In [38]:
#Get regions right for next sections
arctic_aois = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/vector_files/Arctic_AOIs.gpkg')
Arctic_discharge = gpd.sjoin(Arctic_discharge,arctic_aois)
Arctic_discharge['region']=Arctic_discharge['Region_name_right']
Arctic_discharge = Arctic_discharge.drop(columns=['index_right','Region_name_right'])

In [39]:
#ERROR ANALYSIS from ROMAIN
# GeoPandas load 10GB+ in memory through fiona, for some unknown reason, converting to CSV and reading with pandas

# To convert the shapefile in command-line
# ogr2ogr -f CSV Will_frontal_ablation.csv Arctic_discharge_points_epsg3995_Spring2021_v9.gpkg
# Arctic_discharge = gpd.read_file('/home/atom/ongoing/work_frontal_ablation/Arctic_discharge_points_epsg3995_Fall2021_v10.gpkg', driver="GPKG")

# df = pd.read_csv('/home/atom/ongoing/work_frontal_ablation/Will_frontal_ablation.csv')

# Choose "early" or "late"
for period in ['early','late']:
    if period == 'early':
        period_date = '2000_2010'
    else:
        period_date = '2010_2018'

    print('Running for period: '+period)

    # 0/ Perform the bias correction for ice thickness

    print('Correcting ice thickness bias per ice thickness range...')

    best_meas_icethickness = Arctic_discharge['best_measured_ice_thickness_'+period].values
    best_modelled_icethickness = Arctic_discharge['Milan_ice_thickness_'+period].values
    it_diff = best_meas_icethickness - best_modelled_icethickness

    print('Median bias before:' +str(np.nanmean(it_diff)))

    df_bin = xdem.spatialstats.nd_binning(it_diff, list_var=[best_modelled_icethickness], list_var_names=['ice_thickness'], list_var_bins=20)
    y = df_bin.nanmedian.values
    x = pd.IntervalIndex(df_bin.ice_thickness).mid.values
    p = np.polyfit(x, y, deg=1)

    # Plot (optional)
    # import matplotlib.pyplot as plt
    # x0 = np.linspace(-50, 850)
    # xdem.spatialstats.plot_1d_binning(df_bin, var_name='ice_thickness', statistic_name='nanmedian',
    #                                   label_var='Modelled ice thickness (m)', label_statistic='Bias with measured ice thickness (m)')
    # ax = plt.gca()
    # ax.plot(x0, np.polyval(p, x0), color='tab:gray', linestyle='dashed', label='Linear fit')
    # ax.legend()

    # Derive the debiased modelled ice thickness
    Arctic_discharge['Milan_ice_thickness_'+period+'_debiased'] = Arctic_discharge['Milan_ice_thickness_'+period] + np.polyval(p, Arctic_discharge['Milan_ice_thickness_'+period])

    print('Median bias after:' +str(np.nanmean(best_meas_icethickness - Arctic_discharge['Milan_ice_thickness_'+period+'_debiased'])))

    # Replace the Milan ice thickness by their debiased version
    Arctic_discharge.loc[Arctic_discharge['thickness_data_source'].values==5, 'best_thickness_'+period] = Arctic_discharge['Milan_ice_thickness_'+period+'_debiased'][Arctic_discharge['thickness_data_source'].values==5]
    
    print('average thickness error before',Arctic_discharge['best_thickness_'+period+'_err'].mean())
    print('average velocity error before',Arctic_discharge[period_date+'_mean_err'].mean())
    
    #make sure that velocity and thickness errors are not bigger than the signal
    Arctic_discharge.loc[Arctic_discharge['best_thickness_'+period+'_err']>Arctic_discharge['best_thickness_'+period], 'best_thickness_'+period+'_err'] = Arctic_discharge['best_thickness_'+period]
    Arctic_discharge.loc[Arctic_discharge[period_date+'_mean']<Arctic_discharge[period_date+'_mean_err'], period_date+'_mean_err'] = Arctic_discharge[period_date+'_mean']
    
    print('average thickness error after',Arctic_discharge['best_thickness_'+period+'_err'].mean())
    print('average velocity error after',Arctic_discharge[period_date+'_mean_err'].mean())
    
    # 1/ Propagation flux gate -> glacier

    # Get variables of interest: velocities, ice thickness, flux gate segment widths
    Hn = Arctic_discharge['best_thickness_'+period].values
    Hn_err = Arctic_discharge['best_thickness_'+period+'_err'].values
    vn = Arctic_discharge[period_date+'_mean'].values * 0.95 # vertically averaged velocity
    vn_err = Arctic_discharge[period_date+'_mean_err'].values * 0.95 # same for the uncertainty
    d= Arctic_discharge['length_'+period+'_avg'].values
    d_err = Arctic_discharge['length_'+period+'_avg_err'].values

    # Define the NMAD as a robust measure of dispersion
    def nmad(data: np.ndarray, nfact: float = 1.4826) -> float:
        """
        Calculate the normalized median absolute deviation (NMAD) of an array.

        :param data: input data
        :param nfact: normalization factor for the data; default is 1.4826

        :returns nmad: (normalized) median absolute deviation of data.
        """
        data_arr = np.asarray(data)
        return nfact * np.nanmedian(np.abs(data_arr - np.nanmedian(data_arr)))

    # Remove aberrantly large uncertainty values for velocity errors
    vn_err[np.abs(vn_err - np.nanmedian(vn_err)) > 5 * nmad(vn_err)] = np.nan
    # Replace no data that might exist in the uncertainty from the average uncertainty
    vn_err[~np.isfinite(vn_err)] = np.nanmean(vn_err)

    # Correct velocity error given the spread estimated from the yearly comparison to Sentinel-1 and Greenland velocities
    # (theoretical error: 6 m a-1; empirical: 16 m a-1)
#     vn_err *= 16./6.

    # Remove aberrantly large uncertainty values for ice thickness errors
    Hn_err[np.abs(Hn_err - np.nanmedian(Hn_err)) > 5 * nmad(Hn_err)] = np.nan
    # Replace no data that might exist in the uncertainty from the average uncertainty
    Hn_err[~np.isfinite(Hn_err)] = np.nanmean(Hn_err)
    # After bias correction of the Milan model, ice thickness error have the same spread as yielded by the comparison
    # to GlaThiDa and other measurements
    # (theoretical error: 58 m a-1; empirical: 60 m a-1)
    Hn_err *= 1.

    # Calculate ice volume discharge
    Vn = Hn * vn * d * 900 / 10**12

    Vn_err = Hn * vn * d * np.sqrt(Hn_err**2/Hn**2 + vn_err**2/vn**2 + d_err**2/d**2) * 900 / 10**12

    # Remove aberrantly large uncertainty values for volume discharge uncertainty
    Vn_err[np.abs(Vn_err - np.nanmedian(Vn_err)) > 5 * nmad(Vn_err)] = np.nan
    # Replace no data that might exist in the uncertainty from the average uncertainty
    Vn_err[~np.isfinite(Vn_err)] = np.nanmean(Vn_err)

    # (ALL GOOD) Check that calculations match Will's for ice discharge
    # Product of the three variables: volume discharge at the flux gate
    # Vn_check = df['2000_2010_mean_flux'].values
    # Vn_err_check = df['2000_2010_mean_flux_err'].values
    # ratio_Vn = Vn_check/Vn
    # ratio_Vn_err = Vn_err_check/Vn_err

    # Parameters estimated for ghe variograms (nb_model x (correlation range, partial sill)) for velocity and ice thickness:
    # ranges in meters, partial sills are in percentage of the variance
    params_velocity =((691 ,63), (18000, 26),
                      (1040000, 11))
    params_icethickness = ((2100, 28), (47000, 50),
                           (1354000, 22))

    def sum_vgm(h, params, error1, error2):
        """Sum of variograms models"""
        v = 0
        for k in range(len(params)):
            v += skgstat.models.spherical(h, params[k][0], params[k][1] * error1 * error2)

        return v

    def product_vgms(h, mean_vn_err, mean_Hn_err, mean_d, mean_vn, mean_Hn):
        """Quadratic product of independent velocity and ice thickness variograms"""

        v = (sum_vgm(h, params_icethickness, mean_Hn_err, mean_Hn_err) * mean_d ** 2 * mean_vn ** 2 + \
             sum_vgm(h, params_velocity, mean_vn_err, mean_vn_err) * mean_d ** 2 * mean_Hn ** 2) /\
            ((mean_Hn_err**2 * mean_vn**2 * mean_d**2 + mean_vn_err**2 * mean_Hn**2 * mean_d**2) * 100)
        return v

    # Function to integrate the variogram from a spatial support to another: a double sum of covariances
    def double_sum_covar(coords: np.ndarray, areas: np.ndarray, errors: np.ndarray,
                         vgm_func: Callable):
        """
        Double sum of covariance for euclidean coordinates
        :param coords: Spatial support (typically, pixel) coordinates
        :param areas:  Area of supports
        :param errors: Standard errors of supports
        :param vgm_func: Variogram function
        :return:
        """

        n = len(coords)
        pds = pdist(coords)
        var = 0
        for i in range(n):
            for j in range(n):

                # For index calculation of the pairwise distance, see https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.pdist.html
                if i == j:
                    d = 0
                elif i < j:
                    ind = n * i + j - ((i + 2) * (i + 1)) // 2
                    d = pds[ind]
                else:
                    ind = n * j + i - ((j + 2) * (j + 1)) // 2
                    d = pds[ind]

                var += errors[i] * errors[j] * (1 - vgm_func(d)) * areas[i] * areas[j]

        total_area = sum(areas)
        se_dsc = np.sqrt(var / total_area ** 2)

        return se_dsc

    # Looping for every glacier
    unique_rgiid = list(np.unique(Arctic_discharge['RGIId'].values))

    list_df = []
    print('working on individual glaciers')
    for rgiid in unique_rgiid:

        # Select only flux gates of a given RGIId = glacier
        index_glacier = (Arctic_discharge['RGIId'] == rgiid)

#         print('Working on glacier: '+rgiid + ' with '+ str(np.count_nonzero(index_glacier))+ ' flux gates...')

        # Variogram function: derive an average correlation of volume discharge using the mean ice thickness and velocity of
        # the given glacier
        mean_vn = np.nanmean(vn[index_glacier])
        mean_Hn = np.nanmean(Hn[index_glacier])
        mean_d = np.nanmean(d[index_glacier])

        mean_vn_err = np.nanmean(vn_err[index_glacier])
        mean_Hn_err = np.nanmean(Hn_err[index_glacier])
        mean_d_err = np.nanmean(d_err[index_glacier])

        Vn_err_gla = Vn_err[index_glacier]
        Vn_gla = Vn[index_glacier]

        def vgm_func_gla(h):
            v = product_vgms(h, mean_vn_err, mean_Hn_err, mean_d, mean_vn, mean_Hn)
            return v

        # Sum of volume discharges
        V_tot = np.nansum(Vn_gla)

        # Uncertainty on the sum, considering everything independent
        V_tot_err_uncorr = np.sqrt(np.nansum(Vn_err_gla**2))

        # Uncertainty on the sum, considering everything correlated
        V_tot_err_fullycorr = np.nansum(Vn_err_gla)

        # Uncertainty on the sum, accounting for spatial correlations in velocities and ice thickness
        coords = np.array((Arctic_discharge['easting'].values[index_glacier], Arctic_discharge['northing'].values[index_glacier])).T
        V_tot_err_corr = double_sum_covar(coords=coords, areas=np.ones(len(coords)), errors=Vn_err_gla,
                                     vgm_func=vgm_func_gla) * len(coords)

        # Store results in a dataframe
        df_res = pd.DataFrame()
        df_res = df_res.assign(region = [Arctic_discharge['region'].values[index_glacier][0]] ,rgiid = [rgiid], V_tot = [V_tot],
                               V_tot_err_uncorr = [V_tot_err_uncorr], V_tot_err_corr = [V_tot_err_corr],
                               V_tot_err_fullycorr = [V_tot_err_fullycorr],
                               mean_easting = [np.nanmean(Arctic_discharge['easting'].values[index_glacier])],
                               mean_northing = [np.nanmean(Arctic_discharge['northing'].values[index_glacier])],
                               nb_flux_gates = [np.count_nonzero(index_glacier)],
                               mean_vn = [mean_vn], mean_Hn = [mean_Hn], mean_d = [mean_d],
                               mean_vn_err = [mean_vn_err], mean_Hn_err = [mean_Hn_err], mean_d_err = [mean_d_err])
        list_df.append(df_res)

    # Concatenate all results
    if period=='early':
        df_all_early = pd.concat(list_df)
        list_regions = list(np.unique(df_all_early.region.values))
        df_all_early.to_pickle('df_all_early_'+date_string+'.pkl')
    if period=='late':
        df_all_late = pd.concat(list_df)
        list_regions = list(np.unique(df_all_late.region.values))
        df_all_late.to_pickle('df_all_late_'+date_string+'.pkl')
        
    # df_all.to_csv('/home/atom/ongoing/work_frontal_ablation/df_perglacier_final_'+period+'.csv')

    # 2/ Propagation glacier -> region

    

    list_df_reg = []
    for region in list_regions:

        print('Working on region: '+region)

        # Isolate region of interest
        if period=='early':
            df_reg = df_all_early[df_all_early.region == region]
        if period=='late':
            df_reg = df_all_late[df_all_late.region == region]
        # Sum volume discharges
        V_tot_reg = np.sum(df_reg.V_tot.values)

        # Uncertainty on the sum, considering everything independent
        V_tot_err_uncorr_reg = np.sqrt(np.nansum(df_reg.V_tot_err_uncorr.values ** 2))

        # Uncertainty on the sum, considering everything correlated
        V_tot_err_fullycorr_reg = np.nansum(df_reg.V_tot_err_fullycorr)

        # Get the regional weighted mean of velocities, thicknesses, flux gate width, and related uncertainties
        mean_vn_reg = np.nansum(df_reg.mean_vn.values * df_reg.nb_flux_gates.values)/np.nansum(df_reg.nb_flux_gates.values)
        mean_Hn_reg = np.nansum(df_reg.mean_Hn.values * df_reg.nb_flux_gates.values)/np.nansum(df_reg.nb_flux_gates.values)
        mean_d_reg = np.nansum(df_reg.mean_d.values * df_reg.nb_flux_gates.values)/np.nansum(df_reg.nb_flux_gates.values)
        mean_vn_err_reg = np.nansum(df_reg.mean_vn_err.values * df_reg.nb_flux_gates.values)/np.nansum(df_reg.nb_flux_gates.values)
        mean_Hn_err_reg = np.nansum(df_reg.mean_Hn_err.values * df_reg.nb_flux_gates.values)/np.nansum(df_reg.nb_flux_gates.values)
        mean_d_err_reg = np.nansum(df_reg.mean_d_err.values * df_reg.nb_flux_gates.values)/np.nansum(df_reg.nb_flux_gates.values)

        # Define the average ice discharge correlation function in the region accordingly
        def vgm_func_gla(h):
            v = product_vgms(h, mean_vn_err_reg, mean_Hn_err_reg, mean_d_reg, mean_vn_reg, mean_Hn_reg)
            return v

        # Uncertainty on the sum, accounting for spatial correlations in velocities and ice thickness
        coords = np.array((df_reg['mean_easting'].values, df_reg['mean_northing'].values)).T
        V_tot_err_corr_reg = double_sum_covar(coords=coords, areas=np.ones(len(coords)), errors=df_reg.V_tot_err_corr.values,
                                          vgm_func=vgm_func_gla) * len(coords)

        # Store results in a dataframe
        df_res_reg = pd.DataFrame()
        df_res_reg = df_res_reg.assign(region=[region], V_tot=[V_tot_reg],
                               V_tot_err_uncorr=[V_tot_err_uncorr_reg], V_tot_err_corr=[V_tot_err_corr_reg],
                               V_tot_err_fullycorr=[V_tot_err_fullycorr_reg],
                               nb_flux_gates=[np.sum(df_reg.nb_flux_gates.values)],
                               mean_vn=[mean_vn_reg], mean_Hn=[mean_Hn_reg], mean_d=[mean_d_reg],
                               mean_vn_err=[mean_vn_err_reg], mean_Hn_err=[mean_Hn_err_reg], mean_d_err=[mean_d_err_reg])
        list_df_reg.append(df_res_reg)

    df_all_reg = pd.concat(list_df_reg)
    if period=='early':
        df_all_reg.to_pickle('df_all_reg_early_'+date_string+'.pkl')
    if period=='late':
        df_all_reg.to_pickle('df_all_reg_late_'+date_string+'.pkl')

    # 3/ Propagation region -> global

    print('Working on global total')

    # Sum volume discharges
    V_tot_global = np.sum(df_all_reg.V_tot.values)

    # Uncertainty on the sum, considering everything independent
    V_tot_err_uncorr_global = np.sqrt(np.nansum(df_all_reg.V_tot_err_uncorr.values ** 2))

    # Uncertainty on the sum, accounting for spatial correlations in velocities and ice thickness
    # For the global step, no correlations after >1000 km, so we can simply sum quadratically
    V_tot_err_corr_global = np.sqrt(np.nansum(df_all_reg.V_tot_err_corr.values ** 2))

    # Uncertainty on the sum, considering everything correlated
    V_tot_err_fullycorr_global = np.nansum(df_all_reg.V_tot_err_fullycorr)

    # Get the global weighted mean of velocities, thicknesses, flux gate width, and related uncertainties
    mean_vn_global = np.nansum(df_all_reg.mean_vn.values * df_all_reg.nb_flux_gates.values) / np.nansum(df_all_reg.nb_flux_gates.values)
    mean_Hn_global = np.nansum(df_all_reg.mean_Hn.values * df_all_reg.nb_flux_gates.values) / np.nansum(df_all_reg.nb_flux_gates.values)
    mean_d_global = np.nansum(df_all_reg.mean_d.values * df_all_reg.nb_flux_gates.values) / np.nansum(df_all_reg.nb_flux_gates.values)
    mean_vn_err_global = np.nansum(df_all_reg.mean_vn_err.values * df_all_reg.nb_flux_gates.values) / np.nansum(
        df_all_reg.nb_flux_gates.values)
    mean_Hn_err_global = np.nansum(df_all_reg.mean_Hn.values * df_all_reg.nb_flux_gates.values) / np.nansum(
        df_all_reg.nb_flux_gates.values)
    mean_d_err_global = np.nansum(df_all_reg.mean_d_err.values * df_all_reg.nb_flux_gates.values) / np.nansum(
        df_all_reg.nb_flux_gates.values)

    df_global = pd.DataFrame()
    df_global = df_global.assign(region=['Global total'], V_tot=[V_tot_global],
                               V_tot_err_uncorr=[V_tot_err_uncorr_global], V_tot_err_corr=[V_tot_err_corr_global],
                               V_tot_err_fullycorr=[V_tot_err_fullycorr_global],
                               nb_flux_gates=[np.sum(df_all_reg.nb_flux_gates.values)],
                               mean_vn=[mean_vn_global], mean_Hn=[mean_Hn_global], mean_d=[mean_d_global],
                               mean_vn_err=[mean_vn_err_global], mean_Hn_err=[mean_Hn_err_global], mean_d_err=[mean_d_err_global])

    if period=='early':
        df_reg_global_early = pd.concat([df_all_reg, df_global])
    if period=='late':
        df_reg_global_late = pd.concat([df_all_reg, df_global])
    #df_reg_global.to_csv('/home/atom/ongoing/work_frontal_ablation/df_region_global_final_'+period+'.csv')

Running for period: early
Correcting ice thickness bias per ice thickness range...
Median bias before:-134.54867012593377
Median bias after:1.6128630292371475
average thickness error before 54.02309663951476
average velocity error before 39.2258815788063
average thickness error after 53.154607678543066
average velocity error after 39.2258815788063


/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:105: RuntimeWarning: divide by zero encountered in true_divide
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:105: RuntimeWarning: invalid value encountered in true_divide
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:105: RuntimeWarning: invalid value encountered in multiply


working on individual glaciers


/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:193: RuntimeWarning: Mean of empty slice
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:197: RuntimeWarning: Mean of empty slice


Working on region: Alaska
Working on region: BBI
Working on region: FJL
Working on region: Greenland
Working on region: Iceland
Working on region: NZ
Working on region: QEI
Working on region: SZ
Working on region: Svalbard and Jan Mayen
Working on global total
Running for period: late
Correcting ice thickness bias per ice thickness range...
Median bias before:-134.49439949611929
Median bias after:1.5970979079254413
average thickness error before 53.501451458919
average velocity error before 16.296816186024632
average thickness error after 52.17088017715039
average velocity error after 16.29652475486997


/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:105: RuntimeWarning: divide by zero encountered in true_divide
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:105: RuntimeWarning: invalid value encountered in true_divide
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:105: RuntimeWarning: invalid value encountered in multiply


working on individual glaciers


/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:193: RuntimeWarning: Mean of empty slice
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:197: RuntimeWarning: Mean of empty slice


Working on region: Alaska
Working on region: BBI
Working on region: FJL
Working on region: Greenland
Working on region: Iceland
Working on region: NZ
Working on region: QEI
Working on region: SZ
Working on region: Svalbard and Jan Mayen
Working on global total


In [40]:
#make it a pandas data frame so we can groupby
Arctic_discharge_df=pd.DataFrame(Arctic_discharge)

#calculate stats
Arctic_discharge_df_sum = Arctic_discharge_df.groupby(['RGIId']).sum().reset_index()
Arctic_discharge_df_min = Arctic_discharge_df.groupby(['RGIId']).min().reset_index()
Arctic_discharge_df_max = Arctic_discharge_df.groupby(['RGIId']).max().reset_index()
Arctic_discharge_df_mean = Arctic_discharge_df.groupby(['RGIId']).mean().reset_index()
Arctic_discharge_df_count = Arctic_discharge_df.groupby(['RGIId']).count().reset_index()

# Clean the flux data
Arctic_discharge_fluxes_sum = Arctic_discharge_df_sum[['RGIId','2000_2010_mean_flux_gt','2000_2010_mean_flux_err_gt', '2010_2020_mean_flux_gt', '2010_2020_mean_flux_err_gt']]
Arctic_discharge_fluxes_min = Arctic_discharge_df_min[['RGIId','Name','Area','Zmin', 'Zmax', 'Zmed', 'Slope', 'Aspect', 'Lmax', 'Status',
       'Connect', 'Form', 'TermType', 'Surging', 'Linkages','Milan_thickness','region']]
Arctic_discharge_fluxes_max = Arctic_discharge_df_max[['RGIId','Milan_thickness','do_we_have_an_observation_in_middle_fifth']]
Arctic_discharge_fluxes_mean = Arctic_discharge_df_mean[['RGIId','easting','northing','Milan_thickness','0000_v','best_thickness','thickness_uncertainty',
                                                         'length_early_avg','length_early_avg_err','length_late_avg','length_late_avg_err','2000_2010_mean','2000_2010_mean_err',
                                                         '2000_2010_mean_count','2010_2018_mean','2010_2018_mean_err','2010_2018_mean_count',
                                                        'best_thickness_early','best_thickness_early_err','best_thickness_late','best_thickness_late_err',
                                                        'number_velocity_obs_early','number_velocity_obs_late']]
Arctic_discharge_fluxes_count = Arctic_discharge_df_count[['RGIId','CenLon','THICKNESS','tomo_thickness','Russian_thickness']]
Arctic_discharge_fluxes_count = Arctic_discharge_fluxes_count.rename(columns={"CenLon": "num_pts"})
data_frames_for_merge = [Arctic_discharge_fluxes_sum, Arctic_discharge_fluxes_min, Arctic_discharge_fluxes_max, Arctic_discharge_fluxes_mean, Arctic_discharge_fluxes_count]



Arctic_discharge_fluxes = reduce(lambda  left,right: pd.merge(left,right,on=['RGIId'],
                                            how='outer'), data_frames_for_merge)
Arctic_discharge_fluxes = Arctic_discharge_fluxes.rename(columns={"Milan_thickness_x": "Milan_thickness_min",'Milan_thickness_y':'Milan_thickness_max','Milan_thickness':'Milan_thickness_mean','THICKNESS':'GlaThiDa_observations','tomo_thickness':'Tomography_observations','Russian_thickness':'Russian_observations','best_thickness':'best_thickness_mean','thickness_uncertainty':'thickness_mean_uncertainty'})  

Arctic_discharge_fluxes.loc[:,'gate_length_km']=Arctic_discharge_fluxes['num_pts']*((Arctic_discharge_fluxes['length_early_avg']+Arctic_discharge_fluxes['length_late_avg'])/2)/1000 #25 is distance between points you specificed above

Arctic_discharge_fluxes = gpd.GeoDataFrame(Arctic_discharge_fluxes, geometry=gpd.points_from_xy(Arctic_discharge_fluxes.easting, Arctic_discharge_fluxes.northing),crs='EPSG:3995')

#replace my calculations with Romain's
#make sure the RGIIds are in the same order
df_all_early=df_all_early.sort_values('rgiid').reset_index(drop=True)
df_all_late=df_all_late.sort_values('rgiid').reset_index(drop=True)
Arctic_discharge_fluxes=Arctic_discharge_fluxes.sort_values('RGIId')
#do it for early
Arctic_discharge_fluxes.loc[:,'2000_2010_mean_flux_gt']=df_all_early['V_tot']
Arctic_discharge_fluxes.loc[:,'2000_2010_mean_flux_err_gt']=df_all_early['V_tot_err_corr']
Arctic_discharge_fluxes.loc[:,'2000_2010_mean']=df_all_early['mean_vn']
Arctic_discharge_fluxes.loc[:,'2000_2010_mean_err']=df_all_early['mean_vn_err']
Arctic_discharge_fluxes.loc[:,'best_thickness_early']=df_all_early['mean_Hn']
Arctic_discharge_fluxes.loc[:,'best_thickness_early_err']=df_all_early['mean_Hn_err']
Arctic_discharge_fluxes.loc[:,'length_early_avg']=df_all_early['mean_d']
Arctic_discharge_fluxes.loc[:,'length_early_avg_err']=df_all_early['mean_d_err']
#do it for late
Arctic_discharge_fluxes.loc[:,'2010_2020_mean_flux_gt']=df_all_late['V_tot']
Arctic_discharge_fluxes.loc[:,'2010_2020_mean_flux_err_gt']=df_all_late['V_tot_err_corr']
Arctic_discharge_fluxes.loc[:,'2010_2018_mean']=df_all_late['mean_vn']
Arctic_discharge_fluxes.loc[:,'2010_2018_mean_err']=df_all_late['mean_vn_err']
Arctic_discharge_fluxes.loc[:,'best_thickness_late']=df_all_late['mean_Hn']
Arctic_discharge_fluxes.loc[:,'best_thickness_late_err']=df_all_late['mean_Hn_err']
Arctic_discharge_fluxes.loc[:,'length_late_avg']=df_all_late['mean_d']
Arctic_discharge_fluxes.loc[:,'length_late_avg_err']=df_all_late['mean_d_err']

Arctic_discharge_fluxes.to_file('Arctic_discharge_fluxes'+date_string+'.gpkg', driver="GPKG")

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Dropping invalid columns in DataFrameGroupBy.min is deprecated. In a future version, a TypeError will be raised. Before calling .min, select only columns which should be valid for the function.
  
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  import sys


In [41]:
#calculate the area change of every glacier from 2000 to 2020
#read in data files
termini_2000 = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/termini_2000.geojson')
termini_2010 = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/termini_2010.geojson')
termini_2020 = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/termini_2020.geojson')

termini_2000=termini_2000[termini_2000.geometry.type=='Polygon']
termini_2010=termini_2010[termini_2010.geometry.type=='Polygon']
termini_2020=termini_2020[termini_2020.geometry.type=='Polygon']

termini_2000.loc[:,'geometry'] = termini_2000['geometry'].buffer(0)
termini_2010.loc[:,'geometry'] = termini_2010['geometry'].buffer(0)
termini_2020.loc[:,'geometry'] = termini_2020['geometry'].buffer(0)

termini_2000.loc[:,'time_period']='2000'
termini_2010.loc[:,'time_period']='2010'
termini_2020.loc[:,'time_period']='2020'

termini=pd.concat([termini_2000,termini_2010,termini_2020])

boxes = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/Glacier_area_change_boxes.gpkg')
RGI = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/RGI_Arctic_marine_terminating_glacier_outlines.gpkg')
                     
RGI['geometry'] = RGI.geometry.buffer(0)
RGI['RGIId']=RGI['RGIid_2']

#Add RGI and satellite image date attributes to the boxes
boxes_attributed = gpd.sjoin(boxes, RGI, how='inner')
boxes_attributed = boxes_attributed.drop(columns=['index_right'])
boxes_attributed = gpd.sjoin(boxes_attributed, termini_2000, how='left')
boxes_attributed = boxes_attributed.drop(columns=['Date_text','time_period','index_right','index','layer'])
boxes_attributed = boxes_attributed.rename(columns={'Landsat_ID':'Early_satellite_id'})
boxes_attributed = gpd.sjoin(boxes_attributed, termini_2010, how='left')
boxes_attributed = boxes_attributed.drop(columns=['Date_text','time_period','index_right','index'])
boxes_attributed = boxes_attributed.rename(columns={'Landsat_ID':'Middle_satellite_id'})
boxes_attributed = gpd.sjoin(boxes_attributed, termini_2020, how='left')
boxes_attributed = boxes_attributed.drop(columns=['Date_text','time_period','index_right','index','path'])
boxes_attributed = boxes_attributed.rename(columns={'Landsat_ID':'Late_satellite_id'})
boxes_attributed.loc[:,'Early_satellite_id'] = boxes_attributed['Early_satellite_id'].astype(str)
boxes_attributed.loc[:,'Middle_satellite_id'] = boxes_attributed['Middle_satellite_id'].astype(str)
boxes_attributed.loc[:,'Late_satellite_id'] = boxes_attributed['Late_satellite_id'].astype(str)
boxes_attributed = boxes_attributed.reset_index()

#get dates for the satellites images for the box attributes
#2000

boxes_attributed['Early_date']=np.nan
for i in range(0,len(boxes_attributed)):
    if boxes_attributed['Early_satellite_id'][i][0]=='L':
        boxes_attributed.loc[i,'Early_date']=pd.to_datetime(boxes_attributed['Early_satellite_id'][i][17:25],format='%Y%m%d')
    if boxes_attributed['Early_satellite_id'][i][0:4]=='AST_':
        boxes_attributed.loc[i,'Early_date']=pd.to_datetime(boxes_attributed['Early_satellite_id'][i][11:19],format='%m%d%Y')
    if boxes_attributed['Early_satellite_id'][i][0:4]=='AST1':
        boxes_attributed.loc[i,'Early_date']=pd.to_datetime(boxes_attributed['Early_satellite_id'][i][12:20],format='%m%d%Y')
    if boxes_attributed['Early_satellite_id'][i][0:1]=='R':
        boxes_attributed.loc[i,'Early_date']=pd.to_datetime(boxes_attributed['Early_satellite_id'][i][18:27],format='%d%m%Y')

#2010
boxes_attributed['Mid_date']=np.nan
for i in range(0,len(boxes_attributed)):
    if boxes_attributed['Middle_satellite_id'][i][0]=='L':
        boxes_attributed.loc[i,'Mid_date']=pd.to_datetime(boxes_attributed['Middle_satellite_id'][i][17:25],format='%Y%m%d')
    if boxes_attributed['Middle_satellite_id'][i][0:4]=='AST_':
        boxes_attributed.loc[i,'Mid_date']=pd.to_datetime(boxes_attributed['Middle_satellite_id'][i][11:19],format='%m%d%Y')
    if boxes_attributed['Middle_satellite_id'][i][0:4]=='AST1':
        boxes_attributed.loc[i,'Mid_date']=pd.to_datetime(boxes_attributed['Middle_satellite_id'][i][12:20],format='%m%d%Y')
    if boxes_attributed['Middle_satellite_id'][i][0:3]=='IMG':
        boxes_attributed.loc[i,'Mid_date']=pd.to_datetime(boxes_attributed['Middle_satellite_id'][i][26:34],format='%d%m%Y')

#2020
boxes_attributed['Late_date']=np.nan
for i in range(0,len(boxes_attributed)):
    if boxes_attributed['Late_satellite_id'][i][0]=='L':
        boxes_attributed.loc[i,'Late_date']=pd.to_datetime(boxes_attributed['Late_satellite_id'][i][17:25],format='%Y%m%d')
    if boxes_attributed['Late_satellite_id'][i][0:4]=='AST_':
        boxes_attributed.loc[i,'Late_date']=pd.to_datetime(boxes_attributed['Late_satellite_id'][i][11:19],format='%m%d%Y')
    if boxes_attributed['Late_satellite_id'][i][0:4]=='AST1':
        boxes_attributed.loc[i,'Late_date']=pd.to_datetime(boxes_attributed['Late_satellite_id'][i][12:20],format='%m%d%Y')

#get only the glacier terminus area within the box
termini_intersect = gpd.overlay(termini,boxes_attributed,how='intersection')#,keep_geom_type=True)
# termini_intersect['area'] = termini_intersect['geometry'].area/1000000

#get the date from the satellite ID - difference for Landsat and ASTER
termini_intersect['Date']=np.nan
for i in range(0,len(termini_intersect)):
    if termini_intersect['Landsat_ID'][i][0]=='L':
        termini_intersect.loc[i,'Date']=pd.to_datetime(termini_intersect['Landsat_ID'][i][17:25],format='%Y%m%d')
    if termini_intersect['Landsat_ID'][i][0:4]=='AST_':
        termini_intersect.loc[i,'Date']=pd.to_datetime(termini_intersect['Landsat_ID'][i][11:19],format='%m%d%Y')
    if termini_intersect['Landsat_ID'][i][0:4]=='AST1':
        termini_intersect.loc[i,'Date']=pd.to_datetime(termini_intersect['Landsat_ID'][i][12:20],format='%m%d%Y')

#sort by terminus date for differencing in next step
early_termini_intersect=termini_intersect[termini_intersect['time_period']=='2000']
middle_termini_intersect=termini_intersect[termini_intersect['time_period']=='2010']
late_termini_intersect=termini_intersect[termini_intersect['time_period']=='2020']

            #2000 to 2020
#calculate total area lost from 2000 to 2020
#different glaciers to get how much area is lost and gained over the time period
marine_based_glacier_area_lost_total = gpd.overlay(early_termini_intersect,late_termini_intersect,how='difference')
marine_based_glacier_area_lost_total=marine_based_glacier_area_lost_total[['RGIId','Name','geometry']].dissolve(by='RGIId')
marine_based_glacier_area_lost_total = marine_based_glacier_area_lost_total.reset_index()
marine_based_glacier_area_lost_total['geometry_3995']=marine_based_glacier_area_lost_total['geometry']
marine_based_glacier_area_lost_total.loc[:,'latlon'] = marine_based_glacier_area_lost_total.centroid.to_crs(crs='EPSG:4326')
for i in range (0,len(marine_based_glacier_area_lost_total)):
    projection = ccrs.Orthographic(central_longitude=marine_based_glacier_area_lost_total.loc[i]['latlon'].x, central_latitude=marine_based_glacier_area_lost_total.loc[i]['latlon'].y)
    marine_based_glacier_area_lost_total = marine_based_glacier_area_lost_total.to_crs(projection)
    marine_based_glacier_area_lost_total.loc[[i],'geometry']=marine_based_glacier_area_lost_total.loc[[i],'geometry'].buffer(-0.000001)
    marine_based_glacier_area_lost_total.loc[i,'area_km2'] = marine_based_glacier_area_lost_total.loc[i,'geometry'].area/1000000
    marine_based_glacier_area_lost_total.loc[i,'perimeter'] = marine_based_glacier_area_lost_total.loc[i,'geometry'].length
    marine_based_glacier_area_lost_total.loc[i,'perimeter_error'] = marine_based_glacier_area_lost_total.loc[i,'perimeter']/30*30*30/1000000 #/pixels/meter *area/pixel /m2/km2
marine_based_glacier_area_lost_total['perimeter_error'] = marine_based_glacier_area_lost_total['perimeter_error'].fillna(0)
marine_based_glacier_area_lost_total=marine_based_glacier_area_lost_total[marine_based_glacier_area_lost_total['area_km2']>0.000225] #throw out tiny areas that are a mistakes

#calculate total area gained from 2000 to 2020
marine_based_glacier_area_gained_total = gpd.overlay(late_termini_intersect,early_termini_intersect,how='difference')
marine_based_glacier_area_gained_total=marine_based_glacier_area_gained_total[['RGIId','Name','geometry']].dissolve(by='RGIId')
marine_based_glacier_area_gained_total = marine_based_glacier_area_gained_total.reset_index()
marine_based_glacier_area_gained_total['geometry_3995']=marine_based_glacier_area_gained_total['geometry']
marine_based_glacier_area_gained_total.loc[:,'latlon'] = marine_based_glacier_area_gained_total.centroid.to_crs(crs='EPSG:4326')
for i in range (0,len(marine_based_glacier_area_gained_total)):
    projection = ccrs.Orthographic(central_longitude=marine_based_glacier_area_gained_total.loc[i]['latlon'].x, central_latitude=marine_based_glacier_area_gained_total.loc[i]['latlon'].y)
    marine_based_glacier_area_gained_total = marine_based_glacier_area_gained_total.to_crs(projection)
    marine_based_glacier_area_gained_total.loc[[i],'geometry']=marine_based_glacier_area_gained_total.loc[[i],'geometry'].buffer(-0.000001)
    marine_based_glacier_area_gained_total.loc[i,'area_km2'] = marine_based_glacier_area_gained_total.loc[i,'geometry'].area/1000000
    marine_based_glacier_area_gained_total.loc[i,'perimeter'] = marine_based_glacier_area_gained_total.loc[i,'geometry'].length
    marine_based_glacier_area_gained_total.loc[i,'perimeter_error'] = marine_based_glacier_area_gained_total.loc[i,'perimeter']/30*30*30/1000000 #/pixels/meter *area/pixel /m2/km2
marine_based_glacier_area_gained_total['perimeter_error'] = marine_based_glacier_area_gained_total['perimeter_error'].fillna(0)
marine_based_glacier_area_gained_total=marine_based_glacier_area_gained_total[marine_based_glacier_area_gained_total['area_km2']>0.000225] #throw out tiny areas that are a mistakes


marine_based_glacier_area_lost_total.loc[:,'geometry']=marine_based_glacier_area_lost_total['geometry_3995']
marine_based_glacier_area_gained_total.loc[:,'geometry']=marine_based_glacier_area_gained_total['geometry_3995']
marine_based_glacier_area_lost_total.drop(columns=['geometry_3995','latlon'],inplace=True)
marine_based_glacier_area_gained_total.drop(columns=['geometry_3995','latlon'],inplace=True)
#export data
marine_based_glacier_area_lost_total.to_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/marine_based_glacier_area_lost_total_'+date_string+'.geojson', driver='GeoJSON')
marine_based_glacier_area_gained_total.to_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/marine_based_glacier_area_gained_total_'+date_string+'.geojson', driver='GeoJSON')


            #2000 to 2010
#calculate total area lost from 2000 to 2020
#different glaciers to get how much area is lost and gained over the time period
marine_based_glacier_area_lost_early = gpd.overlay(early_termini_intersect,middle_termini_intersect,how='difference')
marine_based_glacier_area_lost_early=marine_based_glacier_area_lost_early[['RGIId','Name','geometry']].dissolve(by='RGIId')
marine_based_glacier_area_lost_early = marine_based_glacier_area_lost_early.reset_index()
marine_based_glacier_area_lost_early['geometry_3995']=marine_based_glacier_area_lost_early['geometry']
marine_based_glacier_area_lost_early.loc[:,'latlon'] = marine_based_glacier_area_lost_early.centroid.to_crs(crs='EPSG:4326')
for i in range (0,len(marine_based_glacier_area_lost_early)):
    projection = ccrs.Orthographic(central_longitude=marine_based_glacier_area_lost_early.loc[i]['latlon'].x, central_latitude=marine_based_glacier_area_lost_early.loc[i]['latlon'].y)
    marine_based_glacier_area_lost_early = marine_based_glacier_area_lost_early.to_crs(projection)
    marine_based_glacier_area_lost_early.loc[[i],'geometry']=marine_based_glacier_area_lost_early.loc[[i],'geometry'].buffer(-0.000001)
    marine_based_glacier_area_lost_early.loc[i,'area_km2'] = marine_based_glacier_area_lost_early.loc[i,'geometry'].area/1000000
    marine_based_glacier_area_lost_early.loc[i,'perimeter'] = marine_based_glacier_area_lost_early.loc[i,'geometry'].length
    marine_based_glacier_area_lost_early.loc[i,'perimeter_error'] = marine_based_glacier_area_lost_early.loc[i,'perimeter']/30*30*30/1000000 #/pixels/meter *area/pixel /m2/km2
marine_based_glacier_area_lost_early['perimeter_error'] = marine_based_glacier_area_lost_early['perimeter_error'].fillna(0)
marine_based_glacier_area_lost_early=marine_based_glacier_area_lost_early[marine_based_glacier_area_lost_early['area_km2']>0.000225] #throw out tiny areas that are a mistakes

#calculate total area gained from 2000 to 2010
marine_based_glacier_area_gained_early = gpd.overlay(middle_termini_intersect,early_termini_intersect,how='difference')
marine_based_glacier_area_gained_early=marine_based_glacier_area_gained_early[['RGIId','Name','geometry']].dissolve(by='RGIId')
marine_based_glacier_area_gained_early = marine_based_glacier_area_gained_early.reset_index()
marine_based_glacier_area_gained_early['geometry_3995']=marine_based_glacier_area_gained_early['geometry']
marine_based_glacier_area_gained_early.loc[:,'latlon'] = marine_based_glacier_area_gained_early.centroid.to_crs(crs='EPSG:4326')
for i in range (0,len(marine_based_glacier_area_gained_early)):
    projection = ccrs.Orthographic(central_longitude=marine_based_glacier_area_gained_early.loc[i]['latlon'].x, central_latitude=marine_based_glacier_area_gained_early.loc[i]['latlon'].y)
    marine_based_glacier_area_gained_early = marine_based_glacier_area_gained_early.to_crs(projection)
    marine_based_glacier_area_gained_early.loc[[i],'geometry']=marine_based_glacier_area_gained_early.loc[[i],'geometry'].buffer(-0.000001)
    marine_based_glacier_area_gained_early.loc[i,'area_km2'] = marine_based_glacier_area_gained_early.loc[i,'geometry'].area/1000000
    marine_based_glacier_area_gained_early.loc[i,'perimeter'] = marine_based_glacier_area_gained_early.loc[i,'geometry'].length
    marine_based_glacier_area_gained_early.loc[i,'perimeter_error'] = marine_based_glacier_area_gained_early.loc[i,'perimeter']/30*30*30/1000000 #/pixels/meter *area/pixel /m2/km2
marine_based_glacier_area_gained_early['perimeter_error'] = marine_based_glacier_area_gained_early['perimeter_error'].fillna(0)
marine_based_glacier_area_gained_early=marine_based_glacier_area_gained_early[marine_based_glacier_area_gained_early['area_km2']>0.000225] #throw out tiny areas that are a mistakes

#export data
marine_based_glacier_area_lost_early.loc[:,'geometry']=marine_based_glacier_area_lost_early['geometry_3995']
marine_based_glacier_area_gained_early.loc[:,'geometry']=marine_based_glacier_area_gained_early['geometry_3995']
marine_based_glacier_area_lost_early.drop(columns=['geometry_3995','latlon'],inplace=True)
marine_based_glacier_area_gained_early.drop(columns=['geometry_3995','latlon'],inplace=True)
#export data
marine_based_glacier_area_lost_early.to_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/marine_based_glacier_area_lost_early_'+date_string+'.geojson', driver='GeoJSON')
marine_based_glacier_area_gained_early.to_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/marine_based_glacier_area_gained_early_'+date_string+'.geojson', driver='GeoJSON')

            #2010 to 2020
#calculate total area lost from 2000 to 2020
#different glaciers to get how much area is lost and gained over the time period
marine_based_glacier_area_lost_late = gpd.overlay(middle_termini_intersect,late_termini_intersect,how='difference')
marine_based_glacier_area_lost_late=marine_based_glacier_area_lost_late[['RGIId','Name','geometry']].dissolve(by='RGIId')
marine_based_glacier_area_lost_late = marine_based_glacier_area_lost_late.reset_index()
marine_based_glacier_area_lost_late['geometry_3995']=marine_based_glacier_area_lost_late['geometry']
marine_based_glacier_area_lost_late.loc[:,'latlon'] = marine_based_glacier_area_lost_late.centroid.to_crs(crs='EPSG:4326')
for i in range (0,len(marine_based_glacier_area_lost_late)):
    projection = ccrs.Orthographic(central_longitude=marine_based_glacier_area_lost_late.loc[i]['latlon'].x, central_latitude=marine_based_glacier_area_lost_late.loc[i]['latlon'].y)
    marine_based_glacier_area_lost_late = marine_based_glacier_area_lost_late.to_crs(projection)
    marine_based_glacier_area_lost_late.loc[[i],'geometry']=marine_based_glacier_area_lost_late.loc[[i],'geometry'].buffer(-0.000001)
    marine_based_glacier_area_lost_late.loc[i,'area_km2'] = marine_based_glacier_area_lost_late.loc[i,'geometry'].area/1000000
    marine_based_glacier_area_lost_late.loc[i,'perimeter'] = marine_based_glacier_area_lost_late.loc[i,'geometry'].length
    marine_based_glacier_area_lost_late.loc[i,'perimeter_error'] = marine_based_glacier_area_lost_late.loc[i,'perimeter']/30*30*30/1000000 #/pixels/meter *area/pixel /m2/km2
marine_based_glacier_area_lost_late['perimeter_error'] = marine_based_glacier_area_lost_late['perimeter_error'].fillna(0)
marine_based_glacier_area_lost_late=marine_based_glacier_area_lost_late[marine_based_glacier_area_lost_late['area_km2']>0.000225] #throw out tiny areas that are a mistakes

#calculate total area gained from 2000 to 2010
marine_based_glacier_area_gained_late = gpd.overlay(late_termini_intersect,middle_termini_intersect,how='difference')
marine_based_glacier_area_gained_late=marine_based_glacier_area_gained_late[['RGIId','Name','geometry']].dissolve(by='RGIId')
marine_based_glacier_area_gained_late = marine_based_glacier_area_gained_late.reset_index()
marine_based_glacier_area_gained_late['geometry_3995']=marine_based_glacier_area_gained_late['geometry']
marine_based_glacier_area_gained_late.loc[:,'latlon'] = marine_based_glacier_area_gained_late.centroid.to_crs(crs='EPSG:4326')
for i in range (0,len(marine_based_glacier_area_gained_late)):
    projection = ccrs.Orthographic(central_longitude=marine_based_glacier_area_gained_late.loc[i]['latlon'].x, central_latitude=marine_based_glacier_area_gained_late.loc[i]['latlon'].y)
    marine_based_glacier_area_gained_late = marine_based_glacier_area_gained_late.to_crs(projection)
    marine_based_glacier_area_gained_late.loc[[i],'geometry']=marine_based_glacier_area_gained_late.loc[[i],'geometry'].buffer(-0.000001)
    marine_based_glacier_area_gained_late.loc[i,'area_km2'] = marine_based_glacier_area_gained_late.loc[i,'geometry'].area/1000000
    marine_based_glacier_area_gained_late.loc[i,'perimeter'] = marine_based_glacier_area_gained_late.loc[i,'geometry'].length
    marine_based_glacier_area_gained_late.loc[i,'perimeter_error'] = marine_based_glacier_area_gained_late.loc[i,'perimeter']/30*30*30/1000000 #/pixels/meter *area/pixel /m2/km2
marine_based_glacier_area_gained_late['perimeter_error'] = marine_based_glacier_area_gained_late['perimeter_error'].fillna(0)
marine_based_glacier_area_gained_late=marine_based_glacier_area_gained_late[marine_based_glacier_area_gained_late['area_km2']>0.000225] #throw out tiny areas that are a mistakes

#export data
marine_based_glacier_area_lost_late.loc[:,'geometry']=marine_based_glacier_area_lost_late['geometry_3995']
marine_based_glacier_area_gained_late.loc[:,'geometry']=marine_based_glacier_area_gained_late['geometry_3995']
marine_based_glacier_area_lost_late.drop(columns=['geometry_3995','latlon'],inplace=True)
marine_based_glacier_area_gained_late.drop(columns=['geometry_3995','latlon'],inplace=True)

#export data
marine_based_glacier_area_lost_late.to_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/marine_based_glacier_area_lost_late_'+date_string+'.geojson', driver='GeoJSON')
marine_based_glacier_area_gained_late.to_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/marine_based_glacier_area_gained_late_'+date_string+'.geojson', driver='GeoJSON')


#create one list of all the glaciers we have terminus area change for
all_glacier_list=marine_based_glacier_area_lost_early.append([marine_based_glacier_area_gained_early, marine_based_glacier_area_lost_late, marine_based_glacier_area_gained_late])
all_glacier_list=all_glacier_list.groupby('RGIId').min().reset_index()[['RGIId','Name']]

#merge datasets to calculate net area change
all_glacier_list=pd.merge(all_glacier_list,marine_based_glacier_area_gained_total,on=['RGIId'],how='left')
net_area_change = pd.merge(all_glacier_list,marine_based_glacier_area_lost_total,on=['RGIId'],how='left').drop(columns=['Name_y','Name']).rename(columns={"geometry_x": "geometry_gained_total", "area_km2_x": "area_km2_gained_total","geometry_y": "geometry_lost_total", "area_km2_y": "area_km2_lost_total",'Name_x':'Name','perimeter_x':'perimeter_gained_total','perimeter_y':'perimeter_lost_total','perimeter_error_x':'perimeter_error_gained_total','perimeter_error_y':'perimeter_error_lost_total'})
net_area_change = pd.merge(net_area_change,marine_based_glacier_area_lost_early,on=['RGIId'],how='left').drop(columns=['Name_y']).rename(columns={"geometry": "geometry_lost_early", "area_km2": "area_km2_lost_early",'Name_x':'Name','perimeter':'perimeter_lost_early','perimeter_error':'perimeter_error_lost_early'})
net_area_change = pd.merge(net_area_change,marine_based_glacier_area_gained_early,on=['RGIId'],how='left').drop(columns=['Name_y']).rename(columns={"geometry": "geometry_gained_early", "area_km2": "area_km2_gained_early",'Name_x':'Name','perimeter':'perimeter_gained_early','perimeter_error':'perimeter_error_gained_early'})
net_area_change = pd.merge(net_area_change,marine_based_glacier_area_lost_late,on=['RGIId'],how='left').drop(columns=['Name_y']).rename(columns={"geometry": "geometry_lost_late", "area_km2": "area_km2_lost_late",'Name_x':'Name','perimeter':'perimeter_lost_late','perimeter_error':'perimeter_error_lost_late'})
net_area_change = pd.merge(net_area_change,marine_based_glacier_area_gained_late,on=['RGIId'],how='left').drop(columns=['Name_y']).rename(columns={"geometry": "geometry_gained_late", "area_km2": "area_km2_gained_late",'Name_x':'Name','perimeter':'perimeter_gained_late','perimeter_error':'perimeter_error_gained_late'})
net_area_change = pd.merge(net_area_change,boxes_attributed[['RGIId','Early_satellite_id','Middle_satellite_id', 'Late_satellite_id', 'Early_date','Mid_date','Late_date']],on=['RGIId'],how='left')


#Set areas to 0 where there is no gain (but a loss) or no loss (but a gain) so that you still get results for net change 
net_area_change=net_area_change.reset_index()
for i in range(0,len(net_area_change)):
    if net_area_change['area_km2_gained_early'].isnull()[i] and (net_area_change['area_km2_lost_early'][i]>0):
        net_area_change.loc[i,'area_km2_gained_early']=0
    if net_area_change['area_km2_lost_early'].isnull()[i] and (net_area_change['area_km2_gained_early'][i]>0):
        net_area_change.loc[i,'area_km2_lost_early']=0
        
    if net_area_change['area_km2_gained_late'].isnull()[i] and (net_area_change['area_km2_lost_late'][i]>0):
        net_area_change.loc[i,'area_km2_gained_late']=0
    if net_area_change['area_km2_lost_late'].isnull()[i] and (net_area_change['area_km2_gained_late'][i]>0):
        net_area_change.loc[i,'area_km2_lost_late']=0
        
    if net_area_change['area_km2_gained_total'].isnull()[i] and (net_area_change['area_km2_lost_total'][i]>0):
        net_area_change.loc[i,'area_km2_gained_total']=0
    if net_area_change['area_km2_lost_total'].isnull()[i] and (net_area_change['area_km2_gained_total'][i]>0):
        net_area_change.loc[i,'area_km2_lost_total']=0


#calculate time elapsed between two images
net_area_change['Early_date_decimal_year']=np.nan
net_area_change['Middle_date_decimal_year']=np.nan
net_area_change['Late_date_decimal_year']=np.nan
for i in range(0,len(net_area_change)):
    if pd.isna(net_area_change['Early_date'][i])==False:#this way it skips nan values
        net_area_change.loc[i,'Early_date_decimal_year'] = net_area_change['Early_date'][i].year+net_area_change['Early_date'][i].dayofyear/365
    if pd.isna(net_area_change['Mid_date'][i])==False:#this way it skips nan values
        net_area_change.loc[i,'Middle_date_decimal_year'] = net_area_change['Mid_date'][i].year+net_area_change['Mid_date'][i].dayofyear/365
    if pd.isna(net_area_change['Late_date'][i])==False:#this way it skips nan values
        net_area_change.loc[i,'Late_date_decimal_year'] = net_area_change['Late_date'][i].year+net_area_change['Late_date'][i].dayofyear/365
net_area_change.loc[:,'total_years_elapsed']=net_area_change['Late_date_decimal_year']-net_area_change['Early_date_decimal_year']
net_area_change.loc[:,'early_to_middle_years_elapsed']=net_area_change['Middle_date_decimal_year']-net_area_change['Early_date_decimal_year']
net_area_change.loc[:,'middle_to_late_years_elapsed']=net_area_change['Late_date_decimal_year']-net_area_change['Middle_date_decimal_year']

net_area_change['total_years_elapsed'].fillna(20,inplace=True)
net_area_change['early_to_middle_years_elapsed'].fillna(10,inplace=True)
net_area_change['middle_to_late_years_elapsed'].fillna(10,inplace=True)

#clean up the dataset by replacing nans with 0 (since you have no area if nan) and then find net change
net_area_change=net_area_change.fillna(value={'area_km2_gained_early':0,'area_km2_gained_late':0,'area_km2_lost_early':0,'area_km2_lost_late':0,
                                              'perimeter_error_gained_early':0,'perimeter_error_lost_early':0,'perimeter_error_gained_late':0,'perimeter_error_lost_late':0,
                                             'area_km2_gained_total':0,'area_km2_lost_total':0,'perimeter_error_gained_total':0,'perimeter_error_lost_total':0})
net_area_change=net_area_change.drop_duplicates(subset='RGIId')
net_area_change.loc[:,'net_area_change_total']=net_area_change['area_km2_gained_total']-net_area_change['area_km2_lost_total']
net_area_change.loc[:,'net_area_change_total_error']=net_area_change['perimeter_error_gained_total']+net_area_change['perimeter_error_lost_total']
net_area_change.loc[:,'net_area_change_per_year_total']=net_area_change['net_area_change_total']/net_area_change['total_years_elapsed']
net_area_change.loc[:,'net_area_change_per_year_total_error']=net_area_change['net_area_change_total_error']/net_area_change['total_years_elapsed']
net_area_change.loc[:,'net_area_change_early']=net_area_change['area_km2_gained_early']-net_area_change['area_km2_lost_early']
net_area_change.loc[:,'net_area_change_early_error']=net_area_change['perimeter_error_gained_early']+net_area_change['perimeter_error_lost_early']
net_area_change.loc[:,'net_area_change_per_year_early']=net_area_change['net_area_change_early']/net_area_change['early_to_middle_years_elapsed']
net_area_change.loc[:,'net_area_change_per_year_early_error']=net_area_change['net_area_change_early_error']/net_area_change['early_to_middle_years_elapsed']
net_area_change.loc[:,'net_area_change_late']=net_area_change['area_km2_gained_late']-net_area_change['area_km2_lost_late']
net_area_change.loc[:,'net_area_change_late_error']=net_area_change['perimeter_error_gained_late']+net_area_change['perimeter_error_lost_late']
net_area_change.loc[:,'net_area_change_per_year_late']=net_area_change['net_area_change_late']/net_area_change['middle_to_late_years_elapsed']
net_area_change.loc[:,'net_area_change_per_year_late_error']=net_area_change['net_area_change_late_error']/net_area_change['middle_to_late_years_elapsed']

#make pretty dates
net_area_change.loc[:,'Early_date'] = pd.to_datetime(net_area_change['Early_date']).dt.strftime('%Y/%m/%d')
net_area_change.loc[:,'Mid_date'] = pd.to_datetime(net_area_change['Mid_date']).dt.strftime('%Y/%m/%d')
net_area_change.loc[:,'Late_date'] = pd.to_datetime(net_area_change['Late_date']).dt.strftime('%Y/%m/%d')

#Do not know why I need this, but you do to make the total area change numbers work out
# net_area_change['net_area_change_total_error']=net_area_change['perimeter_error_gained_total']+net_area_change['perimeter_error_lost_total']
# net_area_change['net_area_change_per_year_total_error']=net_area_change['net_area_change_total_error']/net_area_change['total_years_elapsed']

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:81: UserWarning: `keep_geom_type=True` in overlay resulted in 2477 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:233: FutureWarning: Dropping invalid columns in DataFrameGroupBy.min is deprecated. In a future version, a TypeError will be raised. Before calling .min, select only columns which should be valid for the function.


In [42]:
#Fill zeros in area change data so that errors dont come out null
net_area_change[['index', 'RGIId', 'Name', #'geometry_gained_total',
       'area_km2_gained_total', 'perimeter_gained_total',
       'perimeter_error_gained_total', #'geometry_lost_total',
       'area_km2_lost_total', 'perimeter_lost_total',
       'perimeter_error_lost_total', #'geometry_lost_early',
       'area_km2_lost_early', 'perimeter_lost_early',
       'perimeter_error_lost_early', #'geometry_gained_early',
       'area_km2_gained_early', 'perimeter_gained_early',
       'perimeter_error_gained_early', #'geometry_lost_late',
       'area_km2_lost_late', 'perimeter_lost_late',
       'perimeter_error_lost_late', #'geometry_gained_late',
       'area_km2_gained_late', 'perimeter_gained_late',
       'perimeter_error_gained_late', 'Early_date', 'Mid_date',
       'Late_date', 'Early_date_decimal_year', 'Middle_date_decimal_year',
       'Late_date_decimal_year', 'total_years_elapsed',
       'early_to_middle_years_elapsed', 'middle_to_late_years_elapsed',
       'net_area_change_total', 'net_area_change_total_error',
       'net_area_change_per_year_total',
       'net_area_change_per_year_total_error', 'net_area_change_early',
       'net_area_change_early_error', 'net_area_change_per_year_early',
       'net_area_change_per_year_early_error', 'net_area_change_late',
       'net_area_change_late_error', 'net_area_change_per_year_late',
       'net_area_change_per_year_late_error']] = net_area_change[['index', 'RGIId', 'Name', #'geometry_gained_total',
       'area_km2_gained_total', 'perimeter_gained_total',
       'perimeter_error_gained_total', #'geometry_lost_total',
       'area_km2_lost_total', 'perimeter_lost_total',
       'perimeter_error_lost_total', #'geometry_lost_early',
       'area_km2_lost_early', 'perimeter_lost_early',
       'perimeter_error_lost_early', #'geometry_gained_early',
       'area_km2_gained_early', 'perimeter_gained_early',
       'perimeter_error_gained_early',# 'geometry_lost_late',
       'area_km2_lost_late', 'perimeter_lost_late',
       'perimeter_error_lost_late',# 'geometry_gained_late',
       'area_km2_gained_late', 'perimeter_gained_late',
       'perimeter_error_gained_late', 'Early_date', 'Mid_date',
       'Late_date', 'Early_date_decimal_year', 'Middle_date_decimal_year',
       'Late_date_decimal_year', 'total_years_elapsed',
       'early_to_middle_years_elapsed', 'middle_to_late_years_elapsed',
       'net_area_change_total', 'net_area_change_total_error',
       'net_area_change_per_year_total',
       'net_area_change_per_year_total_error', 'net_area_change_early',
       'net_area_change_early_error', 'net_area_change_per_year_early',
       'net_area_change_per_year_early_error', 'net_area_change_late',
       'net_area_change_late_error', 'net_area_change_per_year_late',
       'net_area_change_per_year_late_error']].fillna(value=0)

In [43]:
net_area_change = net_area_change.reset_index()
column_names = ['geometry_gained_total','geometry_lost_total','geometry_lost_early','geometry_gained_early','geometry_lost_late','geometry_gained_late']
for c in column_names:
    for i in range(0,len(net_area_change)):
        try:
            if net_area_change[c][i] == None:
                net_area_change.loc[i,c] = Polygon([(0, 0),(0, 0),(0, 0)])
        except:
              print("An exception occurred")

In [44]:
#Figure out thickness of lost/gained glacier area
GlaThiDa= gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Arctic_GlaThiDa_EPSG3995.gpkg')
Russia_thickness_andrey = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Russia/Russia_thickness_merged_EPSG3995.geojson')
Russia_thickness_julian = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RES_data/RES_ice_thickness.gpkg')

net_area_change['average_GlaThiDa_thickness_lost_early'] = np.nan
net_area_change['average_GlaThiDa_thickness_gained_early'] = np.nan
net_area_change['average_GlaThiDa_thickness_lost_late'] = np.nan
net_area_change['average_GlaThiDa_thickness_gained_late'] = np.nan
net_area_change['average_GlaThiDa_thickness_lost_early_err'] = np.nan
net_area_change['average_GlaThiDa_thickness_gained_early_err'] = np.nan
net_area_change['average_GlaThiDa_thickness_lost_late_err'] = np.nan
net_area_change['average_GlaThiDa_thickness_gained_late_err'] = np.nan
net_area_change['average_RUS_andrey_thickness_lost_early'] = np.nan
net_area_change['average_RUS_andrey_thickness_gained_early'] = np.nan
net_area_change['average_RUS_andrey_thickness_lost_late'] = np.nan
net_area_change['average_RUS_andrey_thickness_gained_late'] = np.nan
net_area_change['average_RUS_julian_thickness_lost_early'] = np.nan
net_area_change['average_RUS_julian_thickness_gained_early'] = np.nan
net_area_change['average_RUS_julian_thickness_lost_late'] = np.nan
net_area_change['average_RUS_julian_thickness_gained_late'] = np.nan

net_area_change['average_GlaThiDa_thickness_lost_total'] = np.nan
net_area_change['average_GlaThiDa_thickness_lost_total_err'] = np.nan
net_area_change['average_GlaThiDa_thickness_gained_total'] = np.nan
net_area_change['average_GlaThiDa_thickness_gained_total_err'] = np.nan
net_area_change['average_RUS_andrey_thickness_lost_total'] = np.nan
net_area_change['average_RUS_andrey_thickness_gained_total'] = np.nan
net_area_change['average_RUS_julian_thickness_lost_total'] = np.nan
net_area_change['average_RUS_julian_thickness_gained_total'] = np.nan

for i in range(0,len(net_area_change)):
    if net_area_change['geometry_lost_early'][i]!=0:
        net_area_change.loc[i,'average_GlaThiDa_thickness_lost_early'] = GlaThiDa[GlaThiDa.within(net_area_change['geometry_lost_early'][i])]['THICKNESS'].mean()
        net_area_change.loc[i,'average_GlaThiDa_thickness_lost_early_err'] = GlaThiDa[GlaThiDa.within(net_area_change['geometry_lost_early'][i])]['THICKNESS_UNCERTAINTY'].mean()
        net_area_change.loc[i,'average_RUS_andrey_thickness_lost_early'] = Russia_thickness_andrey[Russia_thickness_andrey.within(net_area_change['geometry_lost_early'][i])]['Russian_thickness'].mean()
        net_area_change.loc[i,'average_RUS_julian_thickness_lost_early'] = Russia_thickness_julian[Russia_thickness_julian.within(net_area_change['geometry_lost_early'][i])]['thickness(m)'].mean()
    if net_area_change['geometry_gained_early'][i]!=0:
        net_area_change.loc[i,'average_GlaThiDa_thickness_gained_early'] = GlaThiDa[GlaThiDa.within(net_area_change['geometry_gained_early'][i])]['THICKNESS'].mean()
        net_area_change.loc[i,'average_GlaThiDa_thickness_gained_early_err'] = GlaThiDa[GlaThiDa.within(net_area_change['geometry_gained_early'][i])]['THICKNESS_UNCERTAINTY'].mean()
        net_area_change.loc[i,'average_RUS_andrey_thickness_gained_early'] = Russia_thickness_andrey[Russia_thickness_andrey.within(net_area_change['geometry_gained_early'][i])]['Russian_thickness'].mean()
        net_area_change.loc[i,'average_RUS_julian_thickness_gained_early'] = Russia_thickness_julian[Russia_thickness_julian.within(net_area_change['geometry_gained_early'][i])]['thickness(m)'].mean()
    if net_area_change['geometry_lost_late'][i]!=0:
        net_area_change.loc[i,'average_GlaThiDa_thickness_lost_late'] = GlaThiDa[GlaThiDa.within(net_area_change['geometry_lost_late'][i])]['THICKNESS'].mean()
        net_area_change.loc[i,'average_GlaThiDa_thickness_lost_late_err'] = GlaThiDa[GlaThiDa.within(net_area_change['geometry_lost_late'][i])]['THICKNESS_UNCERTAINTY'].mean()
        net_area_change.loc[i,'average_RUS_andrey_thickness_lost_late']= Russia_thickness_andrey[Russia_thickness_andrey.within(net_area_change['geometry_lost_late'][i])]['Russian_thickness'].mean()
        net_area_change.loc[i,'average_RUS_julian_thickness_lost_late'] = Russia_thickness_julian[Russia_thickness_julian.within(net_area_change['geometry_lost_late'][i])]['thickness(m)'].mean()
    if net_area_change['geometry_gained_late'][i]!=0:
        net_area_change.loc[i,'average_GlaThiDa_thickness_gained_late'] = GlaThiDa[GlaThiDa.within(net_area_change['geometry_gained_late'][i])]['THICKNESS'].mean()
        net_area_change.loc[i,'average_GlaThiDa_thickness_gained_late_err'] = GlaThiDa[GlaThiDa.within(net_area_change['geometry_gained_late'][i])]['THICKNESS_UNCERTAINTY'].mean()
        net_area_change.loc[i,'average_RUS_andrey_thickness_gained_late'] = Russia_thickness_andrey[Russia_thickness_andrey.within(net_area_change['geometry_gained_late'][i])]['Russian_thickness'].mean()
        net_area_change.loc[i,'average_RUS_julian_thickness_gained_late'] = Russia_thickness_julian[Russia_thickness_julian.within(net_area_change['geometry_gained_late'][i])]['thickness(m)'].mean()
    if net_area_change['geometry_lost_total'][i]!=0:
        net_area_change.loc[i,'average_GlaThiDa_thickness_lost_total'] = GlaThiDa[GlaThiDa.within(net_area_change['geometry_lost_total'][i])]['THICKNESS'].mean()
        net_area_change.loc[i,'average_GlaThiDa_thickness_lost_total_err'] = GlaThiDa[GlaThiDa.within(net_area_change['geometry_lost_total'][i])]['THICKNESS_UNCERTAINTY'].mean()
        net_area_change.loc[i,'average_RUS_andrey_thickness_lost_total'] = Russia_thickness_andrey[Russia_thickness_andrey.within(net_area_change['geometry_lost_total'][i])]['Russian_thickness'].mean()
        net_area_change.loc[i,'average_RUS_julian_thickness_lost_total'] = Russia_thickness_julian[Russia_thickness_julian.within(net_area_change['geometry_lost_total'][i])]['thickness(m)'].mean()
    if net_area_change['geometry_gained_total'][i]!=0:
        net_area_change.loc[i,'average_GlaThiDa_thickness_gained_total'] = GlaThiDa[GlaThiDa.within(net_area_change['geometry_gained_total'][i])]['THICKNESS'].mean()
        net_area_change.loc[i,'average_GlaThiDa_thickness_gained_total_err'] = GlaThiDa[GlaThiDa.within(net_area_change['geometry_gained_total'][i])]['THICKNESS_UNCERTAINTY'].mean()
        net_area_change.loc[i,'average_RUS_andrey_thickness_gained_total'] = Russia_thickness_andrey[Russia_thickness_andrey.within(net_area_change['geometry_gained_total'][i])]['Russian_thickness'].mean()
        net_area_change.loc[i,'average_RUS_julian_thickness_gained_total'] = Russia_thickness_julian[Russia_thickness_julian.within(net_area_change['geometry_gained_total'][i])]['thickness(m)'].mean()
        
#Find the Glacier thickness for the areas lost/gained from the Farinotti and other raster datasets
net_area_change['average_Milan_thickness_lost_early'] = np.nan
net_area_change['average_Milan_thickness_gained_early'] = np.nan
net_area_change['average_Milan_thickness_lost_late'] = np.nan
net_area_change['average_Milan_thickness_gained_late'] = np.nan

net_area_change['average_Milan_thickness_lost_early_err'] = np.nan
net_area_change['average_Milan_thickness_gained_early_err'] = np.nan
net_area_change['average_Milan_thickness_lost_late_err'] = np.nan
net_area_change['average_Milan_thickness_gained_late_err'] = np.nan

net_area_change['average_Milan_thickness_lost_total'] = np.nan
net_area_change['average_Milan_thickness_gained_total'] = np.nan
net_area_change['average_Milan_thickness_lost_total_err'] = np.nan
net_area_change['average_Milan_thickness_gained_total_err'] = np.nan

net_area_change['average_tomo_thickness_lost_early'] = np.nan
net_area_change['average_tomo_thickness_gained_early'] = np.nan
net_area_change['average_tomo_thickness_lost_late'] = np.nan
net_area_change['average_tomo_thickness_gained_late'] = np.nan

net_area_change['average_tomo_thickness_lost_total'] = np.nan
net_area_change['average_tomo_thickness_gained_total'] = np.nan


for i in range(0,len(net_area_change)):
    if net_area_change['geometry_lost_early'][i]!=0:
        net_area_change.loc[i,'average_Milan_thickness_lost_early']=rasterstats.zonal_stats(net_area_change['geometry_lost_early'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/Milan_thickness_mosaic.vrt',stats='mean')[0]['mean']
        net_area_change.loc[i,'average_Milan_thickness_lost_early_err']=rasterstats.zonal_stats(net_area_change['geometry_lost_early'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/Milan_thickness_err_mosaic.vrt',stats='mean')[0]['mean']
        net_area_change.loc[i,'average_tomo_thickness_lost_early']=rasterstats.zonal_stats(net_area_change['geometry_lost_early'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/tomo/tomo_glacier_thickness_0_nodata2_v5.tif',stats='mean')[0]['mean']
    if net_area_change['geometry_gained_early'][i]!=0:
        net_area_change.loc[i,'average_Milan_thickness_gained_early']=rasterstats.zonal_stats(net_area_change['geometry_gained_early'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/Milan_thickness_mosaic.vrt',stats='mean')[0]['mean']
        net_area_change.loc[i,'average_Milan_thickness_gained_early_err']=rasterstats.zonal_stats(net_area_change['geometry_gained_early'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/Milan_thickness_err_mosaic.vrt',stats='mean')[0]['mean']
        net_area_change.loc[i,'average_tomo_thickness_gained_early']=rasterstats.zonal_stats(net_area_change['geometry_gained_early'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/tomo/tomo_glacier_thickness_0_nodata2_v5.tif',stats='mean')[0]['mean']
    if net_area_change['geometry_lost_late'][i]!=0:
        net_area_change.loc[i,'average_Milan_thickness_lost_late']=rasterstats.zonal_stats(net_area_change['geometry_lost_late'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/Milan_thickness_mosaic.vrt',stats='mean')[0]['mean']
        net_area_change.loc[i,'average_Milan_thickness_lost_late_err']=rasterstats.zonal_stats(net_area_change['geometry_lost_late'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/Milan_thickness_err_mosaic.vrt',stats='mean')[0]['mean']
        net_area_change.loc[i,'average_tomo_thickness_lost_late']=rasterstats.zonal_stats(net_area_change['geometry_lost_late'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/tomo/tomo_glacier_thickness_0_nodata2_v5.tif',stats='mean')[0]['mean']
    if net_area_change['geometry_gained_late'][i]!=0:
        net_area_change.loc[i,'average_Milan_thickness_gained_late']=rasterstats.zonal_stats(net_area_change['geometry_gained_late'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/Milan_thickness_mosaic.vrt',stats='mean')[0]['mean']
        net_area_change.loc[i,'average_Milan_thickness_gained_late_err']=rasterstats.zonal_stats(net_area_change['geometry_gained_late'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/Milan_thickness_err_mosaic.vrt',stats='mean')[0]['mean']
        net_area_change.loc[i,'average_tomo_thickness_gained_late']=rasterstats.zonal_stats(net_area_change['geometry_gained_late'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/tomo/tomo_glacier_thickness_0_nodata2_v5.tif',stats='mean')[0]['mean']
    if net_area_change['geometry_lost_total'][i]!=0:
        net_area_change.loc[i,'average_Milan_thickness_lost_total']=rasterstats.zonal_stats(net_area_change['geometry_lost_late'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/Milan_thickness_mosaic.vrt',stats='mean')[0]['mean']
        net_area_change.loc[i,'average_Milan_thickness_lost_total_err']=rasterstats.zonal_stats(net_area_change['geometry_lost_late'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/Milan_thickness_err_mosaic.vrt',stats='mean')[0]['mean']
        net_area_change.loc[i,'average_tomo_thickness_lost_total']=rasterstats.zonal_stats(net_area_change['geometry_lost_total'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/tomo/tomo_glacier_thickness_0_nodata2_v5.tif',stats='mean')[0]['mean']
    if net_area_change['geometry_gained_total'][i]!=0:
        net_area_change.loc[i,'average_Milan_thickness_gained_total']=rasterstats.zonal_stats(net_area_change['geometry_gained_total'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/Milan_thickness_mosaic.vrt',stats='mean')[0]['mean']
        net_area_change.loc[i,'average_Milan_thickness_gained_total_err']=rasterstats.zonal_stats(net_area_change['geometry_gained_total'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Romain/Milan_thickness_err_mosaic.vrt',stats='mean')[0]['mean']
        net_area_change.loc[i,'average_tomo_thickness_gained_total']=rasterstats.zonal_stats(net_area_change['geometry_gained_total'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/tomo/tomo_glacier_thickness_0_nodata2_v5.tif',stats='mean')[0]['mean']

net_area_change.to_pickle('net_area_change'+date_string+'.pkl')#change file name based on distance


In [45]:
#calculate the melt below the flux gate
flux_lines = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/vector_files/Arctic_marine_terminating_fluxgates.gpkg')  
#Read in RGI and fluxgate points (on and off glacier) data
RGI = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/RGI_Arctic_marine_terminating_glacier_outlines.gpkg')   
RGI['geometry'] = RGI.geometry.buffer(0)
RGI['Name']=RGI['Name2']
boxes = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/Glacier_area_change_boxes.gpkg')

#produce stranded portion of polygons for each glacier
for g in range(0,len(RGI)):
    for i in range(0,len(flux_lines)):
        if flux_lines.geometry.iloc[i].intersects(RGI.geometry.iloc[g]):
            split_output = split(RGI.geometry.iloc[g],flux_lines.geometry.iloc[i])#.to_file('test_split_RGI_terminus_regions',driver='GPKG')
            gdf = gpd.GeoDataFrame()
            for k in range(0,len(split_output)):
                gdf.loc[k, 'geometry'] = split_output.geoms[k]
                gdf.crs = from_epsg(3995)
                gdf.loc[k, 'area'] = split_output[k].area/1000000
            gdf = gpd.sjoin(gdf,boxes,how='inner')
                
            if g==0:
                stranded_polys = gdf.sort_values('area')[0:1]
            if g>0:
                stranded_polys = stranded_polys.append(gdf.sort_values('area')[0:1])
            
            break
stranded_polys.to_file('stranded_polygons4_'+date_string+'.gpkg',driver='GPKG')

#attribute boxes
boxes = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/Glacier_area_change_boxes.gpkg')
boxes['geometry'] = boxes['geometry'].buffer(-3)

termini_2000 = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/termini_2000.geojson')
termini_2000 = gpd.overlay(termini_2000,boxes,how='intersection')
termini_2010 = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/termini_2010.geojson')
termini_2010 = gpd.overlay(termini_2010,boxes,how='intersection')
termini_2020 = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/termini_2020.geojson')
termini_2020 = gpd.overlay(termini_2020,boxes,how='intersection')

boxes_attributed = gpd.sjoin(boxes, RGI, how='inner')
RGI_buffer = RGI
RGI_buffer['geometry'] = RGI_buffer['geometry'].buffer(-0.1)
max_glacier_attributed = gpd.overlay(RGI_buffer,boxes_attributed,how='union')
max_glacier_attributed['RGIid_2_1'].fillna(max_glacier_attributed['RGIid_2_2'],inplace=True)
max_glacier_attributed = max_glacier_attributed.dissolve(by='RGIid_2_1').reset_index()
max_glacier_attributed = max_glacier_attributed[['geometry','RGIid_2_1']]
max_glacier_attributed.to_file('max_glacier_attributed_v1_'+date_string+'.gpkg',driver='GPKG')

#Get polygons that are just the area we need for computing the area below flux gate for each year
no_ice_2000 = gpd.overlay(boxes,termini_2000,how='difference')
no_ice_2000['geometry'] = no_ice_2000['geometry'].buffer(-0.1)
termini_2000_extra = gpd.overlay(termini_2000,RGI,how='difference')
standed_poly_early = gpd.overlay(termini_2000_extra,stranded_polys,how='union')
standed_poly_early = gpd.overlay(standed_poly_early,no_ice_2000,how='difference')
standed_poly_early = gpd.sjoin(standed_poly_early[['geometry']], max_glacier_attributed, how='inner')
standed_poly_early = standed_poly_early.dissolve(by='RGIid_2_1').reset_index()
standed_poly_early['geometry'] = standed_poly_early['geometry'].buffer(0.1)
standed_poly_early.to_file('stranded_polygons_early_v2_'+date_string+'.gpkg',driver='GPKG')

no_ice_2010 = gpd.overlay(boxes,termini_2010,how='difference')
no_ice_2010['geometry'] = no_ice_2010['geometry'].buffer(-0.1)
termini_2010_extra = gpd.overlay(termini_2010,RGI,how='difference')
standed_poly_middle = gpd.overlay(termini_2010_extra,standed_poly_early,how='union')
standed_poly_middle.to_file('stranded_polygons_middle1_v2_'+date_string,driver='GPKG')
marine_based_glacier_area_lost_early = gpd.overlay(termini_2000,termini_2010,how='difference')
marine_based_glacier_area_gained_early = gpd.overlay(termini_2010,termini_2000,how='difference')
marine_based_glacier_area_lost_early.geometry = [loads(dumps(geom, rounding_precision=3)) for geom in marine_based_glacier_area_lost_early.geometry]
standed_poly_middle.geometry = [loads(dumps(geom, rounding_precision=3)) for geom in standed_poly_middle.geometry]
standed_poly_middle = gpd.overlay(standed_poly_middle,marine_based_glacier_area_lost_early,how='difference')
standed_poly_middle.to_file('stranded_polygons_middle2_v2_'+date_string+'.gpkg',driver='GPKG')
standed_poly_middle = standed_poly_middle[(standed_poly_middle['geometry'].type=='Polygon')|(standed_poly_middle['geometry'].type=='MultiPolygon')]
standed_poly_middle.to_file('stranded_polygons_middle3_v2_'+date_string+'.gpkg',driver='GPKG')
standed_poly_middle = gpd.overlay(marine_based_glacier_area_gained_early,standed_poly_middle,how='union')
standed_poly_middle.to_file('stranded_polygons_middle4_v2_'+date_string+'.gpkg',driver='GPKG')
standed_poly_middle = gpd.overlay(standed_poly_middle,no_ice_2010,how='difference')
standed_poly_middle.to_file('stranded_polygons_middle5_v2_'+date_string+'.gpkg',driver='GPKG')
standed_poly_middle['geometry'] = standed_poly_middle['geometry'].buffer(-0.1)
standed_poly_middle = gpd.sjoin(standed_poly_middle[['geometry']], max_glacier_attributed, how='inner')
standed_poly_middle = standed_poly_middle.dissolve(by='RGIid_2_1').reset_index()
standed_poly_middle['geometry'] = standed_poly_middle['geometry'].buffer(0.3)
standed_poly_middle.to_file('stranded_polygons_middle_v2_'+date_string+'.gpkg',driver='GPKG')

no_ice_2020 = gpd.overlay(boxes,termini_2020,how='difference')
no_ice_2020['geometry'] = no_ice_2020['geometry'].buffer(-0.1)
termini_2020_extra = gpd.overlay(termini_2020,RGI,how='difference')
standed_poly_late = gpd.overlay(termini_2020_extra,standed_poly_middle,how='union')
standed_poly_late.to_file('stranded_polygons_late1_v2_'+date_string,driver='GPKG')
marine_based_glacier_area_lost_late = gpd.overlay(termini_2010,termini_2020,how='difference')
marine_based_glacier_area_gained_late = gpd.overlay(termini_2020,termini_2010,how='difference')
marine_based_glacier_area_lost_late.geometry = [loads(dumps(geom, rounding_precision=3)) for geom in marine_based_glacier_area_lost_late.geometry]
standed_poly_late.geometry = [loads(dumps(geom, rounding_precision=3)) for geom in standed_poly_late.geometry]
standed_poly_late = gpd.overlay(standed_poly_late,marine_based_glacier_area_lost_late,how='difference')
standed_poly_late.to_file('stranded_polygons_late2_v2_'+date_string+'.gpkg',driver='GPKG')
standed_poly_late = standed_poly_late[(standed_poly_late['geometry'].type=='Polygon')|(standed_poly_late['geometry'].type=='MultiPolygon')]
standed_poly_late = gpd.overlay(marine_based_glacier_area_gained_late,standed_poly_late,how='union')
standed_poly_late.to_file('stranded_polygons_late3_v2_'+date_string+'.gpkg',driver='GPKG')
standed_poly_late = gpd.overlay(standed_poly_late,no_ice_2020,how='difference')
standed_poly_late.to_file('stranded_polygons_late4_v2_'+date_string+'.gpkg',driver='GPKG')
standed_poly_late['geometry'] = standed_poly_late['geometry'].buffer(-0.1)
standed_poly_late = gpd.sjoin(standed_poly_late[['geometry']], max_glacier_attributed, how='inner')
standed_poly_late.to_file('stranded_polygons_late5_v2_'+date_string,driver='GPKG')
standed_poly_late = standed_poly_late.dissolve(by='RGIid_2_1').reset_index()
standed_poly_late['geometry'] = standed_poly_late['geometry'].buffer(0.3)
standed_poly_late.to_file('stranded_polygons_late_v2_'+date_string+'.gpkg',driver='GPKG')

# standed_poly_early['area_below_fluxgate_early']=standed_poly_early.area/1000000
# standed_poly_middle['area_below_fluxgate_middle']=standed_poly_middle.area/1000000
# standed_poly_late['area_below_fluxgate_late']=standed_poly_late.area/1000000

standed_poly_early['geometry_3995'] = standed_poly_early['geometry']
standed_poly_early.loc[:,'latlon'] = standed_poly_early.to_crs(crs='EPSG:4326')['geometry']
for i in range (0,len(standed_poly_early)):
    projection = ccrs.Orthographic(central_longitude=standed_poly_early.loc[i]['latlon'].centroid.x, central_latitude=standed_poly_early.loc[i]['latlon'].centroid.y)
    standed_poly_early = standed_poly_early.to_crs(projection)
    standed_poly_early.loc[i,'area_below_fluxgate_early']=standed_poly_early.loc[i].geometry.area/1000000

standed_poly_middle['geometry_3995'] = standed_poly_middle['geometry']
standed_poly_middle.loc[:,'latlon'] = standed_poly_middle.to_crs(crs='EPSG:4326')['geometry']
for i in range (0,len(standed_poly_middle)):
    projection = ccrs.Orthographic(central_longitude=standed_poly_middle.loc[i]['latlon'].centroid.x, central_latitude=standed_poly_middle.loc[i]['latlon'].centroid.y)
    standed_poly_middle = standed_poly_middle.to_crs(projection)
    standed_poly_middle.loc[i,'area_below_fluxgate_middle']=standed_poly_middle.loc[i].geometry.area/1000000
    
standed_poly_late['geometry_3995'] = standed_poly_late['geometry']
standed_poly_late.loc[:,'latlon'] = standed_poly_late.to_crs(crs='EPSG:4326')['geometry']
for i in range (0,len(standed_poly_late)):
    projection = ccrs.Orthographic(central_longitude=standed_poly_late.loc[i]['latlon'].centroid.x, central_latitude=standed_poly_late.loc[i]['latlon'].centroid.y)
    standed_poly_late = standed_poly_late.to_crs(projection)
    standed_poly_late.loc[i,'area_below_fluxgate_late']=standed_poly_late.loc[i].geometry.area/1000000

area_below_fluxgate = pd.merge(standed_poly_early,standed_poly_middle,left_on='RGIid_2_1',right_on='RGIid_2_1',how='outer',)[['RGIid_2_1','area_below_fluxgate_early','area_below_fluxgate_middle']]
area_below_fluxgate = pd.merge(area_below_fluxgate,standed_poly_late,left_on='RGIid_2_1',right_on='RGIid_2_1',how='outer',)[['RGIid_2_1','area_below_fluxgate_early','area_below_fluxgate_middle','area_below_fluxgate_late']]

Arctic_discharge_fluxes = gpd.read_file('Arctic_discharge_fluxes'+date_string+'.gpkg', driver="GPKG")
Arctic_discharge_fluxes = pd.merge(Arctic_discharge_fluxes,area_below_fluxgate,left_on='RGIId',right_on='RGIid_2_1')

Arctic_discharge_fluxes['average_distance_from_fluxgate_to_terminus_early'] = Arctic_discharge_fluxes['area_below_fluxgate_early']/Arctic_discharge_fluxes['gate_length_km']
Arctic_discharge_fluxes['average_distance_from_fluxgate_to_terminus_middle'] = Arctic_discharge_fluxes['area_below_fluxgate_middle']/Arctic_discharge_fluxes['gate_length_km']
Arctic_discharge_fluxes['average_distance_from_fluxgate_to_terminus_late'] = Arctic_discharge_fluxes['area_below_fluxgate_late']/Arctic_discharge_fluxes['gate_length_km']

Arctic_discharge_fluxes_export = gpd.GeoDataFrame(Arctic_discharge_fluxes, geometry=gpd.points_from_xy(Arctic_discharge_fluxes.easting, Arctic_discharge_fluxes.northing),crs='EPSG:3995')
Arctic_discharge_fluxes_export.to_file('Arctic_discharge_fluxes_with_area_below_fluxgate_'+date_string+'.gpkg', driver="GPKG")

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:3995 +no_defs +type=crs
Right CRS: EPSG:3995

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When mak

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:3995 +no_defs +type=crs
Right CRS: EPSG:3995

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When mak

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:3995 +no_defs +type=crs
Right CRS: EPSG:3995

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When mak

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:3995 +no_defs +type=crs
Right CRS: EPSG:3995

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When mak

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:3995 +no_defs +type=crs
Right CRS: EPSG:3995

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When mak

...

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:3995 +no_defs +type=crs
Right CRS: EPSG:3995

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When mak

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:3995 +no_defs +type=crs
Right CRS: EPSG:3995

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When mak

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:53: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:3995
Right CRS: +init=epsg:3995 +no_defs +type=crs

CBR: result (after common-bits addition) is INVALID: Too few points in geometry component at or near point -2189639.3588226107 -2364330.2701757019 (-2189639.3588226106949 -2364330.2701757019386)
<A>
GEOMETRYCOLLECTION (LINESTRING (-2189639.3588226106949151 -2364330.2701757019385695, -2189639.3588226106949151 -2364330.2701757019385695), POLYGON ((-2188541.4419519072398543 -2364274.5712077380158007, -2188562.0685241785831749 -2364260.0734453927725554, -2188504.2910000002011657 -2364253.1120000001974404, -2188527.2919999998994172 -2364276.3379999999888241, -2188549.9259999999776483 -2364268.6080000000074506, -2188541.4419999998062849 -2364274.570

CBR: result (after common-bits addition) is INVALID: Too few points in geometry component at or near point 795380.29066031217 -629483.84877881652 (795380.29066031216644 -629483.8487788165221)
<A>
GEOMETRYCOLLECTION (LINESTRING (795380.2906603121664375 -629483.8487788165220991, 795380.2906603121664375 -629483.8487788165220991), POLYGON ((796723.4889076383551583 -635645.4913191912928596, 796723.4889073207741603 -635645.4913137520197779, 796723.4889076190302148 -635645.4913188603240997, 796723.4889076383551583 -635645.4913191912928596)), POLYGON ((795464.8412214164854959 -628128.9145249983994290, 795466.0402810415253043 -628188.8675062514375895, 795466.0400000000372529 -628188.8669999999692664, 795464.8410000000149012 -628128.9150000000372529, 795416.2938500519376248 -628085.2218907703645527, 795464.8412214164854959 -628128.9145249983994290)), POLYGON ((795466.0442579396767542 -628189.0798865531105548, 795466.0442579396767542 -628189.0798865529941395, 795466.0400000000372529 -628188.86699

CBR: result (after common-bits addition) is INVALID: Self-intersection at or near point 856393.48093873018 -684068.68483378016 (856393.48093873017933 -684068.6848337801639)
<A>
GEOMETRYCOLLECTION (LINESTRING (856400.2908456089207903 -684060.7260413690237328, 856400.2911022934131324 -684060.7257413796614856), POLYGON ((856393.4809353543678299 -684068.6848377254791558, 856393.4809387301793322 -684068.6848337801638991, 856393.4809353543678299 -684068.6848377255955711, 856393.4809342508669943 -684068.6848390151280910, 856393.4809353543678299 -684068.6848377254791558)), POLYGON ((856393.4809419030789286 -684068.6848300721030682, 856393.4809419718803838 -684068.6848299917764962, 856393.4809419030789286 -684068.6848300721030682, 856393.4809387301793322 -684068.6848337801638991, 856393.4809419030789286 -684068.6848300721030682)))
</A>
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:95: UserWarning: `keep_geom_type=True` in overlay resulted in 82 

In [46]:
Arctic_discharge_fluxes_export

,RGIId,2000_2010_mean_flux_gt,2000_2010_mean_flux_err_gt,2010_2020_mean_flux_gt,2010_2020_mean_flux_err_gt,Name,Area,Zmin,Zmax,Zmed,Slope,Aspect,Lmax,Status,Connect,Form,TermType,Surging,Linkages,Milan_thickness_min,region,Milan_thickness_max,do_we_have_an_observation_in_middle_fifth,easting,northing,Milan_thickness_mean,0000_v,best_thickness_mean,thickness_mean_uncertainty,length_early_avg,length_early_avg_err,length_late_avg,length_late_avg_err,2000_2010_mean,2000_2010_mean_err,2000_2010_mean_count,2010_2018_mean,2010_2018_mean_err,2010_2018_mean_count,best_thickness_early,best_thickness_early_err,best_thickness_late,best_thickness_late_err,number_velocity_obs_early,number_velocity_obs_late,num_pts,GlaThiDa_observations,Tomography_observations,Russian_observations,gate_length_km,geometry,RGIid_2_1,area_below_fluxgate_early,area_below_fluxgate_middle,area_below_fluxgate_late,average_distance_from_fluxgate_to_terminus_early,average_distance_from_fluxgate_to_terminus_middle,average_distance_from_fluxgate_to_terminus_late
0,RGI60-01.03377,0.105642,0.005597,0.090733,0.002826,Dawes Glacier,523.786,105,2966,1477,11.0,278,42419,0,0,0,1,9,9,118.098297,Alaska,509.476562,NaN,-2.660021e+06,2.462404e+06,423.249391,377.949742,423.249391,106.149037,24.766521,0.161415,24.536764,0.204209,398.481571,3.488373,25.835484,387.476788,2.989032,51.019713,188.958160,107.268199,166.975384,106.063181,9.193548,10.000000,62,0,0,0,1.528402,POINT (-2660021.155 2462403.921),RGI60-01.03377,2.900824,2.251284,1.090762,1.897946,1.472966,0.713662
1,RGI60-01.03622,0.009973,0.004285,0.003005,0.001419,LeConte Glacier,448.145,0,2840,1485,12.0,187,40046,0,0,0,1,9,9,0.000000,Alaska,319.820312,NaN,-2.736726e+06,2.494982e+06,222.317769,45.444676,222.317769,44.250725,22.712290,2.528288,16.601037,4.055971,82.050444,21.638538,5.848708,45.433898,13.031443,13.692506,129.432245,50.140419,120.579430,45.066576,4.488372,7.953488,43,0,0,0,0.845237,POINT (-2736725.856 2494982.045),RGI60-01.03622,0.328966,0.214494,0.150218,0.389200,0.253768,0.177723
2,RGI60-01.03890,0.003878,0.002188,0.002327,0.001487,Sawyer Glacier,196.525,260,2182,1582,10.0,312,37607,0,0,0,1,9,9,28.424366,Alaska,436.906250,NaN,-2.607257e+06,2.445398e+06,349.319501,8.380942,349.319501,97.104834,22.136463,2.849611,21.950682,2.302537,23.295658,1.929054,33.009524,17.654274,1.524858,176.997354,185.232175,106.869787,151.745712,96.951869,9.690476,10.000000,42,0,0,0,0.925830,POINT (-2607256.894 2445398.039),RGI60-01.03890,1.124399,0.443370,0.400488,1.214476,0.478889,0.432572
3,RGI60-01.04375,0.173264,0.007527,0.185125,0.002735,South Sawyer Glacier,612.706,216,2483,1567,9.0,290,53428,0,0,0,1,9,9,64.859375,Alaska,509.242188,NaN,-2.619929e+06,2.448831e+06,349.007513,613.868023,349.007513,80.694522,24.833879,0.159534,24.485708,0.189971,625.557546,7.498408,27.339394,787.762053,10.100134,45.092593,177.493215,86.277518,151.654772,80.935029,9.969697,10.000000,66,0,0,0,1.627546,POINT (-2619928.724 2448831.425),RGI60-01.04375,5.211953,3.317564,1.989613,3.202338,2.038384,1.222462
4,RGI60-01.09426,0.002825,0.001844,0.004020,0.002051,Blackstone Glacier,30.587,14,1598,997,10.0,27,12477,0,0,0,1,9,9,20.015625,Alaska,174.953125,NaN,-1.691445e+06,2.783601e+06,126.267879,30.354381,126.267879,48.008113,22.702684,1.259695,22.709179,0.430311,23.932095,2.043788,50.265385,35.361098,1.503883,107.485043,105.775335,67.139429,99.058744,49.280260,8.115385,10.000000,52,0,0,0,1.180708,POINT (-1691445.223 2783601.187),RGI60-01.09426,0.380828,0.378260,0.358924,0.322542,0.320367,0.303990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1491,RGI60-09.01065,0.143604,0.155157,0.126887,0.071025,Vostok-4 Ice Cap (Eva-Liv Island),36.825,0,335,182,22.6,37,5956,0,0,0,1,9,9,20.909081,FJL,309.039062,NaN,8.101309e+05,-4.124152e+05,150.503594,17.780832,150.503594,51.380876,18.195376

In [47]:
Arctic_discharge_fluxes = gpd.read_file('Arctic_discharge_fluxes_with_area_below_fluxgate_'+date_string+'.gpkg', driver="GPKG")
#attach melt data from David Rounce
melt_below_gate = pd.read_csv('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Rounce_terminus_melt/ERA5_2000_2019_calving_mbclim_term_mwea.csv', converters={'RGIId': str})

#add 0s where they were truncated
for i in range(0,len(melt_below_gate)):
    if len(melt_below_gate['RGIId'][i])==6:
        melt_below_gate['RGIId'][i] = melt_below_gate['RGIId'][i]+'0'
    if len(melt_below_gate['RGIId'][i])==5:
        melt_below_gate['RGIId'][i] = melt_below_gate['RGIId'][i]+'00'

Arctic_discharge_fluxes['RGIid_original'] = Arctic_discharge_fluxes['RGIid_2_1'].str.slice(7,14).astype(object)
melt_below_gate['RGIId']=melt_below_gate['RGIId'].astype(object)

Arctic_discharge_fluxes = pd.merge(Arctic_discharge_fluxes,melt_below_gate,left_on='RGIid_original',right_on='RGIId',how='left',)

Arctic_discharge_fluxes['Melt_fluxes_from_regional_estimate'] = np.nan
counter = 0
for i in range(0,len(Arctic_discharge_fluxes['region'])):
    if np.isnan(Arctic_discharge_fluxes['Melt_early_avg'][i]):
        counter=counter+1
        Arctic_discharge_fluxes['Melt_early_avg'][i] = (Arctic_discharge_fluxes['Melt_early_avg'][Arctic_discharge_fluxes['region']==Arctic_discharge_fluxes['region'][i]].mean())
        Arctic_discharge_fluxes['Melt_early_avg_err'][i] = (Arctic_discharge_fluxes['Melt_early_avg_err'][Arctic_discharge_fluxes['region']==Arctic_discharge_fluxes['region'][i]].mean())
        Arctic_discharge_fluxes['Melt_late_avg'][i] = (Arctic_discharge_fluxes['Melt_late_avg'][Arctic_discharge_fluxes['region']==Arctic_discharge_fluxes['region'][i]].mean())
        Arctic_discharge_fluxes['Melt_late_avg_err'][i] = (Arctic_discharge_fluxes['Melt_late_avg_err'][Arctic_discharge_fluxes['region']==Arctic_discharge_fluxes['region'][i]].mean())
        Arctic_discharge_fluxes['Melt_fluxes_from_regional_estimate'][i] = 1
        print(Arctic_discharge_fluxes['RGIId_x'][i])

Arctic_discharge_fluxes['Melt_total_avg']=(Arctic_discharge_fluxes['Melt_early_avg']+Arctic_discharge_fluxes['Melt_late_avg'])/2
Arctic_discharge_fluxes['Melt_total_avg_err']=(Arctic_discharge_fluxes['Melt_early_avg_err']+Arctic_discharge_fluxes['Melt_late_avg_err'])/2

print('total numbers of glaciers that use regional numbers: ', counter)
Arctic_discharge_fluxes = Arctic_discharge_fluxes.rename({'RGIId_x': 'RGIId'}, axis=1)

RGI60-05.06401
RGI60-05.07646
RGI60-05.12738
RGI60-07.00072
RGI60-07.01576
RGI60-09.00796
RGI60-10.00004
total numbers of glaciers that use regional numbers:  7


/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

In [48]:
#define functions for Romain
# Define the NMAD as a robust measure of dispersion
def nmad(data: np.ndarray, nfact: float = 1.4826) -> float:
    """
    Calculate the normalized median absolute deviation (NMAD) of an array.

    :param data: input data
    :param nfact: normalization factor for the data; default is 1.4826

    :returns nmad: (normalized) median absolute deviation of data.
    """
    data_arr = np.asarray(data)
    return nfact * np.nanmedian(np.abs(data_arr - np.nanmedian(data_arr)))
# Function for a sum of spherical variogram models
def sum_vgm(h, params, error1, error2):
    """Sum of variograms models"""
    v = 0
    for k in range(len(params)):
        v += skgstat.models.spherical(h, params[k][0], params[k][1] * error1 * error2)

    return v

# Function to integrate the variogram from a spatial support to another: a double sum of covariances
def double_sum_covar(coords: np.ndarray, areas: np.ndarray, errors: np.ndarray,
                     vgm_func: Callable):
    """
    Double sum of covariance for euclidean coordinates
    :param coords: Spatial support (typically, pixel) coordinates
    :param areas:  Area of supports
    :param errors: Standard errors of supports
    :param vgm_func: Variogram function
    :return:
    """
    n = len(coords)
    pds = pdist(coords)
    var = 0
    for i in range(n):
        for j in range(n):
            # For index calculation of the pairwise distance, see https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.pdist.html
            if i == j:
                d = 0
            elif i < j:
                ind = n * i + j - ((i + 2) * (i + 1)) // 2
                d = pds[ind]
            else:
                ind = n * j + i - ((j + 2) * (j + 1)) // 2
                d = pds[ind]

            var += errors[i] * errors[j] * (1 - vgm_func(d)) * areas[i] * areas[j]

    total_area = sum(areas)
    se_dsc = np.sqrt(var / total_area ** 2)

    return se_dsc

# Standardized variogram function (percentage of correlation)
def vgm_func_gla(h):
    v = sum_vgm(h, params=params_icethickness, error1=1, error2=1)
    # Account for percentage partial sills
    return v/100


In [49]:
#load datasets
net_area_change = pd.read_pickle('net_area_change'+date_string+'.pkl')#read area change data so that you dont need to rerun code above
Arctic_discharge_fluxes_with_area = pd.merge(Arctic_discharge_fluxes,net_area_change,on='RGIId',how='left')#combine datasets
#debias millan ice thickness
for period in ['early','late','total']:
    print('Running for period: '+period)
    # 0/ Perform the bias correction for ice thickness
    print('Correcting ice thickness bias per ice thickness range...')

    for area_loc in ['gained', 'lost']:
        print('----',area_loc)
        tomo = Arctic_discharge_fluxes_with_area['average_tomo_thickness_'+area_loc+'_'+period].values
        andrey = Arctic_discharge_fluxes_with_area['average_RUS_andrey_thickness_'+area_loc+'_'+period].values
        julian = Arctic_discharge_fluxes_with_area['average_RUS_julian_thickness_'+area_loc+'_'+period].values
        glathida = Arctic_discharge_fluxes_with_area['average_GlaThiDa_thickness_'+area_loc+'_'+period].values
        best_meas_icethickness = []
        for i in range(len(Arctic_discharge_fluxes_with_area)):
            if np.isfinite(tomo[i]):
                best_meas_icethickness.append(tomo[i])
            elif np.isfinite(andrey[i]):
                best_meas_icethickness.append(andrey[i])
            elif np.isfinite(julian[i]):
                best_meas_icethickness.append(julian[i])
            elif np.isfinite(glathida[i]):
                best_meas_icethickness.append(glathida[i])
            else:
                best_meas_icethickness.append(np.nan)
        best_meas_icethickness = np.array(best_meas_icethickness)
        
        Arctic_discharge_fluxes_with_area.loc[Arctic_discharge_fluxes_with_area['average_Milan_thickness_'+area_loc+'_'+period]>3000,'average_Milan_thickness_'+area_loc+'_'+period]=np.nan
        Arctic_discharge_fluxes_with_area.loc[Arctic_discharge_fluxes_with_area['average_Milan_thickness_'+area_loc+'_'+period]<0,'average_Milan_thickness_'+area_loc+'_'+period]=np.nan
        
        # Modelled
        best_model_icethickness = Arctic_discharge_fluxes_with_area['average_Milan_thickness_'+area_loc+'_'+period].values

        it_diff = best_meas_icethickness - best_model_icethickness
        it_diff[np.abs(it_diff)>3000] = np.nan

        print('Median bias before:' + str(np.nanmean(it_diff)))
        print('mean values before:', np.nanmean(best_model_icethickness))
        df_bin = xdem.spatialstats.nd_binning(it_diff, list_var=[best_model_icethickness], list_var_names=['ice_thickness'], list_var_bins=20)

        df_bin = df_bin[np.isfinite(df_bin.nanmedian)]
        y = df_bin.nanmedian.values
        x = pd.IntervalIndex(df_bin.ice_thickness).mid.values
        p = np.polyfit(x[np.isfinite(y)], y[np.isfinite(y)], deg=1)

        # Derive the debiased modelled ice thickness
        Arctic_discharge_fluxes_with_area['average_Milan_thickness_'+area_loc+'_'+period+'_debiased'] = Arctic_discharge_fluxes_with_area['average_Milan_thickness_'+area_loc+'_'+period] \
                                                                         + np.polyval(p, Arctic_discharge_fluxes_with_area['average_Milan_thickness_'+area_loc+'_'+period])

        print('Median bias after:' +str(np.nanmean(best_meas_icethickness - Arctic_discharge_fluxes_with_area['average_Milan_thickness_'+area_loc+'_'+period+'_debiased'])))
        print('Mean values after:',Arctic_discharge_fluxes_with_area['average_Milan_thickness_'+area_loc+'_'+period+'_debiased'].mean())

Running for period: early
Correcting ice thickness bias per ice thickness range...
---- gained
Median bias before:-225.88118073200525
mean values before: 138.05824085615322
Median bias after:-2.685008998804561
Mean values after: 90.99589179502689
---- lost
Median bias before:-267.635270336131
mean values before: 144.3474116899036
Median bias after:5.873020505081551
Mean values after: 65.59665766114719
Running for period: late
Correcting ice thickness bias per ice thickness range...
---- gained
Median bias before:-309.86102557565806
mean values before: 141.94928442187853
Median bias after:-3.1626824164966907
Mean values after: 98.33670580844553
---- lost
Median bias before:-175.11678517730098
mean values before: 140.32577432874285
Median bias after:-1.4980413797975956
Mean values after: 92.5753210965101
Running for period: total
Correcting ice thickness bias per ice thickness range...
---- gained
Median bias before:-200.68017003334413
mean values before: 155.3181664500823
Median bias af

In [50]:
#Figure out thickness of area gain/lost

# Arctic_discharge_fluxes_with_area.loc[:,'2000_2010_mean_flux_err_gt']=Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_corr_err_gt']
# Arctic_discharge_fluxes_with_area.loc[:,'2010_2020_mean_flux_err_gt']=Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_corr_err_gt']

#Find the best thickness data for areas gained/lost by preference:
Arctic_discharge_fluxes_with_area.loc[:,'thickness_for_area_lost_early'] = Arctic_discharge_fluxes_with_area['average_tomo_thickness_lost_early']
Arctic_discharge_fluxes_with_area.loc[:,'thickness_for_area_lost_early_err'] = np.nan
Arctic_discharge_fluxes_with_area.loc[:,'thickness_for_area_gained_early'] = Arctic_discharge_fluxes_with_area['average_tomo_thickness_gained_early']
Arctic_discharge_fluxes_with_area.loc[:,'thickness_for_area_gained_early_err'] = np.nan
Arctic_discharge_fluxes_with_area.loc[:,'thickness_for_area_lost_late'] = Arctic_discharge_fluxes_with_area['average_tomo_thickness_lost_late']
Arctic_discharge_fluxes_with_area.loc[:,'thickness_for_area_lost_late_err'] = np.nan
Arctic_discharge_fluxes_with_area.loc[:,'thickness_for_area_gained_late'] = Arctic_discharge_fluxes_with_area['average_tomo_thickness_gained_late']
Arctic_discharge_fluxes_with_area.loc[:,'thickness_for_area_gained_late_err'] = np.nan

Arctic_discharge_fluxes_with_area.loc[:,'thickness_for_area_lost_total'] = Arctic_discharge_fluxes_with_area['average_tomo_thickness_lost_total']
Arctic_discharge_fluxes_with_area.loc[:,'thickness_for_area_lost_total_err'] = np.nan
Arctic_discharge_fluxes_with_area.loc[:,'thickness_for_area_gained_total'] = Arctic_discharge_fluxes_with_area['average_tomo_thickness_gained_total']
Arctic_discharge_fluxes_with_area.loc[:,'thickness_for_area_gained_total_err'] = np.nan

Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early'].fillna(Arctic_discharge_fluxes_with_area['average_RUS_andrey_thickness_lost_early'], inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early'].fillna(Arctic_discharge_fluxes_with_area['average_RUS_andrey_thickness_gained_early'], inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late'].fillna(Arctic_discharge_fluxes_with_area['average_RUS_andrey_thickness_lost_late'], inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late'].fillna(Arctic_discharge_fluxes_with_area['average_RUS_andrey_thickness_gained_late'], inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_total'].fillna(Arctic_discharge_fluxes_with_area['average_RUS_andrey_thickness_gained_total'], inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_lost_total'].fillna(Arctic_discharge_fluxes_with_area['average_RUS_andrey_thickness_lost_total'], inplace=True)

Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early'].fillna(Arctic_discharge_fluxes_with_area['average_RUS_julian_thickness_lost_early'], inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early'].fillna(Arctic_discharge_fluxes_with_area['average_RUS_julian_thickness_gained_early'], inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late'].fillna(Arctic_discharge_fluxes_with_area['average_RUS_julian_thickness_lost_late'], inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late'].fillna(Arctic_discharge_fluxes_with_area['average_RUS_julian_thickness_gained_late'], inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_total'].fillna(Arctic_discharge_fluxes_with_area['average_RUS_julian_thickness_gained_total'], inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_lost_total'].fillna(Arctic_discharge_fluxes_with_area['average_RUS_julian_thickness_lost_total'], inplace=True)

#make 10 m the error for all the data you just put in 
for i in range(0,len(Arctic_discharge_fluxes_with_area)):
    if Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early'][i] > 0:
        Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_lost_early_err'] = 10
    if Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early'][i] >0:
        Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_gained_early_err']=10
    if Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late'][i]>0:
        Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_lost_late_err']=10
    if Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late'][i]>0:
        Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_gained_late_err'] = 10
    if Arctic_discharge_fluxes_with_area['thickness_for_area_lost_total'][i]>0:
        Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_lost_total_err']=10
    if Arctic_discharge_fluxes_with_area['thickness_for_area_gained_total'][i]>0:
        Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_gained_total_err'] = 10

Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early'].fillna(Arctic_discharge_fluxes_with_area['average_GlaThiDa_thickness_lost_early'], inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early'].fillna(Arctic_discharge_fluxes_with_area['average_GlaThiDa_thickness_gained_early'], inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late'].fillna(Arctic_discharge_fluxes_with_area['average_GlaThiDa_thickness_lost_late'], inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late'].fillna(Arctic_discharge_fluxes_with_area['average_GlaThiDa_thickness_gained_late'], inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_total'].fillna(Arctic_discharge_fluxes_with_area['average_GlaThiDa_thickness_gained_total'], inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_lost_total'].fillna(Arctic_discharge_fluxes_with_area['average_GlaThiDa_thickness_lost_total'], inplace=True)

Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early_err'].fillna(Arctic_discharge_fluxes_with_area['average_GlaThiDa_thickness_lost_early_err'], inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early_err'].fillna(Arctic_discharge_fluxes_with_area['average_GlaThiDa_thickness_gained_early_err'], inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late_err'].fillna(Arctic_discharge_fluxes_with_area['average_GlaThiDa_thickness_lost_late_err'], inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late_err'].fillna(Arctic_discharge_fluxes_with_area['average_GlaThiDa_thickness_gained_late_err'], inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_total_err'].fillna(Arctic_discharge_fluxes_with_area['average_GlaThiDa_thickness_gained_total_err'], inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_lost_total_err'].fillna(Arctic_discharge_fluxes_with_area['average_GlaThiDa_thickness_lost_total_err'], inplace=True)

Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early'].fillna(Arctic_discharge_fluxes_with_area['average_Milan_thickness_lost_early_debiased'], inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early'].fillna(Arctic_discharge_fluxes_with_area['average_Milan_thickness_gained_early_debiased'], inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late'].fillna(Arctic_discharge_fluxes_with_area['average_Milan_thickness_lost_late_debiased'], inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late'].fillna(Arctic_discharge_fluxes_with_area['average_Milan_thickness_gained_late_debiased'], inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_total'].fillna(Arctic_discharge_fluxes_with_area['average_Milan_thickness_gained_total_debiased'], inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_lost_total'].fillna(Arctic_discharge_fluxes_with_area['average_Milan_thickness_lost_total_debiased'], inplace=True)

Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early_err'].fillna(Arctic_discharge_fluxes_with_area['average_Milan_thickness_lost_early_err'], inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early_err'].fillna(Arctic_discharge_fluxes_with_area['average_Milan_thickness_gained_early_err'], inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late_err'].fillna(Arctic_discharge_fluxes_with_area['average_Milan_thickness_lost_late_err'], inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late_err'].fillna(Arctic_discharge_fluxes_with_area['average_Milan_thickness_gained_late_err'], inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_total_err'].fillna(Arctic_discharge_fluxes_with_area['average_Milan_thickness_gained_total_err'], inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_lost_total_err'].fillna(Arctic_discharge_fluxes_with_area['average_Milan_thickness_lost_total_err'], inplace=True)

Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early'].fillna(Arctic_discharge_fluxes_with_area['best_thickness_early']*0.60, inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early'].fillna(Arctic_discharge_fluxes_with_area['best_thickness_early']*0.60, inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late'].fillna(Arctic_discharge_fluxes_with_area['best_thickness_late']*0.60, inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late'].fillna(Arctic_discharge_fluxes_with_area['best_thickness_late']*0.60, inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_total'].fillna((Arctic_discharge_fluxes_with_area['best_thickness_early']+Arctic_discharge_fluxes_with_area['best_thickness_late'])/2*0.60, inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_lost_total'].fillna((Arctic_discharge_fluxes_with_area['best_thickness_early']+Arctic_discharge_fluxes_with_area['best_thickness_late'])/2*0.60, inplace=True)

Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early_err'].fillna(Arctic_discharge_fluxes_with_area['best_thickness_early']*0.30, inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early_err'].fillna(Arctic_discharge_fluxes_with_area['best_thickness_early']*0.30, inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late_err'].fillna(Arctic_discharge_fluxes_with_area['best_thickness_late']*0.30, inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late_err'].fillna(Arctic_discharge_fluxes_with_area['best_thickness_late']*0.30, inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_total_err'].fillna((Arctic_discharge_fluxes_with_area['best_thickness_early']+Arctic_discharge_fluxes_with_area['best_thickness_late'])/2*0.30, inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_lost_total_err'].fillna((Arctic_discharge_fluxes_with_area['best_thickness_early']+Arctic_discharge_fluxes_with_area['best_thickness_late'])/2*0.30, inplace=True)

Arctic_discharge_fluxes_with_area.loc[Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early_err']<=0,'thickness_for_area_lost_early_err']=Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early']*0.5
Arctic_discharge_fluxes_with_area.loc[Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late_err']<=0,'thickness_for_area_lost_late_err']=Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late']*0.5
Arctic_discharge_fluxes_with_area.loc[Arctic_discharge_fluxes_with_area['thickness_for_area_lost_total_err']<=0,'thickness_for_area_lost_total_err']=Arctic_discharge_fluxes_with_area['thickness_for_area_lost_total']*0.5
Arctic_discharge_fluxes_with_area.loc[Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early_err']<=0,'thickness_for_area_gained_early_err']=Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early']*0.5
Arctic_discharge_fluxes_with_area.loc[Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late_err']<=0,'thickness_for_area_gained_late_err']=Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late']*0.5
Arctic_discharge_fluxes_with_area.loc[Arctic_discharge_fluxes_with_area['thickness_for_area_gained_total_err']<=0,'thickness_for_area_gained_total_err']=Arctic_discharge_fluxes_with_area['thickness_for_area_gained_total']*0.5

#We are having trouble with the thickness of Parish Glacier in Arctic Canada, so set the thickness to the best_thickness and this will fix it
Arctic_discharge_fluxes_with_area.loc[Arctic_discharge_fluxes_with_area['RGIId']=='RGI60-03.00191','thickness_for_area_gained_early']=Arctic_discharge_fluxes_with_area['best_thickness_early']*0.60
Arctic_discharge_fluxes_with_area.loc[Arctic_discharge_fluxes_with_area['RGIId']=='RGI60-03.00191','thickness_for_area_gained_early_err']=Arctic_discharge_fluxes_with_area['best_thickness_early_err']*0.30

bad_thickness_glaciers=0
for i in range(0, len(Arctic_discharge_fluxes_with_area)):
    for period in ['early','late']:
        for var in ['thickness_for_area_lost_'+period,'thickness_for_area_gained_'+period]:
            if (Arctic_discharge_fluxes_with_area[var][i]<10):
                Arctic_discharge_fluxes_with_area.loc[i,var]=Arctic_discharge_fluxes_with_area['best_thickness_'+period][i]*0.60
                Arctic_discharge_fluxes_with_area.loc[i,var+'_err']=Arctic_discharge_fluxes_with_area['best_thickness_'+period][i]*0.30
                bad_thickness_glaciers = bad_thickness_glaciers+1
            if Arctic_discharge_fluxes_with_area[var][i]>2000:
                print(Arctic_discharge_fluxes_with_area['Name_x'][i],' has a very large thickness for terminus change',var,Arctic_discharge_fluxes_with_area[var][i])
                Arctic_discharge_fluxes_with_area.loc[i,var]=Arctic_discharge_fluxes_with_area['best_thickness_mean'][i]*0.60
                Arctic_discharge_fluxes_with_area.loc[i,var+'_err']=Arctic_discharge_fluxes_with_area['best_thickness_mean'][i]*0.30
        if (Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early'][i]+1>Arctic_discharge_fluxes_with_area['best_thickness_early'][i])&(Arctic_discharge_fluxes_with_area['net_area_change_early'][i]>0):
            Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_gained_early']=Arctic_discharge_fluxes_with_area['best_thickness_early'][i]*0.60
            Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_gained_early'+'_err']=Arctic_discharge_fluxes_with_area['best_thickness_early'][i]*0.30
            print(Arctic_discharge_fluxes_with_area['Name_x'][i],Arctic_discharge_fluxes_with_area['best_thickness_mean'][i],Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early'][i])
            bad_thickness_glaciers = bad_thickness_glaciers+1
        if (Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late'][i]+1>Arctic_discharge_fluxes_with_area['best_thickness_late'][i])&(Arctic_discharge_fluxes_with_area['net_area_change_late'][i]>0):
            Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_gained_late']=Arctic_discharge_fluxes_with_area['best_thickness_late'][i]*0.60
            Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_gained_late'+'_err']=Arctic_discharge_fluxes_with_area['best_thickness_late'][i]*0.30
            bad_thickness_glaciers = bad_thickness_glaciers+1

print('start')
for i in range(0, len(Arctic_discharge_fluxes_with_area)): 
    for var in ['thickness_for_area_lost_total','thickness_for_area_gained_total']:
#         print(i)
        if Arctic_discharge_fluxes_with_area[var][i]>2000:
            print(Arctic_discharge_fluxes_with_area['Name_x'][i],' has a very large thickness for TOTAL terminus change',var,Arctic_discharge_fluxes_with_area[var][i])
            Arctic_discharge_fluxes_with_area.loc[i,var]=(Arctic_discharge_fluxes_with_area['best_thickness_early_mean'][i]+Arctic_discharge_fluxes_with_area['best_thickness_late_mean'][i])/2*0.60
            Arctic_discharge_fluxes_with_area.loc[i,var+'_err']=(Arctic_discharge_fluxes_with_area['best_thickness_early_mean'][i]+Arctic_discharge_fluxes_with_area['best_thickness_late_mean'][i])/2*0.30
        if Arctic_discharge_fluxes_with_area[var][i]<0:
            print(Arctic_discharge_fluxes_with_area['Name_x'][i],' has a negative thickness:',Arctic_discharge_fluxes_with_area[var][i])
            Arctic_discharge_fluxes_with_area.loc[i,var]=(Arctic_discharge_fluxes_with_area['best_thickness_early_mean'][i]+Arctic_discharge_fluxes_with_area['best_thickness_late_mean'][i])/2*0.60
            Arctic_discharge_fluxes_with_area.loc[i,var+'_err']=(Arctic_discharge_fluxes_with_area['best_thickness_early_mean'][i]+Arctic_discharge_fluxes_with_area['best_thickness_late_mean'][i])/2*0.30
print('glaciers that have their thickness replaced with 60±30% at the terminus:', bad_thickness_glaciers)

#If the thickness err is greater than the signal, make the error the signal for the terminus volume change calculations
for i in range(0,len(Arctic_discharge_fluxes_with_area)):
    if Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early_err'][i]>Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early'][i]:
        Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_lost_early_err']=Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early'][i]
    if Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early_err'][i]>Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early'][i]:
        Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_gained_early_err']=Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early'][i]
    if Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late_err'][i]>Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late'][i]:
        Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_lost_late_err']=Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late'][i]
    if Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late_err'][i]>Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late'][i]:
        Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_gained_late_err']=Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late'][i]
    if Arctic_discharge_fluxes_with_area['thickness_for_area_lost_total_err'][i]>Arctic_discharge_fluxes_with_area['thickness_for_area_lost_total'][i]:
        Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_lost_total_err']=Arctic_discharge_fluxes_with_area['thickness_for_area_lost_total'][i]
    if Arctic_discharge_fluxes_with_area['thickness_for_area_gained_total_err'][i]>Arctic_discharge_fluxes_with_area['thickness_for_area_gained_total'][i]:
        Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_gained_total_err']=Arctic_discharge_fluxes_with_area['thickness_for_area_gained_total'][i]

#Use the fluxgate mean thickness instead of Farinotti results if we have observations at the fluxgate
for i in range(0,len(Arctic_discharge_fluxes_with_area)):
    if (Arctic_discharge_fluxes_with_area['do_we_have_an_observation_in_middle_fifth'][i] ==1)&\
    (math.isnan(Arctic_discharge_fluxes_with_area['average_tomo_thickness_lost_early'][i]))&\
    (math.isnan(Arctic_discharge_fluxes_with_area['average_GlaThiDa_thickness_lost_early'][i]))&\
    (math.isnan(Arctic_discharge_fluxes_with_area['average_RUS_andrey_thickness_lost_early'][i]))&\
    (math.isnan(Arctic_discharge_fluxes_with_area['average_RUS_julian_thickness_lost_early'][i])):
        Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_lost_early']=Arctic_discharge_fluxes_with_area['best_thickness_early'][i]*0.6
        Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_lost_early_err']=Arctic_discharge_fluxes_with_area['best_thickness_early'][i]*0.3
    if (Arctic_discharge_fluxes_with_area['do_we_have_an_observation_in_middle_fifth'][i] ==1)&\
    (math.isnan(Arctic_discharge_fluxes_with_area['average_tomo_thickness_gained_early'][i]))&\
    (math.isnan(Arctic_discharge_fluxes_with_area['average_GlaThiDa_thickness_gained_early'][i]))&\
    (math.isnan(Arctic_discharge_fluxes_with_area['average_RUS_andrey_thickness_gained_early'][i]))&\
    (math.isnan(Arctic_discharge_fluxes_with_area['average_RUS_julian_thickness_gained_early'][i])):
        Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_gained_early']=Arctic_discharge_fluxes_with_area['best_thickness_early'][i]*0.6
        Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_gained_early_err']=Arctic_discharge_fluxes_with_area['best_thickness_early'][i]*0.3
    if (Arctic_discharge_fluxes_with_area['do_we_have_an_observation_in_middle_fifth'][i] ==1)&\
    (math.isnan(Arctic_discharge_fluxes_with_area['average_tomo_thickness_lost_late'][i]))&\
    (math.isnan(Arctic_discharge_fluxes_with_area['average_GlaThiDa_thickness_lost_late'][i]))&\
    (math.isnan(Arctic_discharge_fluxes_with_area['average_RUS_andrey_thickness_lost_late'][i]))&\
    (math.isnan(Arctic_discharge_fluxes_with_area['average_RUS_julian_thickness_lost_late'][i])):
        Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_lost_late']=Arctic_discharge_fluxes_with_area['best_thickness_late'][i]*0.6
        Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_lost_late_err']=Arctic_discharge_fluxes_with_area['best_thickness_late'][i]*0.3
    if (Arctic_discharge_fluxes_with_area['do_we_have_an_observation_in_middle_fifth'][i] ==1)&\
    (math.isnan(Arctic_discharge_fluxes_with_area['average_tomo_thickness_gained_late'][i]))&\
    (math.isnan(Arctic_discharge_fluxes_with_area['average_GlaThiDa_thickness_gained_late'][i]))&\
    (math.isnan(Arctic_discharge_fluxes_with_area['average_RUS_andrey_thickness_gained_late'][i]))&\
    (math.isnan(Arctic_discharge_fluxes_with_area['average_RUS_julian_thickness_gained_late'][i])):
        Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_gained_late']=Arctic_discharge_fluxes_with_area['best_thickness_late'][i]*0.6
        Arctic_discharge_fluxes_with_area.loc[i,'thickness_for_area_gained_late_err']=Arctic_discharge_fluxes_with_area['best_thickness_late'][i]*0.3

Yahtse Glacier 247.71682201279592 125.64804720343194
None 30.6328125 48.05141318071955
None 66.14436229792508 54.10553531842593
None 30.005544354838708 48.85809429065379
None 39.77660892486572 50.89807590964387
None 35.674383425712584 48.98962937208013
None 30.39814814814815 46.71690397255606
None 30.0 47.01025370231723
None 30.356770833333332 46.64089342235448
None 42.28196022727273 50.61349338895192
Austfonna Duvebreen 77.30375346260388 43.73034169001231
Munthbreen 35.81022071838379 51.50435166564504
Jarlbreen 58.49248231132076 52.499248415641866
Fuhrmeisterbreen 55.29918981481482 52.62953290118377
Buchananisen N 50.0213373655914 51.79416688786426
Bakaninbreen, Paulabreen, and Skobreen 69.03878678733776 41.90308653080628
Moscow Ice Cap (Hall Island) 37.55058530020704 21.469156252069
start
glaciers that have their thickness replaced with 60±30% at the terminus: 74


In [51]:
#calculate the spatial correlation of the terminus change volume error from Romain
for period in ['early','late','total']:
    print('Running for period: '+period)
    # 1/ Derive uncertainty for pixel -> glacier based on the area of averaging (lost or gained)
    # Get variables of interest: ice thickness

    if period == 'early':
        dt = Arctic_discharge_fluxes_with_area['early_to_middle_years_elapsed'].values
    else:
        dt = Arctic_discharge_fluxes_with_area['middle_to_late_years_elapsed'].values

    # Lost
    meanHl = Arctic_discharge_fluxes_with_area['thickness_for_area_lost_' + period].values
    meanHl_err_fullycorr = Arctic_discharge_fluxes_with_area['thickness_for_area_lost_' + period + '_err'].values
    dSldt = Arctic_discharge_fluxes_with_area['area_km2_lost_' + period].values / dt
    
    # Remove aberrantly large uncertainty values for ice thickness uncertainty
    meanHl_err_fullycorr[np.abs(meanHl_err_fullycorr - np.nanmedian(meanHl_err_fullycorr)) > 5 * nmad(meanHl_err_fullycorr)] = np.nan
    # Replace no data that might exist in the uncertainty from the average uncertainty
    meanHl_err_fullycorr[~np.isfinite(meanHl_err_fullycorr)] = np.nanmean(meanHl_err_fullycorr)

    # Gained
    meanHg = Arctic_discharge_fluxes_with_area['thickness_for_area_gained_' + period].values
    meanHg_err_fullycorr = Arctic_discharge_fluxes_with_area['thickness_for_area_gained_' + period + '_err'].values
    dSgdt = Arctic_discharge_fluxes_with_area['area_km2_gained_' + period].values / dt

    # Remove aberrantly large uncertainty values for ice thickness uncertainty
    meanHg_err_fullycorr[np.abs(meanHg_err_fullycorr - np.nanmedian(meanHg_err_fullycorr)) > 5 * nmad(meanHg_err_fullycorr)] = np.nan
    # Replace no data that might exist in the uncertainty from the average uncertainty
    meanHg_err_fullycorr[~np.isfinite(meanHg_err_fullycorr)] = np.nanmean(meanHg_err_fullycorr)

    # Parameters estimated for the ice thickness variogram (correlation range, partial sill):
    # range in meters, partial sill is in percentage of the variance
    params_icethickness = ((2100, 28), (47000, 50),
                           (1354000, 22))
    
    # Looping for every glacier
    unique_rgiid = list(np.unique(Arctic_discharge_fluxes_with_area['RGIId'].values))

    list_Hl_err_corr, list_Hg_err_corr = ([] for i in range(2))
    for rgiid in unique_rgiid:

        # Select only a given RGIId = glacier
        index_glacier = (Arctic_discharge_fluxes_with_area['RGIId'] == rgiid)

#         print('Working on glacier: '+rgiid + ' with '+ str(np.count_nonzero(index_glacier))+ ' flux gates...')

        # Variograms parameters formatted for xdem
        list_vgm=[(params_icethickness[0][0], 'Sph', params_icethickness[0][1]),
                  (params_icethickness[1][0], 'Sph', params_icethickness[1][1]),
                  (params_icethickness[2][0], 'Sph', params_icethickness[2][1])]

        # a/ Terminus lost
        # Number of effective samples in the area from the variogram
        dSl = dSldt[index_glacier] * dt[index_glacier]
        neff_l = xdem.spatialstats.neff_circ(area=dSl*10**6, list_vgm=list_vgm)

        # Uncertainty value for the average ice thickness of the glacier considering spatial correlation
        if dSl != 0:
            meanHl_err_corr = meanHl_err_fullycorr[index_glacier][0] / np.sqrt(neff_l)[0]
        else:
            meanHl_err_corr = 0

#         print('For terminus lost:')
#         print('Replacing fully correlated averaged ice thickness error {:.1f} m into spatially correlated {:.1f} m'.format(meanHl_err_fullycorr[index_glacier][0], meanHl_err_corr))
        list_Hl_err_corr.append(meanHl_err_corr)

        # b/ Terminus gained
        # Number of effective samples in the area from the variogram
        dSg = dSgdt[index_glacier] * dt[index_glacier]
        neff_g = xdem.spatialstats.neff_circ(area=dSg*10**6, list_vgm=list_vgm)

        # Uncertainty value for the average ice thickness of the glacier considering spatial correlation
        if dSg != 0:
            meanHg_err_corr = meanHg_err_fullycorr[index_glacier][0] / np.sqrt(neff_g)[0]
        else:
            meanHg_err_corr = 0

#         print('For terminus gained:')
#         print('Replacing fully correlated averaged ice thickness error {:.1f} m into spatially correlated {:.1f} m'.format(meanHg_err_fullycorr[index_glacier][0],meanHg_err_corr))
        list_Hg_err_corr.append(meanHg_err_corr)

    # Save the old columns of error into a fullycorr column
    Arctic_discharge_fluxes_with_area['thickness_for_area_lost_' + period + '_err_fullycorr'] = Arctic_discharge_fluxes_with_area['thickness_for_area_lost_' + period + '_err'].values
    Arctic_discharge_fluxes_with_area['thickness_for_area_gained_' + period + '_err_fullycorr'] = Arctic_discharge_fluxes_with_area['thickness_for_area_gained_' + period + '_err'].values
    # Replace the normal error column with the new values
    Arctic_discharge_fluxes_with_area['thickness_for_area_lost_' + period + '_err'] = list_Hl_err_corr
    Arctic_discharge_fluxes_with_area['thickness_for_area_gained_' + period + '_err'] = list_Hg_err_corr

    print('---------------------',period,len(Arctic_discharge_fluxes_with_area[(Arctic_discharge_fluxes_with_area['thickness_for_area_lost_' + period + '_err']<=0)&(Arctic_discharge_fluxes_with_area['thickness_for_area_lost_' + period]>0)]))
    print('---------------------',period,len(Arctic_discharge_fluxes_with_area[(Arctic_discharge_fluxes_with_area['thickness_for_area_gained_' + period + '_err']<=0)&(Arctic_discharge_fluxes_with_area['thickness_for_area_gained_' + period]>0)]))
    Arctic_discharge_fluxes_with_area.loc[(Arctic_discharge_fluxes_with_area['thickness_for_area_lost_' + period + '_err']<=0)&(Arctic_discharge_fluxes_with_area['thickness_for_area_lost_' + period]>0),'thickness_for_area_lost_' + period + '_err'] = Arctic_discharge_fluxes_with_area['thickness_for_area_lost_' + period]*0.5
    Arctic_discharge_fluxes_with_area.loc[(Arctic_discharge_fluxes_with_area['thickness_for_area_gained_' + period + '_err']<=0)&(Arctic_discharge_fluxes_with_area['thickness_for_area_gained_' + period]>0),'thickness_for_area_gained_' + period + '_err'] = Arctic_discharge_fluxes_with_area['thickness_for_area_gained_' + period]*0.5
    print('---------------------',period,len(Arctic_discharge_fluxes_with_area[(Arctic_discharge_fluxes_with_area['thickness_for_area_lost_' + period + '_err']<=0)&(Arctic_discharge_fluxes_with_area['thickness_for_area_lost_' + period]>0)]))
    print('---------------------',period,len(Arctic_discharge_fluxes_with_area[(Arctic_discharge_fluxes_with_area['thickness_for_area_gained_' + period + '_err']<=0)&(Arctic_discharge_fluxes_with_area['thickness_for_area_gained_' + period]>0)]))
    
    # Load under the names previously used
    meanHl_err = Arctic_discharge_fluxes_with_area['thickness_for_area_lost_' + period + '_err'].values
    meanHg_err = Arctic_discharge_fluxes_with_area['thickness_for_area_gained_' + period + '_err'].values

    # 2/ Propagation glacier -> region

    list_regions = list(np.unique(Arctic_discharge_fluxes_with_area.region.values))

    list_df_reg = []
    for region in list_regions:

        print('Working on region: '+region)

        # Isolate region of interest
        ind_reg = Arctic_discharge_fluxes_with_area.region.values == region

        # Terminus ice lost
        Tl_reg = np.sum(meanHl[ind_reg] * dSldt[ind_reg]) * 0.9 / 1000
        mean_meanHl_reg = np.nanmean(meanHl[ind_reg])
        mean_dSldt_reg = np.nanmean(dSldt[ind_reg])
        mean_meanHl_err_reg = np.nanmean(meanHl_err[ind_reg])

        # Terminus ice gained
        Tg_reg = np.sum(meanHg_err[ind_reg] * dSgdt[ind_reg]) * 0.9 / 1000
        mean_meanHg_reg = np.nanmean(meanHg[ind_reg])
        mean_dSgdt_reg = np.nanmean(dSgdt[ind_reg])
        mean_meanHg_err_reg = np.nanmean(meanHg_err[ind_reg])

        # Separating two sources of terminus ice loss: one based on area uncertainty, one based on ice thickness uncertainty
        # Here we only compute only the ice thickness source (second term of Equation 6 in the paper)

        # a/ Terminus mass lost
        # Considering all ice thicknesses uncertainties fully independent
        Tl_errH_uncorr_reg = np.sqrt(np.nansum((meanHl_err[ind_reg] * dSldt[ind_reg]) ** 2)) * 0.9 / 1000

        # Uncertainty on the sum, considering everything correlated
        Tl_errH_fulluycorr_reg = np.nansum(meanHl_err[ind_reg] * dSldt[ind_reg]) * 0.9 / 1000

        # Uncertainty on the sum, accounting for spatial correlations in velocities and ice thickness
        coords = np.array((Arctic_discharge_fluxes_with_area['easting'].values[ind_reg], Arctic_discharge_fluxes_with_area['northing'].values[ind_reg])).T
        Tl_errH_corr_reg = double_sum_covar(coords=coords, areas=dSldt[ind_reg], errors=meanHl_err[ind_reg],
                                          vgm_func=vgm_func_gla) * sum(dSldt[ind_reg]) * 0.9 / 1000

        # b/ Terminus mass gained
        # Considering all ice thicknesses uncertainties fully independent
        Tg_errH_uncorr_reg = np.sqrt(np.nansum((meanHg_err[ind_reg] * dSgdt[ind_reg]) ** 2)) * 0.9 / 1000

        # Uncertainty on the sum, considering everything correlated
        Tg_errH_fulluycorr_reg = np.nansum(meanHg_err[ind_reg] * dSgdt[ind_reg]) * 0.9 / 1000

        # Uncertainty on the sum, accounting for spatial correlations in velocities and ice thickness
        coords = np.array((Arctic_discharge_fluxes_with_area['easting'].values[ind_reg], Arctic_discharge_fluxes_with_area['northing'].values[ind_reg])).T
        Tg_errH_corr_reg = double_sum_covar(coords=coords, areas=dSgdt[ind_reg], errors=meanHg_err[ind_reg],
                                          vgm_func=vgm_func_gla) * sum(dSgdt[ind_reg]) * 0.9 / 1000

        # Store results in a dataframe
        df_res_reg = pd.DataFrame()
        df_res_reg = df_res_reg.assign(region=[region], Tl=[Tl_reg], Tg=[Tg_reg],
                               Tl_errH_uncorr=[Tl_errH_uncorr_reg], Tl_errH_fullycor=[Tl_errH_fulluycorr_reg],
                               Tl_errH_corr=[Tl_errH_corr_reg],
                               Tg_errH_uncorr=[Tg_errH_uncorr_reg], Tg_errH_fullycor=[Tg_errH_fulluycorr_reg],
                               Tg_errH_corr=[Tg_errH_corr_reg],
                               mean_Hl=[mean_meanHl_reg], mean_meanHl_err=[mean_meanHl_err_reg] ,mean_dSl=[mean_dSldt_reg],
                               mean_Hg=[mean_meanHg_reg], mean_meanHg_err=[mean_meanHg_err_reg] ,mean_dSg=[mean_dSgdt_reg])
        list_df_reg.append(df_res_reg)

    if period=='early':
        df_all_reg_early = pd.concat(list_df_reg)
        df_all_reg_early.to_pickle('df_all_reg_early_TC_'+date_string+'.pkl')
    if period=='late':
        df_all_reg_late = pd.concat(list_df_reg)
        df_all_reg_late.to_pickle('df_all_reg_late_TC_'+date_string+'.pkl')
#     df_all_reg.to_csv('/home/atom/ongoing/work_frontal_ablation/df_terminus_region.csv')

Running for period: early


/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/xdem/spatialstats.py:884: RuntimeWarning: invalid value encountered in true_divide
  std_err = np.sqrt(2*np.pi*full_int / area)


--------------------- early 58
--------------------- early 1165
--------------------- early 0
--------------------- early 0
Working on region: Alaska
Working on region: BBI
Working on region: FJL
Working on region: Greenland
Working on region: Iceland
Working on region: NZ
Working on region: QEI
Working on region: SZ
Working on region: Svalbard and Jan Mayen
Running for period: late


/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/xdem/spatialstats.py:884: RuntimeWarning: invalid value encountered in true_divide
  std_err = np.sqrt(2*np.pi*full_int / area)


--------------------- late 99
--------------------- late 1206
--------------------- late 0
--------------------- late 0
Working on region: Alaska
Working on region: BBI
Working on region: FJL
Working on region: Greenland
Working on region: Iceland
Working on region: NZ
Working on region: QEI


/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:52: RuntimeWarning: invalid value encountered in double_scalars


Working on region: SZ
Working on region: Svalbard and Jan Mayen
Running for period: total
--------------------- total 28
--------------------- total 1331
--------------------- total 0
--------------------- total 0
Working on region: Alaska
Working on region: BBI
Working on region: FJL
Working on region: Greenland
Working on region: Iceland
Working on region: NZ
Working on region: QEI
Working on region: SZ
Working on region: Svalbard and Jan Mayen


In [52]:
#Merge area change data for every glacier with the discharge estimates
#calculate discharge due to terminus retreat/advance with uncertainties (change thickness uncertainty above)
Arctic_discharge_fluxes_with_area.loc[:,'area_flux_gt_gained_early'] = Arctic_discharge_fluxes_with_area['area_km2_gained_early']*Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early']/1000*0.900
Arctic_discharge_fluxes_with_area.loc[:,'area_flux_gt_gained_early_error'] = \
    np.sqrt((Arctic_discharge_fluxes_with_area['perimeter_error_gained_early']*Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early']/1000*0.9)**2+
    (Arctic_discharge_fluxes_with_area['area_km2_gained_early']*Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early_err']/1000*0.9)**2)

Arctic_discharge_fluxes_with_area.loc[:,'area_flux_gt_lost_early'] = Arctic_discharge_fluxes_with_area['area_km2_lost_early']*Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early']/1000*0.900
Arctic_discharge_fluxes_with_area.loc[:,'area_flux_gt_lost_early_error'] = \
    np.sqrt((Arctic_discharge_fluxes_with_area['perimeter_error_lost_early']*Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early']/1000*0.9)**2+
    (Arctic_discharge_fluxes_with_area['area_km2_lost_early']*Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early_err']/1000*0.9)**2)

Arctic_discharge_fluxes_with_area.loc[:,'area_flux_gt_gained_late'] = Arctic_discharge_fluxes_with_area['area_km2_gained_late']*Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late']/1000*0.900
Arctic_discharge_fluxes_with_area.loc[:,'area_flux_gt_gained_late_error'] = \
    np.sqrt((Arctic_discharge_fluxes_with_area['perimeter_error_gained_late']*Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late']/1000*0.9)**2+
    (Arctic_discharge_fluxes_with_area['area_km2_gained_late']*Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late_err']/1000*0.9)**2)

Arctic_discharge_fluxes_with_area.loc[:,'area_flux_gt_lost_late'] = Arctic_discharge_fluxes_with_area['area_km2_lost_late']*Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late']/1000*0.900
Arctic_discharge_fluxes_with_area.loc[:,'area_flux_gt_lost_late_error'] = \
    np.sqrt((Arctic_discharge_fluxes_with_area['perimeter_error_lost_late']*Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late']/1000*0.9)**2+
    (Arctic_discharge_fluxes_with_area['area_km2_lost_late']*Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late_err']/1000*0.9)**2)

Arctic_discharge_fluxes_with_area.loc[:,'area_flux_gt_gained_total'] = Arctic_discharge_fluxes_with_area['area_km2_gained_total']*Arctic_discharge_fluxes_with_area['thickness_for_area_gained_total']/1000*0.900
Arctic_discharge_fluxes_with_area.loc[:,'area_flux_gt_gained_total_error'] = \
    np.sqrt((Arctic_discharge_fluxes_with_area['perimeter_error_gained_total']*Arctic_discharge_fluxes_with_area['thickness_for_area_gained_total']/1000*0.9)**2+
    (Arctic_discharge_fluxes_with_area['area_km2_gained_total']*Arctic_discharge_fluxes_with_area['thickness_for_area_gained_total_err']/1000*0.9)**2)

Arctic_discharge_fluxes_with_area.loc[:,'area_flux_gt_lost_total'] = Arctic_discharge_fluxes_with_area['area_km2_lost_total']*Arctic_discharge_fluxes_with_area['thickness_for_area_lost_total']/1000*0.900
Arctic_discharge_fluxes_with_area.loc[:,'area_flux_gt_lost_total_error'] = \
    np.sqrt((Arctic_discharge_fluxes_with_area['perimeter_error_lost_total']*Arctic_discharge_fluxes_with_area['thickness_for_area_lost_total']/1000*0.9)**2+
    (Arctic_discharge_fluxes_with_area['area_km2_lost_total']*Arctic_discharge_fluxes_with_area['thickness_for_area_lost_total_err']/1000*0.9)**2)

Arctic_discharge_fluxes_with_area.loc[:,'terminus_gt_change_per_year_early_without_melt']=(Arctic_discharge_fluxes_with_area['area_flux_gt_gained_early']/Arctic_discharge_fluxes_with_area['early_to_middle_years_elapsed'])-(Arctic_discharge_fluxes_with_area['area_flux_gt_lost_early']/Arctic_discharge_fluxes_with_area['early_to_middle_years_elapsed'])
Arctic_discharge_fluxes_with_area.loc[:,'terminus_gt_change_per_year_early_area_error_without_melt']=np.sqrt((Arctic_discharge_fluxes_with_area['area_flux_gt_gained_early_error']/Arctic_discharge_fluxes_with_area['early_to_middle_years_elapsed'])**2 + (Arctic_discharge_fluxes_with_area['area_flux_gt_lost_early_error']/Arctic_discharge_fluxes_with_area['early_to_middle_years_elapsed'])**2)
Arctic_discharge_fluxes_with_area.loc[:,'terminus_gt_change_per_year_late_without_melt']=(Arctic_discharge_fluxes_with_area['area_flux_gt_gained_late']/Arctic_discharge_fluxes_with_area['middle_to_late_years_elapsed'])-(Arctic_discharge_fluxes_with_area['area_flux_gt_lost_late']/Arctic_discharge_fluxes_with_area['middle_to_late_years_elapsed'])
Arctic_discharge_fluxes_with_area.loc[:,'terminus_gt_change_per_year_late_area_error_without_melt']=np.sqrt((Arctic_discharge_fluxes_with_area['area_flux_gt_gained_late_error']/Arctic_discharge_fluxes_with_area['middle_to_late_years_elapsed'])**2 + (Arctic_discharge_fluxes_with_area['area_flux_gt_lost_late_error']/Arctic_discharge_fluxes_with_area['middle_to_late_years_elapsed'])**2)
Arctic_discharge_fluxes_with_area.loc[:,'terminus_gt_change_per_year_total_without_melt']=(Arctic_discharge_fluxes_with_area['area_flux_gt_gained_total']/Arctic_discharge_fluxes_with_area['total_years_elapsed'])-(Arctic_discharge_fluxes_with_area['area_flux_gt_lost_total']/Arctic_discharge_fluxes_with_area['total_years_elapsed'])
Arctic_discharge_fluxes_with_area.loc[:,'terminus_gt_change_per_year_total_area_error_without_melt']=np.sqrt((Arctic_discharge_fluxes_with_area['area_flux_gt_gained_total_error']/Arctic_discharge_fluxes_with_area['total_years_elapsed'])**2 + (Arctic_discharge_fluxes_with_area['area_flux_gt_lost_total_error']/Arctic_discharge_fluxes_with_area['total_years_elapsed'])**2)

#save discharge without correction for melt
Arctic_discharge_fluxes_with_area.loc[:,'2000_2010_mean_flux_gt_without_melt'] = Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt']
Arctic_discharge_fluxes_with_area.loc[:,'2000_2010_mean_flux_err_gt_without_melt'] = Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_err_gt']
Arctic_discharge_fluxes_with_area.loc[:,'2010_2020_mean_flux_gt_without_melt'] = Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt']
Arctic_discharge_fluxes_with_area.loc[:,'2010_2020_mean_flux_err_gt_without_melt'] = Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_err_gt']

#calculate the melt correction
Arctic_discharge_fluxes_with_area.loc[:,'melt_correct_gt_early_terminus_change'] = Arctic_discharge_fluxes_with_area['net_area_change_early']/2*Arctic_discharge_fluxes_with_area['Melt_early_avg']/1000
Arctic_discharge_fluxes_with_area.loc[:,'melt_correct_gt_late_terminus_change'] = Arctic_discharge_fluxes_with_area['net_area_change_late']/2*Arctic_discharge_fluxes_with_area['Melt_late_avg']/1000
Arctic_discharge_fluxes_with_area.loc[:,'melt_correct_gt_total_terminus_change'] = Arctic_discharge_fluxes_with_area['net_area_change_total']/2*Arctic_discharge_fluxes_with_area['Melt_total_avg']/1000

Arctic_discharge_fluxes_with_area.loc[:,'melt_correct_gt_early_err_terminus_change'] = np.sqrt((np.abs(Arctic_discharge_fluxes_with_area['net_area_change_early']/2)*Arctic_discharge_fluxes_with_area['Melt_early_avg_err']/1000)**2+
    (np.abs(Arctic_discharge_fluxes_with_area['net_area_change_early_error']/2)*Arctic_discharge_fluxes_with_area['Melt_early_avg']/1000)**2)
Arctic_discharge_fluxes_with_area.loc[:,'melt_correct_gt_late_err_terminus_change'] = np.sqrt((np.abs(Arctic_discharge_fluxes_with_area['net_area_change_late']/2)*Arctic_discharge_fluxes_with_area['Melt_late_avg_err']/1000)**2+
    (np.abs(Arctic_discharge_fluxes_with_area['net_area_change_late_error']/2)*Arctic_discharge_fluxes_with_area['Melt_late_avg']/1000)**2)
Arctic_discharge_fluxes_with_area.loc[:,'melt_correct_gt_total_err_terminus_change'] = np.sqrt((np.abs(Arctic_discharge_fluxes_with_area['net_area_change_total']/2)*Arctic_discharge_fluxes_with_area['Melt_total_avg_err']/1000)**2+
    (np.abs(Arctic_discharge_fluxes_with_area['net_area_change_total_error']/2)*Arctic_discharge_fluxes_with_area['Melt_total_avg']/1000)**2)

#only multiply by end of time period for area because otherwise you are double counting it with terminus change, you want just the area unique to discarhge during that time period.
for i in range(0,len(Arctic_discharge_fluxes_with_area)):
    if Arctic_discharge_fluxes_with_area['net_area_change_early'][i]>0:
        Arctic_discharge_fluxes_with_area.loc[i,'melt_correct_gt_early_discharge'] = Arctic_discharge_fluxes_with_area['area_below_fluxgate_early'][i]*Arctic_discharge_fluxes_with_area['Melt_early_avg'][i]/1000
        Arctic_discharge_fluxes_with_area.loc[i,'melt_correct_gt_early_err_discharge'] = Arctic_discharge_fluxes_with_area['area_below_fluxgate_early'][i]*Arctic_discharge_fluxes_with_area['Melt_early_avg_err'][i]/1000
    if Arctic_discharge_fluxes_with_area['net_area_change_early'][i]<0:
        Arctic_discharge_fluxes_with_area.loc[i,'melt_correct_gt_early_discharge'] = Arctic_discharge_fluxes_with_area['area_below_fluxgate_middle'][i]*Arctic_discharge_fluxes_with_area['Melt_early_avg'][i]/1000
        Arctic_discharge_fluxes_with_area.loc[i,'melt_correct_gt_early_err_discharge'] = Arctic_discharge_fluxes_with_area['area_below_fluxgate_middle'][i]*Arctic_discharge_fluxes_with_area['Melt_early_avg_err'][i]/1000
    if Arctic_discharge_fluxes_with_area['net_area_change_early'][i]==0:
        Arctic_discharge_fluxes_with_area.loc[i,'melt_correct_gt_early_discharge'] = 0
        Arctic_discharge_fluxes_with_area.loc[i,'melt_correct_gt_early_err_discharge'] = 0
    if Arctic_discharge_fluxes_with_area['net_area_change_late'][i]>0:
        Arctic_discharge_fluxes_with_area.loc[i,'melt_correct_gt_late_discharge'] = Arctic_discharge_fluxes_with_area['area_below_fluxgate_middle'][i]*Arctic_discharge_fluxes_with_area['Melt_late_avg'][i]/1000
        Arctic_discharge_fluxes_with_area.loc[i,'melt_correct_gt_late_err_discharge'] = Arctic_discharge_fluxes_with_area['area_below_fluxgate_middle'][i]*Arctic_discharge_fluxes_with_area['Melt_late_avg_err'][i]/1000
    if Arctic_discharge_fluxes_with_area['net_area_change_late'][i]<0:
        Arctic_discharge_fluxes_with_area.loc[i,'melt_correct_gt_late_discharge'] = Arctic_discharge_fluxes_with_area['area_below_fluxgate_late'][i]*Arctic_discharge_fluxes_with_area['Melt_late_avg'][i]/1000
        Arctic_discharge_fluxes_with_area.loc[i,'melt_correct_gt_late_err_discharge'] = Arctic_discharge_fluxes_with_area['area_below_fluxgate_late'][i]*Arctic_discharge_fluxes_with_area['Melt_late_avg_err'][i]/1000
    if Arctic_discharge_fluxes_with_area['net_area_change_late'][i]==0:
        Arctic_discharge_fluxes_with_area.loc[i,'melt_correct_gt_late_discharge'] = 0
        Arctic_discharge_fluxes_with_area.loc[i,'melt_correct_gt_late_err_discharge'] = 0

Arctic_discharge_fluxes_with_area.loc[:,'terminus_gt_change_per_year_early']=np.nan
Arctic_discharge_fluxes_with_area.loc[:,'terminus_gt_change_per_year_early_area_error']=np.nan
Arctic_discharge_fluxes_with_area.loc[:,'terminus_gt_change_per_year_late']=np.nan
Arctic_discharge_fluxes_with_area.loc[:,'terminus_gt_change_per_year_late_area_error']=np.nan
Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_total'] = Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_total_without_melt'] + Arctic_discharge_fluxes_with_area['melt_correct_gt_total_terminus_change']
Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_total_area_error'] = np.sqrt(Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_total_area_error_without_melt']**2 + Arctic_discharge_fluxes_with_area['melt_correct_gt_total_err_terminus_change']**2)

Arctic_discharge_fluxes_with_area.loc[:,'2000_2010_mean_flux_gt']=np.nan
Arctic_discharge_fluxes_with_area.loc[:,'2000_2010_mean_flux_err_gt']=np.nan
Arctic_discharge_fluxes_with_area.loc[:,'2010_2020_mean_flux_gt']=np.nan
Arctic_discharge_fluxes_with_area.loc[:,'2010_2020_mean_flux_err_gt']=np.nan

Arctic_discharge_fluxes_with_area.loc[:,'FA_not_calculated_in_normal_way_early'] = 0
Arctic_discharge_fluxes_with_area.loc[:,'FA_not_calculated_in_normal_way_late'] = 0

#if the glaciers are gone now, set them to 0
mask = Arctic_discharge_fluxes_with_area['Middle_satellite_id']=='nan'
Arctic_discharge_fluxes_with_area.loc[mask,'terminus_gt_change_per_year_late']= 0
Arctic_discharge_fluxes_with_area.loc[mask,'terminus_gt_change_per_year_late_area_error'] = 0
Arctic_discharge_fluxes_with_area.loc[mask,'2010_2020_mean_flux_gt'] = 0
Arctic_discharge_fluxes_with_area.loc[mask,'2010_2020_mean_flux_err_gt']= 0
Arctic_discharge_fluxes_with_area.loc[mask,'melt_correct_gt_late_discharge'] = 0
Arctic_discharge_fluxes_with_area.loc[mask,'melt_correct_gt_late_err_discharge'] = 0
Arctic_discharge_fluxes_with_area.loc[mask,'melt_correct_gt_late_terminus_change'] = 0
Arctic_discharge_fluxes_with_area.loc[mask,'melt_correct_gt_late_err_terminus_change'] = 0
Arctic_discharge_fluxes_with_area.loc[mask,'2010_2020_mean_flux_gt_without_melt'] = 0

#add melt correction and deal with instances where the melt is greater than the total mass change
discharge_adjusted_early=0
discharge_adjusted_late=0
discharge_adjusted_fixed_early=0
discharge_adjusted_fixed_late=0

discharge_adjusted_fixed_early_total=0
discharge_adjusted_fixed_early_total_err=0
discharge_adjusted_early_total=0
discharge_adjusted_early_total_err=0

discharge_adjusted_fixed_late_total=0
discharge_adjusted_fixed_late_total_err=0
discharge_adjusted_late_total=0
discharge_adjusted_late_total_err=0

for i in range(0,len(Arctic_discharge_fluxes_with_area)):
    for time_period in ['early','late']:
        if (Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_'+time_period+'_without_melt'][i] > Arctic_discharge_fluxes_with_area['melt_correct_gt_'+time_period+'_terminus_change'][i])&(Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_'+time_period+'_without_melt'][i]<0):
            print(i,Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_'+time_period+'_without_melt'][i],Arctic_discharge_fluxes_with_area['melt_correct_gt_'+time_period+'_terminus_change'][i])
            Arctic_discharge_fluxes_with_area.loc[i,'terminus_gt_change_per_year_'+time_period] = Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_'+time_period+'_without_melt'][i]-Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_'+time_period+'_area_error_without_melt'][i] + Arctic_discharge_fluxes_with_area['melt_correct_gt_'+time_period+'_terminus_change'][i]+Arctic_discharge_fluxes_with_area['melt_correct_gt_'+time_period+'_err_terminus_change'][i]
            if Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_'+time_period][i] <0:
                Arctic_discharge_fluxes_with_area.loc[i,'terminus_gt_change_per_year_'+time_period]=Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_'+time_period+''][i]/2
                Arctic_discharge_fluxes_with_area.loc[i,'terminus_gt_change_per_year_'+time_period+'_area_error']=-Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_'+time_period][i]/2
                if time_period == 'early':
                    Arctic_discharge_fluxes_with_area.loc[i,'FA_not_calculated_in_normal_way_early'] = 1
                if time_period == 'late':
                    Arctic_discharge_fluxes_with_area.loc[i,'FA_not_calculated_in_normal_way_late'] = 1
            if Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_'+time_period][i]>0:
                print(i,' still a problem terminus ', time_period)
        elif time_period == 'early':
            Arctic_discharge_fluxes_with_area.loc[i,'terminus_gt_change_per_year_early'] = Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early_without_melt'][i] + Arctic_discharge_fluxes_with_area['melt_correct_gt_early_terminus_change'][i]
            Arctic_discharge_fluxes_with_area.loc[i,'terminus_gt_change_per_year_early_area_error'] = np.sqrt(Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early_area_error_without_melt'][i]**2 + Arctic_discharge_fluxes_with_area['melt_correct_gt_early_err_terminus_change'][i]**2)
        elif time_period == 'late':
            Arctic_discharge_fluxes_with_area.loc[i,'terminus_gt_change_per_year_late'] = Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late_without_melt'][i] + Arctic_discharge_fluxes_with_area['melt_correct_gt_late_terminus_change'][i]
            Arctic_discharge_fluxes_with_area.loc[i,'terminus_gt_change_per_year_late_area_error'] = np.sqrt(Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late_area_error_without_melt'][i]**2 + Arctic_discharge_fluxes_with_area['melt_correct_gt_late_err_terminus_change'][i]**2)

    if np.abs(Arctic_discharge_fluxes_with_area['melt_correct_gt_early_discharge'][i])>Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt_without_melt'][i]:
        Arctic_discharge_fluxes_with_area.loc[i,'2000_2010_mean_flux_gt'] = Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt_without_melt'][i]+Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_err_gt_without_melt'][i]+Arctic_discharge_fluxes_with_area['melt_correct_gt_early_discharge'][i]+(Arctic_discharge_fluxes_with_area['melt_correct_gt_early_err_discharge'][i]*(Arctic_discharge_fluxes_with_area['melt_correct_gt_early_discharge'][i]/np.abs(Arctic_discharge_fluxes_with_area['melt_correct_gt_early_discharge'][i])))
        if Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt'][i]>0:
            Arctic_discharge_fluxes_with_area.loc[i,'2000_2010_mean_flux_gt']=Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt'][i]/2
            Arctic_discharge_fluxes_with_area.loc[i,'2000_2010_mean_flux_err_gt']=Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt'][i]/2
            Arctic_discharge_fluxes_with_area.loc[i,'FA_not_calculated_in_normal_way_early'] = 1
        if Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt'][i]<0:
            Arctic_discharge_fluxes_with_area.loc[i,'2000_2010_mean_flux_gt']=(Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt_without_melt'][i]+Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_err_gt_without_melt'][i])/2
            Arctic_discharge_fluxes_with_area.loc[i,'2000_2010_mean_flux_err_gt']=Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt'][i]
            Arctic_discharge_fluxes_with_area.loc[i,'FA_not_calculated_in_normal_way_early'] = 1
            discharge_adjusted_fixed_early=discharge_adjusted_fixed_early+1
            discharge_adjusted_fixed_early_total=discharge_adjusted_fixed_early_total+Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt'][i]
            discharge_adjusted_fixed_early_total_err=discharge_adjusted_fixed_early_total_err+Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_err_gt'][i]
        discharge_adjusted_early=discharge_adjusted_early+1
        discharge_adjusted_early_total=discharge_adjusted_early_total+Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt'][i]
        discharge_adjusted_early_total_err=discharge_adjusted_early_total_err+Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_err_gt'][i]
    else:
        Arctic_discharge_fluxes_with_area.loc[i,'2000_2010_mean_flux_gt'] = Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt_without_melt'][i] + Arctic_discharge_fluxes_with_area['melt_correct_gt_early_discharge'][i]
        Arctic_discharge_fluxes_with_area.loc[i,'2000_2010_mean_flux_err_gt'] = np.sqrt(Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_err_gt_without_melt'][i]**2 + Arctic_discharge_fluxes_with_area['melt_correct_gt_early_err_discharge'][i]**2)
        
    if np.abs(Arctic_discharge_fluxes_with_area['melt_correct_gt_late_discharge'][i])>Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt_without_melt'][i]:
        Arctic_discharge_fluxes_with_area.loc[i,'2010_2020_mean_flux_gt'] = Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt_without_melt'][i]+Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_err_gt_without_melt'][i]+Arctic_discharge_fluxes_with_area['melt_correct_gt_late_discharge'][i]+(Arctic_discharge_fluxes_with_area['melt_correct_gt_late_err_discharge'][i]*(-Arctic_discharge_fluxes_with_area['melt_correct_gt_late_discharge'][i]/np.abs(Arctic_discharge_fluxes_with_area['melt_correct_gt_late_discharge'][i])))
        if Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt'][i]>0:
            Arctic_discharge_fluxes_with_area.loc[i,'2010_2020_mean_flux_gt']=Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt'][i]/2
            Arctic_discharge_fluxes_with_area.loc[i,'2010_2020_mean_flux_err_gt']=Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt'][i]/2
            Arctic_discharge_fluxes_with_area.loc[i,'FA_not_calculated_in_normal_way_late'] = 1
        if Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt'][i]<0:
            Arctic_discharge_fluxes_with_area.loc[i,'2010_2020_mean_flux_gt']=(Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt_without_melt'][i]+Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_err_gt_without_melt'][i])/2
            Arctic_discharge_fluxes_with_area.loc[i,'2010_2020_mean_flux_err_gt']=Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt'][i]
            Arctic_discharge_fluxes_with_area.loc[i,'FA_not_calculated_in_normal_way_late'] = 1
            discharge_adjusted_fixed_late=discharge_adjusted_fixed_late+1
            discharge_adjusted_fixed_late_total=discharge_adjusted_fixed_late_total+Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt'][i]
            discharge_adjusted_fixed_late_total_err=discharge_adjusted_fixed_late_total_err+Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_err_gt'][i]
        discharge_adjusted_late=discharge_adjusted_late+1
        discharge_adjusted_late_total=discharge_adjusted_late_total+Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt'][i]
        discharge_adjusted_late_total_err=discharge_adjusted_late_total_err+Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_err_gt'][i]
    else:
        Arctic_discharge_fluxes_with_area.loc[i,'2010_2020_mean_flux_gt'] = Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt_without_melt'][i] + Arctic_discharge_fluxes_with_area['melt_correct_gt_late_discharge'][i]
        Arctic_discharge_fluxes_with_area.loc[i,'2010_2020_mean_flux_err_gt'] = np.sqrt(Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_err_gt_without_melt'][i]**2 + Arctic_discharge_fluxes_with_area['melt_correct_gt_late_err_discharge'][i]**2)

print('2',Arctic_discharge_fluxes_with_area['Name_x'][951],Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late_without_melt'][951])

        
print('Glaciers that had melt bigger than discharge early:',discharge_adjusted_early, ' which was',discharge_adjusted_early_total,' ± ',discharge_adjusted_early_total_err)
print('Glaciers that had melt bigger than discharge early fixed:',discharge_adjusted_fixed_early, ' which was ',discharge_adjusted_fixed_early_total,' ± ',discharge_adjusted_fixed_early_total_err)
print()
print('Glaciers that had melt bigger than discharge late:',discharge_adjusted_late, ' which was',discharge_adjusted_late_total,' ± ',discharge_adjusted_late_total_err)
print('Glaciers that had melt bigger than discharge late fixed:',discharge_adjusted_fixed_late, ' which was',discharge_adjusted_fixed_late_total,' ± ',discharge_adjusted_fixed_late_total_err)
print()
print()
#deal with what happens when the terminus change is greater than the discharge
sum_discharge_early = 0
sum_terminus_early = 0
sum_discharge_late = 0
sum_terminus_late = 0
count_early = 0
count_late = 0
        
for i in range(0,len(Arctic_discharge_fluxes_with_area)): #make terminus change as small as possible to be less than discharge
    if (Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt'][i]<Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early'][i])&(Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt'][i]>(Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early'][i]-Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early_area_error'][i])):
        mass_gap=Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt'][i]-(Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early'][i]-Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early_area_error'][i])
        Arctic_discharge_fluxes_with_area.loc[i,'terminus_gt_change_per_year_early']=Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early'][i]-Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early_area_error'][i]+mass_gap/2
        Arctic_discharge_fluxes_with_area.loc[i,'terminus_gt_change_per_year_early_area_error']=mass_gap/2
        count_early = count_early +1
    if (Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt'][i]<Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late'][i])&(Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt'][i]>(Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late'][i]-Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late_area_error'][i])):
        mass_gap=Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt'][i]-(Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late'][i]-Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late_area_error'][i])
        Arctic_discharge_fluxes_with_area.loc[i,'terminus_gt_change_per_year_late']=Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late'][i]-Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late_area_error'][i]+mass_gap/2
        Arctic_discharge_fluxes_with_area.loc[i,'terminus_gt_change_per_year_late_area_error']=mass_gap/2
        count_late = count_late +1  
print('Number of glaciers that we reduced terminus changes because it was bigger (within uncertainty) than discarhge (advancing glaciers) early: ',count_early)
print('Number of glaciers that we reduced terminus changes because it was bigger (within uncertainty) than discarhge (advancing glaciers) late: ',count_late)


#calculate frontal ablation wihtout any melt
Arctic_discharge_fluxes_with_area.loc[:,'Frontal_ablation_2000_to_2010_gt_per_yr_mean_without_melt'] = -Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt_without_melt'] + Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early_without_melt']
Arctic_discharge_fluxes_with_area.loc[:,'Frontal_ablation_2000_to_2010_gt_per_yr_mean_without_melt_err'] = np.sqrt(Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_err_gt_without_melt']**2 
                                                                                                                   + Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early_area_error_without_melt']**2)
Arctic_discharge_fluxes_with_area.loc[:,'Frontal_ablation_2010_to_2020_gt_per_yr_mean_without_melt'] = -Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt_without_melt'] + Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late_without_melt']
Arctic_discharge_fluxes_with_area.loc[:,'Frontal_ablation_2010_to_2020_gt_per_yr_mean_without_melt_err'] = np.sqrt(Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_err_gt_without_melt']**2 + Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late_area_error_without_melt']**2)

#calculate final numbers
#sqrt on uncertainties because of this link: http://lectureonline.cl.msu.edu/~mmp/labs/error/e2.htm
Arctic_discharge_fluxes_with_area.loc[:,'Frontal_ablation_2000_to_2010_gt_per_yr_mean'] = -Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt'] + Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early']
Arctic_discharge_fluxes_with_area.loc[:,'Frontal_ablation_2000_to_2010_gt_per_yr_mean_err'] = np.sqrt(Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_err_gt']**2 + Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early_area_error']**2)
Arctic_discharge_fluxes_with_area.loc[:,'Frontal_ablation_2010_to_2020_gt_per_yr_mean'] = -Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt'] + Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late']
Arctic_discharge_fluxes_with_area.loc[:,'Frontal_ablation_2010_to_2020_gt_per_yr_mean_err'] = np.sqrt(Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_err_gt']**2 + Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late_area_error']**2)

Arctic_discharge_fluxes_with_area.loc[:,'2000_2010_mean_flux_gt_without_melt_without_correction'] = Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt_without_melt']
Arctic_discharge_fluxes_with_area.loc[:,'2000_2010_mean_flux_err_gt_without_melt_without_correction'] = Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_err_gt_without_melt']
Arctic_discharge_fluxes_with_area.loc[:,'terminus_gt_change_per_year_early_without_melt_without_correction'] = Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early_without_melt']
Arctic_discharge_fluxes_with_area.loc[:,'terminus_gt_change_per_year_early_area_error_without_melt_without_correction'] = Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early_area_error_without_melt']

Arctic_discharge_fluxes_with_area.loc[:,'2010_2020_mean_flux_gt_without_melt_without_correction'] = Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt_without_melt']
Arctic_discharge_fluxes_with_area.loc[:,'2010_2020_mean_flux_err_gt_without_melt_without_correction'] = Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_err_gt_without_melt']
Arctic_discharge_fluxes_with_area.loc[:,'terminus_gt_change_per_year_late_without_melt_without_correction'] = Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late_without_melt']
Arctic_discharge_fluxes_with_area.loc[:,'terminus_gt_change_per_year_late_area_error_without_melt_without_correction'] = Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late_area_error_without_melt']

for z in ['','_without_melt']:
    counter_big_term_late = 0
    counter_big_term_early = 0
    count_tiny_early = 0
    count_tiny_late=0
    sum_FA_early =0
    for i in range(0,len(Arctic_discharge_fluxes_with_area)): #make terminus change as small as possible to be less than discharge
        if (Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt'+z][i]<Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early'+z][i])&(Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt'+z][i]>0.04):
            print('==============================')
            print('discharge early',Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt'+z][i],Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_err_gt'+z][i])
            print('terminus early',Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early'+z][i],Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early_area_error'+z][i])
            dis_term_max = Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt'+z][i]+Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_err_gt'+z][i]
            dis_term_min = Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early'+z][i]-Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early_area_error'+z][i]
            print('max min',dis_term_max,dis_term_min)
            if dis_term_max<dis_term_min:
                print('PROBLEM PROBLEM PROBLEM')
            else:
                print('SUCCESS SUCCESS SUCCCESS')
            dis_term_avg = (dis_term_max+dis_term_min)/2
            dis_term_err = (dis_term_max-dis_term_min)/2
            Arctic_discharge_fluxes_with_area.loc[i,'2000_2010_mean_flux_gt'+z] = dis_term_avg
            Arctic_discharge_fluxes_with_area.loc[i,'2000_2010_mean_flux_err_gt'+z] = dis_term_err
            Arctic_discharge_fluxes_with_area.loc[i,'terminus_gt_change_per_year_early'+z] = dis_term_avg-dis_term_err
            Arctic_discharge_fluxes_with_area.loc[i,'terminus_gt_change_per_year_early_area_error'+z] = dis_term_err
            Arctic_discharge_fluxes_with_area.loc[i,'Frontal_ablation_2000_to_2010_gt_per_yr_mean'+z] = -dis_term_err
            Arctic_discharge_fluxes_with_area.loc[i,'Frontal_ablation_2000_to_2010_gt_per_yr_mean_err'+z] = dis_term_err
            print('still big early')
            print(Arctic_discharge_fluxes_with_area['RGIId'][i],Arctic_discharge_fluxes_with_area['Name_x'][i])
            print('FA',dis_term_err,Arctic_discharge_fluxes_with_area['Frontal_ablation_2000_to_2010_gt_per_yr_mean'+z][i],Arctic_discharge_fluxes_with_area['Frontal_ablation_2000_to_2010_gt_per_yr_mean_err'+z][i])
            counter_big_term_early = counter_big_term_early+1
        if (Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt'+z][i]<Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late'+z][i])&(Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt'+z][i]>0.04):
            print('==============================')
            print('discharge late',Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt'+z][i],Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_err_gt'+z][i])
            print('terminus late',Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late'+z][i],Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late_area_error'+z][i])
            dis_term_max = Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt'+z][i]+Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_err_gt'+z][i]
            dis_term_min = Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late'+z][i]-Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late_area_error'+z][i]
            print('max min',dis_term_max,dis_term_min)
            if dis_term_max<dis_term_min:
                print('PROBLEM PROBLEM PROBLEM')
            else:
                print('SUCCESS SUCCESS SUCCCESS')
            dis_term_avg = (dis_term_max+dis_term_min)/2
            dis_term_err = (dis_term_max-dis_term_min)/2
            Arctic_discharge_fluxes_with_area.loc[i,'2010_2020_mean_flux_gt'+z] = dis_term_avg
            Arctic_discharge_fluxes_with_area.loc[i,'2010_2020_mean_flux_err_gt'+z] = dis_term_err
            Arctic_discharge_fluxes_with_area.loc[i,'terminus_gt_change_per_year_late'+z] = dis_term_avg-dis_term_err
            Arctic_discharge_fluxes_with_area.loc[i,'terminus_gt_change_per_year_late_area_error'+z] = dis_term_err
            Arctic_discharge_fluxes_with_area.loc[i,'Frontal_ablation_2010_to_2020_gt_per_yr_mean'+z] = -dis_term_err
            Arctic_discharge_fluxes_with_area.loc[i,'Frontal_ablation_2010_to_2020_gt_per_yr_mean_err'+z] = np.abs(dis_term_err)
            print('still big late')
            print(Arctic_discharge_fluxes_with_area['RGIId'][i],Arctic_discharge_fluxes_with_area['Name_x'][i])
            print('FA',dis_term_err,Arctic_discharge_fluxes_with_area['Frontal_ablation_2010_to_2020_gt_per_yr_mean'+z][i],Arctic_discharge_fluxes_with_area['Frontal_ablation_2010_to_2020_gt_per_yr_mean_err'+z][i])
            counter_big_term_late = counter_big_term_late+1      
        #for glacires with small terminus changes, just make the discharge bigger
        if (Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt'][i]<Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early'][i])&(Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early'][i]<0.05):
            Arctic_discharge_fluxes_with_area.loc[i,'2000_2010_mean_flux_gt']=Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt'][i]+Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early'][i]
            Arctic_discharge_fluxes_with_area.loc[i,'2000_2010_mean_flux_err_gt']=Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt'][i]
            print('+++++++++++++++++++++++++',Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt'][i],Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early'][i])
            Arctic_discharge_fluxes_with_area.loc[i,'Frontal_ablation_2000_to_2010_gt_per_yr_mean']=-Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt'][i]+Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early'][i]
            Arctic_discharge_fluxes_with_area.loc[i,'Frontal_ablation_2000_to_2010_gt_per_yr_mean_err']=-Arctic_discharge_fluxes_with_area['Frontal_ablation_2000_to_2010_gt_per_yr_mean'][i]
            print(Arctic_discharge_fluxes_with_area['Frontal_ablation_2000_to_2010_gt_per_yr_mean'][i])
            sum_FA_early = sum_FA_early+Arctic_discharge_fluxes_with_area['Frontal_ablation_2000_to_2010_gt_per_yr_mean'][i]
            count_tiny_early = count_tiny_early+1
        if (Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt'][i]<Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late'][i])&(Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late'][i]<0.05):
            Arctic_discharge_fluxes_with_area.loc[i,'2010_2020_mean_flux_gt']=Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt'][i]+Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late'][i]
            Arctic_discharge_fluxes_with_area.loc[i,'2010_2020_mean_flux_err_gt']=Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt'][i]
            print('+++++++++++++++++++++++++',Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt'][i],Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late'][i])
            Arctic_discharge_fluxes_with_area.loc[i,'Frontal_ablation_2010_to_2020_gt_per_yr_mean']=-Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt'][i]+Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late'][i]
            Arctic_discharge_fluxes_with_area.loc[i,'Frontal_ablation_2010_to_2020_gt_per_yr_mean_err']=-Arctic_discharge_fluxes_with_area['Frontal_ablation_2010_to_2020_gt_per_yr_mean'][i]
            print(Arctic_discharge_fluxes_with_area['Frontal_ablation_2010_to_2020_gt_per_yr_mean'][i])
            count_tiny_late = count_tiny_late+1

    print('Number of glaciers that terminus changes was bigger than discahrge that I altered within uncertainties --early ',z,counter_big_term_early)
    print('Glaciers with terminus change less than 0.05 gt a-1 that now have changed discharge to be greater',count_tiny_early, ' accounting for this much FA:  ', sum_FA_early)
    print('Number of glaciers that terminus changes was bigger than discahrge that I altered within uncertainties --late ',z,counter_big_term_late)
    print('Glaciers with terminus change less than 0.05 gt a-1 that now have changed discharge to be greater',count_tiny_late)



#If the glacier was no longer marine terminating after 2010 - the later flux should be 0
Arctic_discharge_fluxes_with_area['Frontal_ablation_2010_to_2020_gt_per_yr_mean_without_melt']= np.where((Arctic_discharge_fluxes_with_area['Middle_satellite_id']=='nan'),0,Arctic_discharge_fluxes_with_area['Frontal_ablation_2010_to_2020_gt_per_yr_mean_without_melt'])     
Arctic_discharge_fluxes_with_area['Frontal_ablation_2010_to_2020_gt_per_yr_mean_err_without_melt']= np.where((Arctic_discharge_fluxes_with_area['Middle_satellite_id']=='nan'),0,Arctic_discharge_fluxes_with_area['Frontal_ablation_2010_to_2020_gt_per_yr_mean_err_without_melt'])

Arctic_discharge_fluxes_with_area['Frontal_ablation_2010_to_2020_gt_per_yr_mean']= np.where((Arctic_discharge_fluxes_with_area['Middle_satellite_id']=='nan'),0,Arctic_discharge_fluxes_with_area['Frontal_ablation_2010_to_2020_gt_per_yr_mean'])     
Arctic_discharge_fluxes_with_area['Frontal_ablation_2010_to_2020_gt_per_yr_mean_err']= np.where((Arctic_discharge_fluxes_with_area['Middle_satellite_id']=='nan'),0,Arctic_discharge_fluxes_with_area['Frontal_ablation_2010_to_2020_gt_per_yr_mean_err'])

Arctic_discharge_fluxes_with_area=Arctic_discharge_fluxes_with_area.drop(columns=['Name_y', 'geometry_gained_total','geometry_lost_total', 'geometry_gained_early','geometry_lost_early', 'geometry_gained_late','geometry_lost_late'])

#export data with flux and area change estimates
easting=[]
northing=[]
for i in range(0,len(Arctic_discharge_fluxes_with_area)):
    easting.append(Arctic_discharge_fluxes_with_area['easting'][i])
    northing.append(Arctic_discharge_fluxes_with_area['northing'][i])

#add lat lon in decimal degrees
inProj = Proj(init='epsg:3995')
outProj = Proj(init='epsg:4326')    
Arctic_discharge_fluxes_with_area.loc[:,'lon'],Arctic_discharge_fluxes_with_area.loc[:,'lat'] = transform(inProj,outProj,easting,northing)
    
# Arctic_discharge_fluxes_with_area.to_csv('Arctic_discharge_fluxes_with_area_'+date_string+'.txt')
#export to geopackage
Arctic_discharge_fluxes_with_area_export = gpd.GeoDataFrame(Arctic_discharge_fluxes_with_area, geometry=gpd.points_from_xy(Arctic_discharge_fluxes_with_area.easting, Arctic_discharge_fluxes_with_area.northing),crs='EPSG:3995')

#Add regions as an attribute
arctic_aois = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/vector_files/Arctic_AOIs.gpkg')
arctic_subaois = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/vector_files/Arctic_subAOIs.gpkg')
arctic_subaois2 = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/vector_files/Arctic_subAOIs_v2.gpkg')
Arctic_discharge_fluxes_with_area_export = gpd.sjoin(Arctic_discharge_fluxes_with_area_export,arctic_aois)
Arctic_discharge_fluxes_with_area_export = Arctic_discharge_fluxes_with_area_export.drop(columns='index_right')
Arctic_discharge_fluxes_with_area_export = gpd.sjoin(Arctic_discharge_fluxes_with_area_export,arctic_subaois)
Arctic_discharge_fluxes_with_area_export = Arctic_discharge_fluxes_with_area_export.drop(columns='index_right')
Arctic_discharge_fluxes_with_area_export = gpd.sjoin(Arctic_discharge_fluxes_with_area_export,arctic_subaois2)
Arctic_discharge_fluxes_with_area_export = Arctic_discharge_fluxes_with_area_export.drop(columns='index_right')

Arctic_discharge_fluxes_with_area_export.loc[:,'Frontal_ablation_2000_to_2010_gt_per_yr_mean_without_melt'] = -Arctic_discharge_fluxes_with_area_export['Frontal_ablation_2000_to_2010_gt_per_yr_mean_without_melt']
Arctic_discharge_fluxes_with_area_export.loc[:,'Frontal_ablation_2010_to_2020_gt_per_yr_mean_without_melt'] = -Arctic_discharge_fluxes_with_area_export['Frontal_ablation_2010_to_2020_gt_per_yr_mean_without_melt']

Arctic_discharge_fluxes_with_area_export.loc[:,'Frontal_ablation_2000_to_2010_gt_per_yr_mean'] = -Arctic_discharge_fluxes_with_area_export['Frontal_ablation_2000_to_2010_gt_per_yr_mean']
Arctic_discharge_fluxes_with_area_export.loc[:,'Frontal_ablation_2010_to_2020_gt_per_yr_mean'] = -Arctic_discharge_fluxes_with_area_export['Frontal_ablation_2010_to_2020_gt_per_yr_mean']

Arctic_discharge_fluxes_with_area_export.to_file('Arctic_discharge_fluxes_with_area_epsg3995_'+date_string+'_V4.gpkg', driver="GPKG")#change file name based on distance

Arctic_discharge_fluxes_with_area_export_important = Arctic_discharge_fluxes_with_area_export[['terminus_gt_change_per_year_early','terminus_gt_change_per_year_early_area_error','terminus_gt_change_per_year_late','terminus_gt_change_per_year_late_area_error',
    '2000_2010_mean_flux_gt','2000_2010_mean_flux_err_gt','2010_2020_mean_flux_gt','2010_2020_mean_flux_err_gt',
    'melt_correct_gt_early_terminus_change', 'melt_correct_gt_early_err_terminus_change','melt_correct_gt_late_terminus_change','melt_correct_gt_late_err_terminus_change',
    'melt_correct_gt_early_discharge','melt_correct_gt_early_err_discharge','melt_correct_gt_late_discharge','melt_correct_gt_late_err_discharge',
    'Frontal_ablation_2000_to_2010_gt_per_yr_mean','Frontal_ablation_2000_to_2010_gt_per_yr_mean_err','Frontal_ablation_2010_to_2020_gt_per_yr_mean','Frontal_ablation_2010_to_2020_gt_per_yr_mean_err','Melt_fluxes_from_regional_estimate','geometry']]
Arctic_discharge_fluxes_with_area_export_important.to_file('Arctic_discharge_fluxes_with_area_epsg3995_most_important_'+date_string+'_V4.gpkg', driver="GPKG")#change file name based on distance

2 Arnesenbreen 0.0016609653893010173
Glaciers that had melt bigger than discharge early: 546  which was nan  ±  nan
Glaciers that had melt bigger than discharge early fixed: 366  which was  0.7175334767425519  ±  0.7175334767425519

Glaciers that had melt bigger than discharge late: 641  which was nan  ±  nan
Glaciers that had melt bigger than discharge late fixed: 400  which was 0.45044775346245675  ±  0.45044775346245675


Number of glaciers that we reduced terminus changes because it was bigger (within uncertainty) than discarhge (advancing glaciers) early:  10
Number of glaciers that we reduced terminus changes because it was bigger (within uncertainty) than discarhge (advancing glaciers) late:  11
+++++++++++++++++++++++++ 0.048225649406087895 0.042164515968904456
-0.0060611334371834386
+++++++++++++++++++++++++ 0.031775056639097826 0.02656924731056309
-0.005205809328534736
+++++++++++++++++++++++++ 0.021575197971389803 0.018287825909179455
-0.003287372062210348
++++++++++++++++++

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:338: DeprecationWarning: This function is deprecated. See: htt

In [53]:
Arctic_discharge_fluxes_with_area_export.sum()[['terminus_gt_change_per_year_early','terminus_gt_change_per_year_early_area_error','terminus_gt_change_per_year_late','terminus_gt_change_per_year_late_area_error',
# 'area_flux_gt_gained_early','area_flux_gt_lost_early','area_flux_gt_gained_late','area_flux_gt_lost_late','area_flux_gt_gained_total','area_flux_gt_lost_total',
'terminus_gt_change_per_year_total','terminus_gt_change_per_year_total_area_error',
'2000_2010_mean_flux_gt','2000_2010_mean_flux_err_gt','2010_2020_mean_flux_gt','2010_2020_mean_flux_err_gt','melt_correct_gt_early_terminus_change', 'melt_correct_gt_early_err_terminus_change','melt_correct_gt_late_terminus_change','melt_correct_gt_late_err_terminus_change',
'melt_correct_gt_early_discharge','melt_correct_gt_early_err_discharge','melt_correct_gt_late_discharge','melt_correct_gt_late_err_discharge',
'Frontal_ablation_2000_to_2010_gt_per_yr_mean_without_melt','Frontal_ablation_2000_to_2010_gt_per_yr_mean_without_melt_err','Frontal_ablation_2010_to_2020_gt_per_yr_mean_without_melt','Frontal_ablation_2010_to_2020_gt_per_yr_mean_without_melt_err',
'Frontal_ablation_2000_to_2010_gt_per_yr_mean','Frontal_ablation_2000_to_2010_gt_per_yr_mean_err','Frontal_ablation_2010_to_2020_gt_per_yr_mean','Frontal_ablation_2010_to_2020_gt_per_yr_mean_err']]

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


terminus_gt_change_per_year_early                                -9.059662
terminus_gt_change_per_year_early_area_error                      6.551871
terminus_gt_change_per_year_late                                -11.565544
terminus_gt_change_per_year_late_area_error                       8.221507
terminus_gt_change_per_year_total                               -10.024504
terminus_gt_change_per_year_total_area_error                      7.441876
2000_2010_mean_flux_gt                                           30.730958
2000_2010_mean_flux_err_gt                                       12.984306
2010_2020_mean_flux_gt                                           40.248919
2010_2020_mean_flux_err_gt                                        6.181625
melt_correct_gt_early_terminus_change                             0.918582
melt_correct_gt_early_err_terminus_change                         0.328643
melt_correct_gt_late_terminus_change                              1.025022
melt_correct_gt_late_err_

In [36]:
Arctic_discharge_fluxes_with_area_export.sum()[['terminus_gt_change_per_year_early','terminus_gt_change_per_year_early_area_error','terminus_gt_change_per_year_late','terminus_gt_change_per_year_late_area_error',
# 'area_flux_gt_gained_early','area_flux_gt_lost_early','area_flux_gt_gained_late','area_flux_gt_lost_late','area_flux_gt_gained_total','area_flux_gt_lost_total',
'terminus_gt_change_per_year_total','terminus_gt_change_per_year_total_area_error',
'2000_2010_mean_flux_gt','2000_2010_mean_flux_err_gt','2010_2020_mean_flux_gt','2010_2020_mean_flux_err_gt','melt_correct_gt_early_terminus_change', 'melt_correct_gt_early_err_terminus_change','melt_correct_gt_late_terminus_change','melt_correct_gt_late_err_terminus_change',
'melt_correct_gt_early_discharge','melt_correct_gt_early_err_discharge','melt_correct_gt_late_discharge','melt_correct_gt_late_err_discharge',
'Frontal_ablation_2000_to_2010_gt_per_yr_mean_without_melt','Frontal_ablation_2000_to_2010_gt_per_yr_mean_without_melt_err','Frontal_ablation_2010_to_2020_gt_per_yr_mean_without_melt','Frontal_ablation_2010_to_2020_gt_per_yr_mean_without_melt_err',
'Frontal_ablation_2000_to_2010_gt_per_yr_mean','Frontal_ablation_2000_to_2010_gt_per_yr_mean_err','Frontal_ablation_2010_to_2020_gt_per_yr_mean','Frontal_ablation_2010_to_2020_gt_per_yr_mean_err']]

/Users/willkochtitzky/opt/miniconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


terminus_gt_change_per_year_early                                -9.061087
terminus_gt_change_per_year_early_area_error                      6.565674
terminus_gt_change_per_year_late                                -11.564857
terminus_gt_change_per_year_late_area_error                       8.222194
terminus_gt_change_per_year_total                               -10.024504
terminus_gt_change_per_year_total_area_error                      7.441876
2000_2010_mean_flux_gt                                           39.869441
2000_2010_mean_flux_err_gt                                       17.105696
2010_2020_mean_flux_gt                                           41.538986
2010_2020_mean_flux_err_gt                                        7.229616
melt_correct_gt_early_terminus_change                             0.918582
melt_correct_gt_early_err_terminus_change                         0.328643
melt_correct_gt_late_terminus_change                              1.025022
melt_correct_gt_late_err_

In [37]:
Arctic_discharge_fluxes_with_area_export.groupby('Region_name').sum()[[#'terminus_gt_change_per_year_early','terminus_gt_change_per_year_early_area_error','terminus_gt_change_per_year_late','terminus_gt_change_per_year_late_area_error',
# 'area_flux_gt_gained_early','area_flux_gt_lost_early','area_flux_gt_gained_late','area_flux_gt_lost_late','area_flux_gt_gained_total','area_flux_gt_lost_total',
# 'terminus_gt_change_per_year_total','terminus_gt_change_per_year_total_area_error',
# '2000_2010_mean_flux_gt','2000_2010_mean_flux_err_gt','2010_2020_mean_flux_gt','2010_2020_mean_flux_err_gt','melt_correct_gt_early_terminus_change', 'melt_correct_gt_early_err_terminus_change','melt_correct_gt_late_terminus_change','melt_correct_gt_late_err_terminus_change',
# 'melt_correct_gt_early_discharge','melt_correct_gt_early_err_discharge','melt_correct_gt_late_discharge','melt_correct_gt_late_err_discharge',
# 'Frontal_ablation_2000_to_2010_gt_per_yr_mean_without_melt','Frontal_ablation_2000_to_2010_gt_per_yr_mean_without_melt_err','Frontal_ablation_2010_to_2020_gt_per_yr_mean_without_melt','Frontal_ablation_2010_to_2020_gt_per_yr_mean_without_melt_err',
'Frontal_ablation_2000_to_2010_gt_per_yr_mean','Frontal_ablation_2000_to_2010_gt_per_yr_mean_err','Frontal_ablation_2010_to_2020_gt_per_yr_mean','Frontal_ablation_2010_to_2020_gt_per_yr_mean_err']]

,Frontal_ablation_2000_to_2010_gt_per_yr_mean,Frontal_ablation_2000_to_2010_gt_per_yr_mean_err,Frontal_ablation_2010_to_2020_gt_per_yr_mean,Frontal_ablation_2010_to_2020_gt_per_yr_mean_err
Region_name,,,,
Alaska,11.442468,0.476742,10.532239,0.348871
BBI,0.087311,0.072394,0.086404,0.065935
FJL,13.070762,8.144343,8.146393,3.861116
Greenland,4.353049,2.436437,3.201971,1.423562
Iceland,0.109139,0.107108,0.047713,0.027308
NZ,2.668718,1.240468,4.149321,1.015437
QEI,4.127140,1.479966,4.255577,1.294991
SZ,4.486049,1.501100,5.691324,1.013822
Svalbard and Jan Mayen,8.585893,4.115323,16.992901,3.024780


In [ ]:
Arctic_discharge_fluxes_with_area_export.groupby('Region_name').sum()[[#'terminus_gt_change_per_year_early','terminus_gt_change_per_year_early_area_error','terminus_gt_change_per_year_late','terminus_gt_change_per_year_late_area_error',
# 'area_flux_gt_gained_early','area_flux_gt_lost_early','area_flux_gt_gained_late','area_flux_gt_lost_late','area_flux_gt_gained_total','area_flux_gt_lost_total',
# 'terminus_gt_change_per_year_total','terminus_gt_change_per_year_total_area_error',
# '2000_2010_mean_flux_gt','2000_2010_mean_flux_err_gt','2010_2020_mean_flux_gt','2010_2020_mean_flux_err_gt','melt_correct_gt_early_terminus_change', 'melt_correct_gt_early_err_terminus_change','melt_correct_gt_late_terminus_change','melt_correct_gt_late_err_terminus_change',
# 'melt_correct_gt_early_discharge','melt_correct_gt_early_err_discharge','melt_correct_gt_late_discharge','melt_correct_gt_late_err_discharge',
# 'Frontal_ablation_2000_to_2010_gt_per_yr_mean_without_melt','Frontal_ablation_2000_to_2010_gt_per_yr_mean_without_melt_err','Frontal_ablation_2010_to_2020_gt_per_yr_mean_without_melt','Frontal_ablation_2010_to_2020_gt_per_yr_mean_without_melt_err',
'Frontal_ablation_2000_to_2010_gt_per_yr_mean','Frontal_ablation_2000_to_2010_gt_per_yr_mean_err','Frontal_ablation_2010_to_2020_gt_per_yr_mean','Frontal_ablation_2010_to_2020_gt_per_yr_mean_err']]

In [35]:
Arctic_discharge_fluxes_with_area_export.groupby('Region_name').sum()[[#'terminus_gt_change_per_year_early','terminus_gt_change_per_year_early_area_error','terminus_gt_change_per_year_late','terminus_gt_change_per_year_late_area_error',
# 'area_flux_gt_gained_early','area_flux_gt_lost_early','area_flux_gt_gained_late','area_flux_gt_lost_late','area_flux_gt_gained_total','area_flux_gt_lost_total',
# 'terminus_gt_change_per_year_total','terminus_gt_change_per_year_total_area_error',
# '2000_2010_mean_flux_gt','2000_2010_mean_flux_err_gt','2010_2020_mean_flux_gt','2010_2020_mean_flux_err_gt','melt_correct_gt_early_terminus_change', 'melt_correct_gt_early_err_terminus_change','melt_correct_gt_late_terminus_change','melt_correct_gt_late_err_terminus_change',
# 'melt_correct_gt_early_discharge','melt_correct_gt_early_err_discharge','melt_correct_gt_late_discharge','melt_correct_gt_late_err_discharge',
# 'Frontal_ablation_2000_to_2010_gt_per_yr_mean_without_melt','Frontal_ablation_2000_to_2010_gt_per_yr_mean_without_melt_err','Frontal_ablation_2010_to_2020_gt_per_yr_mean_without_melt','Frontal_ablation_2010_to_2020_gt_per_yr_mean_without_melt_err',
'Frontal_ablation_2000_to_2010_gt_per_yr_mean','Frontal_ablation_2000_to_2010_gt_per_yr_mean_err','Frontal_ablation_2010_to_2020_gt_per_yr_mean','Frontal_ablation_2010_to_2020_gt_per_yr_mean_err']]

,Frontal_ablation_2000_to_2010_gt_per_yr_mean,Frontal_ablation_2000_to_2010_gt_per_yr_mean_err,Frontal_ablation_2010_to_2020_gt_per_yr_mean,Frontal_ablation_2010_to_2020_gt_per_yr_mean_err
Region_name,,,,
Alaska,11.889793,0.602154,11.054783,0.417706
BBI,0.157309,0.078864,0.130141,0.063326
FJL,15.802416,18.464506,10.723992,6.068254
Greenland,6.143472,3.529993,6.173673,2.230100
Iceland,0.220396,0.128744,0.110609,0.031803
NZ,5.990013,2.082721,6.085706,1.318148
QEI,4.950557,1.889801,5.132011,1.658635
SZ,4.211556,2.631401,5.690452,1.504125
Svalbard and Jan Mayen,9.674883,8.997562,17.647918,4.797424


In [56]:
# Arctic_discharge_fluxes_with_area_export[Arctic_discharge_fluxes_with_area_export['Frontal_ablation_2000_to_2010_gt_per_yr_mean']<Arctic_discharge_fluxes_with_area_export['Frontal_ablation_2000_to_2010_gt_per_yr_mean_err']]
# Arctic_discharge_fluxes_with_area_export[np.abs(Arctic_discharge_fluxes_with_area_export['terminus_gt_change_per_year_early'])<Arctic_discharge_fluxes_with_area_export['terminus_gt_change_per_year_early_area_error']]
Arctic_discharge_fluxes_with_area_export[np.abs(Arctic_discharge_fluxes_with_area_export['2000_2010_mean_flux_gt'])<Arctic_discharge_fluxes_with_area_export['2000_2010_mean_flux_err_gt']]

,RGIId,2000_2010_mean_flux_gt,2000_2010_mean_flux_err_gt,2010_2020_mean_flux_gt,2010_2020_mean_flux_err_gt,Name_x,Area,Zmin,Zmax,Zmed,Slope,Aspect,Lmax,Status,Connect,Form,TermType,Surging,Linkages,Milan_thickness_min,region,Milan_thickness_max,do_we_have_an_observation_in_middle_fifth,easting,northing,Milan_thickness_mean,0000_v,best_thickness_mean,thickness_mean_uncertainty,length_early_avg,length_early_avg_err,length_late_avg,length_late_avg_err,2000_2010_mean,2000_2010_mean_err,2000_2010_mean_count,2010_2018_mean,2010_2018_mean_err,2010_2018_mean_count,best_thickness_early,best_thickness_early_err,best_thickness_late,best_thickness_late_err,number_velocity_obs_early,number_velocity_obs_late,num_pts,GlaThiDa_observations,Tomography_observations,Russian_observations,gate_length_km,RGIid_2_1,area_below_fluxgate_early,area_below_fluxgate_middle,area_below_fluxgate_late,average_distance_from_fluxgate_to_terminus_early,average_distance_from_fluxgate_to_terminus_middle,average_distance_from_fluxgate_to_terminus_late,geometry,RGIid_original,RGIId_y,Melt_early_avg,Melt_late_avg,Melt_early_avg_err,Melt_late_avg_err,Melt_fluxes_from_regional_estimate,Melt_total_avg,Melt_total_avg_err,level_0,index,area_km2_gained_total,perimeter_gained_total,perimeter_error_gained_total,area_km2_lost_total,perimeter_lost_total,perimeter_error_lost_total,area_km2_lost_early,perimeter_lost_early,perimeter_error_lost_early,area_km2_gained_early,perimeter_gained_early,perimeter_error_gained_early,area_km2_lost_late,perimeter_lost_late,perimeter_error_lost_late,area_km2_gained_late,perimeter_gained_late,perimeter_error_gained_late,Early_satellite_id,Middle_satellite_id,Late_satellite_id,Early_date,Mid_date,Late_date,Early_date_decimal_year,Middle_date_decimal_year,Late_date_decimal_year,total_years_elapsed,early_to_middle_years_elapsed,middle_to_late_years_elapsed,net_area_change_total,net_area_change_total_error,net_area_change_per_year_total,net_area_change_per_year_total_error,net_area_change_early,net_area_change_early_error,net_area_change_per_year_early,net_area_change_per_year_early_error,net_area_change_late,net_area_change_late_error,net_area_change_per_year_late,net_area_change_per_year_late_error,average_GlaThiDa_thickness_lost_early,average_GlaThiDa_thickness_gained_early,average_GlaThiDa_thickness_lost_late,average_GlaThiDa_thickness_gained_late,average_GlaThiDa_thickness_lost_early_err,average_GlaThiDa_thickness_gained_early_err,average_GlaThiDa_thickness_lost_late_err,average_GlaThiDa_thickness_gained_late_err,average_RUS_andrey_thickness_lost_early,average_RUS_andrey_thickness_gained_early,average_RUS_andrey_thickness_lost_late,average_RUS_andrey_thickness_gained_late,average_RUS_julian_thickness_lost_early,average_RUS_julian_thickness_gained_early,average_RUS_julian_thickness_lost_late,average_RUS_julian_thickness_gained_late,average_GlaThiDa_thickness_lost_total,average_GlaThiDa_thickness_lost_total_err,average_GlaThiDa_thickness_gained_total,average_GlaThiDa_thickness_gained_total_err,average_RUS_andrey_thickness_lost_total,average_RUS_andrey_thickness_gained_total,average_RUS_julian_thickness_lost_total,average_RUS_julian_thickness_gained_total,average_Milan_thickness_lost_early,average_Milan_thickness_gained_early,average_Milan_thickness_lost_late,average_Milan_thickness_gained_late,average_Milan_thickness_lost_early_err,average_Milan_thickness_gained_early_err,average_Milan_thickness_lost_late_err,average_Milan_thickness_gained_late_err,average_Milan_thickness_lost_total,average_Milan_thickness_gained_total,average_Milan_thickness_lost_total_err,average_Milan_thickness_gained_total_err,average_tomo_thickness_lost_early,average_tomo_thickness_gained_early,average_tomo_thickness_lost_late,average_tomo_thickness_gained_late,average_tomo_thickness_lost_total,average_tomo_thickness_gained_total,thickness_for_area_lost_early,thickness_for_area_lost_early_err,thickness_for_area_gained_early,thickness_for_area_gained_early_err,thickness_for_

In [56]:
#check if any glaciers have a frontal ablation that is not equal to discharge and terminus change
import_FA = Arctic_discharge_fluxes_with_area_export
import_FA[(import_FA['2010_2020_mean_flux_gt']-import_FA['terminus_gt_change_per_year_late'])!=import_FA['Frontal_ablation_2010_to_2020_gt_per_yr_mean']][['Name_x','RGIId','area_below_fluxgate_middle',
'net_area_change_late','melt_correct_gt_late_discharge','melt_correct_gt_late_terminus_change','2000_2010_mean_flux_gt','2010_2020_mean_flux_gt','terminus_gt_change_per_year_late','terminus_gt_change_per_year_late_area_error','terminus_gt_change_per_year_early',
'Frontal_ablation_2010_to_2020_gt_per_yr_mean',
'2010_2020_mean_flux_gt_without_melt','terminus_gt_change_per_year_late_without_melt','Frontal_ablation_2010_to_2020_gt_per_yr_mean_without_melt',
'2010_2020_mean_flux_gt_without_melt_without_correction','terminus_gt_change_per_year_late_without_melt_without_correction']]


,Name_x,RGIId,area_below_fluxgate_middle,net_area_change_late,melt_correct_gt_late_discharge,melt_correct_gt_late_terminus_change,2000_2010_mean_flux_gt,2010_2020_mean_flux_gt,terminus_gt_change_per_year_late,terminus_gt_change_per_year_late_area_error,terminus_gt_change_per_year_early,Frontal_ablation_2010_to_2020_gt_per_yr_mean,2010_2020_mean_flux_gt_without_melt,terminus_gt_change_per_year_late_without_melt,Frontal_ablation_2010_to_2020_gt_per_yr_mean_without_melt,2010_2020_mean_flux_gt_without_melt_without_correction,terminus_gt_change_per_year_late_without_melt_without_correction


In [ ]:
import_FA[(import_FA['2000_2010_mean_flux_gt']-import_FA['terminus_gt_change_per_year_early'])!=import_FA['Frontal_ablation_2000_to_2010_gt_per_yr_mean']][['Name_x','2000_2010_mean_flux_gt','2000_2010_mean_flux_gt_without_melt','terminus_gt_change_per_year_early',
'thickness_for_area_gained_late','best_thickness_mean',
'Frontal_ablation_2000_to_2010_gt_per_yr_mean']]


In [ ]:
Arctic_discharge_fluxes_with_area[['Frontal_ablation_2000_to_2010_gt_per_yr_mean_without_melt','Frontal_ablation_2000_to_2010_gt_per_yr_mean_without_melt_err','Frontal_ablation_2010_to_2020_gt_per_yr_mean_without_melt','Frontal_ablation_2010_to_2020_gt_per_yr_mean_without_melt_err']]

In [ ]:
%matplotlib notebook
# (Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt'+z][i]<Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late'+z][i])
FA_plot = Arctic_discharge_fluxes_with_area[(Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt_without_melt_without_correction']<Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late_without_melt_without_correction'])&(Arctic_discharge_fluxes_with_area['2010_2020_mean_flux_gt_without_melt_without_correction']>0.04)]

barWidth = .3
bar_position = np.arange(len(FA_plot))
cap_width=3

fig,ax = plt.subplots()
fig.set_figheight(9)
fig.set_figwidth(18)
# plt.figure(figsize=(30,10))
plt.xticks(rotation='horizontal',fontsize=12)
plt.yticks(fontsize=20)
plt.ylabel('Mass change (gt/a)',fontsize=20)

plt.bar(bar_position-barWidth,FA_plot['2010_2020_mean_flux_gt_without_melt_without_correction'],width=barWidth,label='Discharge')
er_late_flux = ax.errorbar(bar_position-barWidth,FA_plot['2010_2020_mean_flux_gt_without_melt_without_correction'],yerr=FA_plot['2010_2020_mean_flux_err_gt_without_melt_without_correction'],ecolor='black',capsize=cap_width, linestyle="none")

plt.bar(bar_position,FA_plot['terminus_gt_change_per_year_late_without_melt_without_correction'],width=barWidth,label='Terminus change')
er_late_term = ax.errorbar(bar_position,FA_plot['terminus_gt_change_per_year_late_without_melt_without_correction'],yerr= FA_plot['terminus_gt_change_per_year_late_area_error_without_melt_without_correction'],ecolor='black',capsize=cap_width, linestyle="none")

plt.bar(bar_position+barWidth,-FA_plot['Frontal_ablation_2010_to_2020_gt_per_yr_mean_without_melt'],width=barWidth,label='Corrected frontal ablation')
er_late_fa = ax.errorbar(bar_position+barWidth,-FA_plot['Frontal_ablation_2010_to_2020_gt_per_yr_mean_without_melt'],yerr= FA_plot['Frontal_ablation_2010_to_2020_gt_per_yr_mean_err_without_melt'],ecolor='black',capsize=cap_width, linestyle="none")
plt.legend(loc='upper left',fontsize=20)

fig.savefig('figures/Altering_frontal_ablation_v1.jpg',format='jpg',dpi=300)   

In [ ]:
FA_plot[['Name_x','terminus_gt_change_per_year_late_without_melt_without_correction','2010_2020_mean_flux_gt_without_melt_without_correction','Frontal_ablation_2010_to_2020_gt_per_yr_mean_without_melt','Frontal_ablation_2010_to_2020_gt_per_yr_mean_err_without_melt']]

In [57]:
import_FA = Arctic_discharge_fluxes_with_area_export
for region in import_FA['Region_name'].unique():
    FA_total = (import_FA[import_FA['Region_name']==region]['Frontal_ablation_2000_to_2010_gt_per_yr_mean'].sum()+import_FA[import_FA['Region_name']==region]['Frontal_ablation_2010_to_2020_gt_per_yr_mean'].sum())/2
    FA_err = (import_FA[import_FA['Region_name']==region]['Frontal_ablation_2000_to_2010_gt_per_yr_mean_err'].sum()+import_FA[import_FA['Region_name']==region]['Frontal_ablation_2010_to_2020_gt_per_yr_mean_err'].sum())/2
    print(region,' frontal ablation 2000-2020:',round(FA_total,2),'±',round(FA_err,2))
    
FA_total = (import_FA['Frontal_ablation_2000_to_2010_gt_per_yr_mean'].sum()+import_FA['Frontal_ablation_2010_to_2020_gt_per_yr_mean'].sum())/2
FA_err = (import_FA['Frontal_ablation_2000_to_2010_gt_per_yr_mean_err'].sum()+import_FA['Frontal_ablation_2010_to_2020_gt_per_yr_mean_err'].sum())/2
print('Total frontal ablation 2000-2020:',round(FA_total,2),'±',round(FA_err,2))

Alaska  frontal ablation 2000-2020: 11.42 ± 0.4
QEI  frontal ablation 2000-2020: 4.08 ± 1.27
BBI  frontal ablation 2000-2020: 0.08 ± 0.07
Greenland  frontal ablation 2000-2020: 3.68 ± 1.79
Iceland  frontal ablation 2000-2020: 0.08 ± 0.07
Svalbard and Jan Mayen  frontal ablation 2000-2020: 12.58 ± 3.66
FJL  frontal ablation 2000-2020: 10.42 ± 5.84
NZ  frontal ablation 2000-2020: 3.37 ± 0.99
SZ  frontal ablation 2000-2020: 4.87 ± 1.12
Total frontal ablation 2000-2020: 50.59 ± 15.21


In [ ]:
import_FA['Frontal_ablation_2010_to_2020_gt_per_yr_mean'].sum()

In [46]:
import_FA

,RGIId,2000_2010_mean_flux_gt,2000_2010_mean_flux_err_gt,2010_2020_mean_flux_gt,2010_2020_mean_flux_err_gt,Name_x,Area,Zmin,Zmax,Zmed,Slope,Aspect,Lmax,Status,Connect,Form,TermType,Surging,Linkages,Milan_thickness_min,region,Milan_thickness_max,do_we_have_an_observation_in_middle_fifth,easting,northing,Milan_thickness_mean,0000_v,best_thickness_mean,thickness_mean_uncertainty,length_early_avg,length_early_avg_err,length_late_avg,length_late_avg_err,2000_2010_mean,2000_2010_mean_err,2000_2010_mean_count,2010_2018_mean,2010_2018_mean_err,2010_2018_mean_count,best_thickness_early,best_thickness_early_err,best_thickness_late,best_thickness_late_err,number_velocity_obs_early,number_velocity_obs_late,num_pts,GlaThiDa_observations,Tomography_observations,Russian_observations,gate_length_km,RGIid_2_1,area_below_fluxgate_early,area_below_fluxgate_middle,area_below_fluxgate_late,average_distance_from_fluxgate_to_terminus_early,average_distance_from_fluxgate_to_terminus_middle,average_distance_from_fluxgate_to_terminus_late,geometry,RGIid_original,RGIId_y,Melt_early_avg,Melt_late_avg,Melt_early_avg_err,Melt_late_avg_err,Melt_fluxes_from_regional_estimate,Melt_total_avg,Melt_total_avg_err,level_0,index,area_km2_gained_total,perimeter_gained_total,perimeter_error_gained_total,area_km2_lost_total,perimeter_lost_total,perimeter_error_lost_total,area_km2_lost_early,perimeter_lost_early,perimeter_error_lost_early,area_km2_gained_early,perimeter_gained_early,perimeter_error_gained_early,area_km2_lost_late,perimeter_lost_late,perimeter_error_lost_late,area_km2_gained_late,perimeter_gained_late,perimeter_error_gained_late,Early_satellite_id,Middle_satellite_id,Late_satellite_id,Early_date,Mid_date,Late_date,Early_date_decimal_year,Middle_date_decimal_year,Late_date_decimal_year,total_years_elapsed,early_to_middle_years_elapsed,middle_to_late_years_elapsed,net_area_change_total,net_area_change_total_error,net_area_change_per_year_total,net_area_change_per_year_total_error,net_area_change_early,net_area_change_early_error,net_area_change_per_year_early,net_area_change_per_year_early_error,net_area_change_late,net_area_change_late_error,net_area_change_per_year_late,net_area_change_per_year_late_error,average_GlaThiDa_thickness_lost_early,average_GlaThiDa_thickness_gained_early,average_GlaThiDa_thickness_lost_late,average_GlaThiDa_thickness_gained_late,average_GlaThiDa_thickness_lost_early_err,average_GlaThiDa_thickness_gained_early_err,average_GlaThiDa_thickness_lost_late_err,average_GlaThiDa_thickness_gained_late_err,average_RUS_andrey_thickness_lost_early,average_RUS_andrey_thickness_gained_early,average_RUS_andrey_thickness_lost_late,average_RUS_andrey_thickness_gained_late,average_RUS_julian_thickness_lost_early,average_RUS_julian_thickness_gained_early,average_RUS_julian_thickness_lost_late,average_RUS_julian_thickness_gained_late,average_GlaThiDa_thickness_lost_total,average_GlaThiDa_thickness_lost_total_err,average_GlaThiDa_thickness_gained_total,average_GlaThiDa_thickness_gained_total_err,average_RUS_andrey_thickness_lost_total,average_RUS_andrey_thickness_gained_total,average_RUS_julian_thickness_lost_total,average_RUS_julian_thickness_gained_total,average_Milan_thickness_lost_early,average_Milan_thickness_gained_early,average_Milan_thickness_lost_late,average_Milan_thickness_gained_late,average_Milan_thickness_lost_early_err,average_Milan_thickness_gained_early_err,average_Milan_thickness_lost_late_err,average_Milan_thickness_gained_late_err,average_Milan_thickness_lost_total,average_Milan_thickness_gained_total,average_Milan_thickness_lost_total_err,average_Milan_thickness_gained_total_err,average_tomo_thickness_lost_early,average_tomo_thickness_gained_early,average_tomo_thickness_lost_late,average_tomo_thickness_gained_late,average_tomo_thickness_lost_total,average_tomo_thickness_gained_total,average_Milan_thickness_gained_early_debiased,average_Milan_thickness_lost_early_debiased,average_Milan_thickness_gained_late_debiased,average_Mil

In [52]:
#sea level rise estimates
import_FA = Arctic_discharge_fluxes_with_area_export
import_FA.loc[(import_FA['Region_name']=='FJL')|(import_FA['Region_name']=='NZ')|(import_FA['Region_name']=='SZ'),'Region_name']='Russia'
# import_FA['Region_name']=import_FA['region']
import_G = {'Region_name':['Alaska', 'QEI', 'BBI', 'Greenland', 'Iceland','Svalbard and Jan Mayen', 'Russia'],
            'Mass_loss':[66.7,30.6,26.5,35.5,9.4,10.5,10.4],
            'Mass_err':[10.9,4.8,4.3,5.8,1.4,1.7,1.9]}
import_G=pd.DataFrame(data=import_G)
for region in import_FA['Region_name'].unique():
#     print('70±20% ',region,' terminus change (terminus change missing from geodetic studies) 2000-2010:',import_FA[import_FA['Region_name']==region]['terminus_gt_change_per_year_early'].sum()*.7,'±',np.sqrt((import_FA[import_FA['Region_name']==region]['terminus_gt_change_per_year_early_area_error'].sum()*0.7)**2+(import_FA[import_FA['Region_name']==region]['terminus_gt_change_per_year_early'].sum()*.2)**2))
#     print('70±20% ',region,' terminus change (terminus change missing from geodetic studies) 2010-2020:',import_FA[import_FA['Region_name']==region]['terminus_gt_change_per_year_late'].sum()*.7,'±',np.sqrt((import_FA[import_FA['Region_name']==region]['terminus_gt_change_per_year_late_area_error'].sum()*0.7)**2+(import_FA[import_FA['Region_name']==region]['terminus_gt_change_per_year_late'].sum()*.2)**2))
#     print('30±20% ',region,' terminus change (terminus change to SLR) 2000-2010:',import_FA[import_FA['Region_name']==region]['terminus_gt_change_per_year_early'].sum()*0.2,'±',np.sqrt((import_FA[import_FA['Region_name']==region]['terminus_gt_change_per_year_early_area_error'].sum()*0.2)**2+(import_FA[import_FA['Region_name']==region]['terminus_gt_change_per_year_early'].sum()*.2)**2))
#     print('30±20% ',region,' terminus change (terminus change to SLR) 2010-2020:',import_FA[import_FA['Region_name']==region]['terminus_gt_change_per_year_late'].sum()*0.2,'±',np.sqrt((import_FA[import_FA['Region_name']==region]['terminus_gt_change_per_year_late_area_error'].sum()*0.2)**2+(import_FA[import_FA['Region_name']==region]['terminus_gt_change_per_year_late'].sum()*.2)**2))

    FA_total = (import_FA[import_FA['Region_name']==region]['terminus_gt_change_per_year_early'].sum()+import_FA[import_FA['Region_name']==region]['terminus_gt_change_per_year_late'].sum())*0.7/2
    FA_err_early = np.sqrt((import_FA[import_FA['Region_name']==region]['terminus_gt_change_per_year_early_area_error'].sum()*0.7)**2+(import_FA[import_FA['Region_name']==region]['terminus_gt_change_per_year_early'].sum()*.2)**2)
    FA_err_late = np.sqrt((import_FA[import_FA['Region_name']==region]['terminus_gt_change_per_year_late_area_error'].sum()*0.7)**2+(import_FA[import_FA['Region_name']==region]['terminus_gt_change_per_year_late'].sum()*.2)**2)
    FA_err = (FA_err_early+FA_err_late)/2
    
    G_total = import_G[import_G['Region_name']==region]['Mass_loss'].sum()
    G_err = import_G[import_G['Region_name']==region]['Mass_err'].sum()
    print('70±20% ',region,' terminus change (terminus change missing from geodetic studies) 2000-2020:',round(FA_total,2),'±',round(FA_err,2))
    print(region,' corrected geodetic mass loss 2000-2020:',-round(FA_total-G_total,2),'±',round(np.sqrt(FA_err**2+G_err**2),2))
    print()
    
FA_total = (import_FA['terminus_gt_change_per_year_early'].sum()+import_FA['terminus_gt_change_per_year_late'].sum())*0.7/2
FA_err_early = np.sqrt((np.sqrt((import_FA['terminus_gt_change_per_year_early_area_error']**2).sum())*0.7)**2+((np.sqrt(import_FA['terminus_gt_change_per_year_early']**2).sum())*.2)**2)
FA_err_late = np.sqrt((np.sqrt((import_FA['terminus_gt_change_per_year_late_area_error']**2).sum())*0.7)**2+((np.sqrt(import_FA['terminus_gt_change_per_year_late']**2).sum())*.2)**2)
FA_err = (FA_err_early+FA_err_late)/2
print('70±20% all regions terminus change (terminus change missing from geodetic studies) 2000-2020:',FA_total,'±',FA_err)
G_total = import_G['Mass_loss'].sum()
G_err = np.sqrt((import_G['Mass_err']**2).sum())
print('All region corrected geodetic mass loss 2000-2020:',-round(FA_total-G_total,2),'±',round(np.sqrt(FA_err**2+G_err**2),2))
print()
    
print('70±20% total terminus change (terminus change missing from geodetic studies) 2000-2010:',import_FA['terminus_gt_change_per_year_early'].sum()*.7,'±',np.sqrt((import_FA['terminus_gt_change_per_year_early_area_error'].sum()*0.7)**2+(import_FA['terminus_gt_change_per_year_early'].sum()*.2)**2))
print('70±20% total terminus change (terminus change missing from geodetic studies) 2010-2020:',import_FA['terminus_gt_change_per_year_late'].sum()*.7,'±',np.sqrt((import_FA['terminus_gt_change_per_year_late_area_error'].sum()*0.7)**2+(import_FA['terminus_gt_change_per_year_late'].sum()*.2)**2))

print('30±20% total terminus change (terminus change to SLR) 2000-2010:',import_FA['terminus_gt_change_per_year_early'].sum()*0.2,'±',np.sqrt((import_FA['terminus_gt_change_per_year_early_area_error'].sum()*0.2)**2+(import_FA['terminus_gt_change_per_year_early'].sum()*.2)**2))
print('30±20% total terminus change (terminus change to SLR) 2010-2020:',import_FA['terminus_gt_change_per_year_late'].sum()*0.2,'±',np.sqrt((import_FA['terminus_gt_change_per_year_late_area_error'].sum()*0.2)**2+(import_FA['terminus_gt_change_per_year_late'].sum()*.2)**2))
print()
vol = (import_FA['terminus_gt_change_per_year_early'].sum()*0.2+import_FA['terminus_gt_change_per_year_late'].sum()*0.2)/2*20/362
err = (np.sqrt((import_FA['terminus_gt_change_per_year_early_area_error'].sum()*0.2)**2+(import_FA['terminus_gt_change_per_year_early'].sum()*.2)**2)+np.sqrt((import_FA['terminus_gt_change_per_year_late_area_error'].sum()*0.2)**2+(import_FA['terminus_gt_change_per_year_late'].sum()*.2)**2))/2*20/362
print('terminus change contribution to SLR 2000-2020',vol,'±',err)
print()
print('50% total terminus change (terminus change missing from geodetic studies) 2000-2010:',(import_FA['terminus_gt_change_per_year_early'].sum()+import_FA['terminus_gt_change_per_year_late'].sum())/2*.5)
print('90% total terminus change (terminus change missing from geodetic studies) 2010-2020:',(import_FA['terminus_gt_change_per_year_early'].sum()+import_FA['terminus_gt_change_per_year_late'].sum())/2*.9)


70±20%  Alaska  terminus change (terminus change missing from geodetic studies) 2000-2020: -0.35 ± 0.18
Alaska  corrected geodetic mass loss 2000-2020: 67.05 ± 10.9

70±20%  QEI  terminus change (terminus change missing from geodetic studies) 2000-2020: -1.22 ± 0.75
QEI  corrected geodetic mass loss 2000-2020: 31.82 ± 4.86

70±20%  BBI  terminus change (terminus change missing from geodetic studies) 2000-2020: -0.04 ± 0.04
BBI  corrected geodetic mass loss 2000-2020: 26.54 ± 4.3

70±20%  Greenland  terminus change (terminus change missing from geodetic studies) 2000-2020: -1.17 ± 0.91
Greenland  corrected geodetic mass loss 2000-2020: 36.67 ± 5.87

70±20%  Iceland  terminus change (terminus change missing from geodetic studies) 2000-2020: -0.01 ± 0.01
Iceland  corrected geodetic mass loss 2000-2020: 9.41 ± 1.4

70±20%  Svalbard and Jan Mayen  terminus change (terminus change missing from geodetic studies) 2000-2020: -1.8 ± 1.52
Svalbard and Jan Mayen  corrected geodetic mass loss 2000-

In [43]:
FA_err

5.569710709757131

In [ ]:
#glaciers to add to geodetic without melt
import_FA = Arctic_discharge_fluxes_with_area_export

print('70±20% Terminus change (terminus change missing from geodetic studies) 2000-2010:',import_FA['terminus_gt_change_per_year_early_without_melt'].sum()*0.7,'±',np.sqrt((import_FA['terminus_gt_change_per_year_early_area_error_without_melt'].sum()*0.7)**2+(import_FA['terminus_gt_change_per_year_early_without_melt'].sum()*.2)**2))
print('70±20% Terminus change (terminus change missing from geodetic studies) 2010-2020:',import_FA['terminus_gt_change_per_year_late_without_melt'].sum()*0.7,'±',np.sqrt((import_FA['terminus_gt_change_per_year_late_area_error_without_melt'].sum()*0.7)**2+(import_FA['terminus_gt_change_per_year_late_without_melt'].sum()*.2)**2))



In [ ]:
#get frontal ablation numbers for Columbia Glacier, Alaska
import_FA = Arctic_discharge_fluxes_with_area_export[Arctic_discharge_fluxes_with_area_export['RGIId']=='RGI60-01.10689'].reset_index(drop=True)

print('Terminus change 2000-2010:',import_FA['terminus_gt_change_per_year_early_without_melt'][0],'±',import_FA['terminus_gt_change_per_year_early_area_error_without_melt'][0])
print('Terminus change 2010-2020:',import_FA['terminus_gt_change_per_year_late_without_melt'][0],'±',import_FA['terminus_gt_change_per_year_late_area_error_without_melt'][0])
print()
print('70±20% Terminus change 2000-2010:',import_FA['terminus_gt_change_per_year_early_without_melt'][0]*0.7,'±',np.sqrt((import_FA['terminus_gt_change_per_year_early_area_error_without_melt'][0]*0.7)**2+(import_FA['terminus_gt_change_per_year_early_without_melt'][0]*.2)**2))
print('70±20% Terminus change 2010-2020:',import_FA['terminus_gt_change_per_year_late_without_melt'][0]*0.7,'±',np.sqrt((import_FA['terminus_gt_change_per_year_late_area_error_without_melt'][0]*0.7)**2+(import_FA['terminus_gt_change_per_year_late_without_melt'][0]*.2)**2))
print()
print('Frontal ablation 2000-2010:',import_FA['terminus_gt_change_per_year_early_without_melt'][0]+import_FA['2000_2010_mean_flux_gt_without_melt'][0],'±',np.sqrt(import_FA['terminus_gt_change_per_year_early_area_error_without_melt'][0]**2+import_FA['2000_2010_mean_flux_err_gt_without_melt'][0]**2))
print('Frontal ablation 2010-2020:',import_FA['terminus_gt_change_per_year_late_without_melt'][0]+import_FA['2010_2020_mean_flux_gt_without_melt'][0],'±',np.sqrt(import_FA['terminus_gt_change_per_year_late_area_error_without_melt'][0]**2+import_FA['2010_2020_mean_flux_err_gt_without_melt'][0]**2))

In [ ]:
#calculate the area change of every glacier from 2000 to 2020
#read in data files
termini_2000 = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/termini_2000.geojson')
termini_2010 = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/termini_2010.geojson')
termini_2020 = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/termini_2020.geojson')

termini_2000=termini_2000[termini_2000.geometry.type=='Polygon']
termini_2010=termini_2010[termini_2010.geometry.type=='Polygon']
termini_2020=termini_2020[termini_2020.geometry.type=='Polygon']

termini_2000['geometry'] = termini_2000['geometry'].buffer(0)
termini_2010['geometry'] = termini_2010['geometry'].buffer(0)
termini_2020['geometry'] = termini_2020['geometry'].buffer(0)

termini_2000['time_period']='2000'
termini_2010['time_period']='2010'
termini_2020['time_period']='2020'

termini=pd.concat([termini_2000,termini_2010,termini_2020])

boxes = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/Glacier_area_change_boxes.gpkg')
RGI = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/RGI_Arctic_marine_terminating_glacier_outlines.gpkg')
                     
RGI['geometry'] = RGI.geometry.buffer(0)
RGI['RGIId']=RGI['RGIid_2']

#Add RGI and satellite image date attributes to the boxes
boxes_attributed = gpd.sjoin(boxes, RGI, how='inner')
boxes_attributed = boxes_attributed.drop(columns=['index_right'])
boxes_attributed = boxes_attributed.reset_index()

In [ ]:
termini_2000 = gpd.sjoin(termini_2000[['geometry','Landsat_ID','time_period']], boxes_attributed[['geometry','RGIId']], how='left')
termini_2010 = gpd.sjoin(termini_2010[['geometry','Landsat_ID','time_period']], boxes_attributed[['geometry','RGIId']], how='left')
termini_2020 = gpd.sjoin(termini_2020[['geometry','Landsat_ID','time_period']], boxes_attributed[['geometry','RGIId']], how='left')

In [ ]:
gpd.clip(termini_2020[i:i+1].exterior,boxes_attributed)

In [ ]:
for i in range(0,len(termini_2020)):
    print(i)
    termini_2020.loc[i,'geometry'] = gpd.clip(termini_2020[i:i+1].exterior,boxes_attributed)
#     print(termini_2020[i:i+1])
termini_2020.to_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/calving_front_line_2020_'+date_string+'.geojson', driver='GeoJSON')

In [ ]:
termini_2000